In [ ]:
import cv2
import numpy as np
import pandas as pd
import mediapipe as mp
import matplotlib.pyplot as plt
import time
import os
import scipy  #
from scipy.interpolate import interp1d  # 프레임 보간에 사용
from scipy.signal import find_peaks
import traceback
import torch.nn.functional as F
import subprocess
import math


# --- 기능 활성화 플래그 ---
DEBUG_MODE = True
USE_DISTANCE_SMOOTHING = True
USE_VELOCITY_ACCELERATION_CONDITIONS = False
USE_FRAME_INTERPOLATION = True
USE_BALL_VELOCITY_CHANGE_DETECTION = False
USE_PREVIOUS_BALL_INFO_TRACKING = True
USE_ENHANCED_FOOT_MODEL = True
USE_KICKING_FOOT_KINEMATICS_CONDITION = False
USE_BACKSWING_APPROACH_PATTERN = True
USE_BALL_SEGMENTATION = False
USE_BALL_SHRINK_DETECTION = False
USE_PREDICTIVE_TRACKING = True
USE_DIFFERENTIAL_FOOT_SPEED_CHECK = True
USE_INTER_FOOT_DISTANCE_CHECK = True
USE_OCCLUSION_HANDLING_PAPER_LOGIC = True
CALCULATE_SUPPORTING_FOOT_STABILITY = True
USE_PLAYER_ANGLE_COMPARISON = True

# --- 영상회전 ---
# manual_rotate_code = cv2.ROTATE_90_COUNTERCLOCKWISE
manual_rotate_code = None

# --- 하이퍼파라미터 ---
REAL_BALL_DIAMETER_CM = 22.0
DEFAULT_PIXEL_TO_CM_SCALE = 0.85
YOLO_CONF_THRESHOLD = 0.15
YOLO_IOU_THRESHOLD = 0.45
MP_POSE_MIN_DETECTION_CONFIDENCE = 0.3
MP_POSE_MIN_TRACKING_CONFIDENCE = 0.3
BALL_SELECTION_ALPHA_FOOT_DIST = 0.008
BALL_TRACKING_DIST_WEIGHT = 1.5
BALL_TRACKING_RADIUS_WEIGHT = 1.0
BALL_TRACKING_CONF_WEIGHT = -20.0
BALL_TRACKING_MAX_RADIUS_DIFF_RATIO = 0.7
BALL_TRACKING_MAX_DIST_FROM_PREV = 200
BETA_PROXIMITY_SCALE = 1.1  # original = 1.5
KICK_FOOT_APPROACH_VEL_THRESHOLD = 0.05
MIN_KICKING_FOOT_SPEED_AT_IMPACT = 2.0
MAX_KICKING_FOOT_ACCEL_AT_IMPACT = 15.0
BACKSWING_PEAK_PROMINENCE = 4
BACKSWING_MIN_PEAK_DISTANCE_FRAMES = 8
BACKSWING_SEARCH_WINDOW_RATIO = 0.8
IOU_CONTACT_THRESHOLD = 0.15  # original = 0.08
SMOOTHING_WINDOW_SIZE = 3
INTERPOLATION_WINDOW_RADIUS = 7
INTERPOLATION_DENSITY_FACTOR = 50
FOOT_BOX_RADIUS = 40
SAMPLING_RATE_TARGET_FRAMES = 30
FOOT_WIDTH_SCALE = 0.4
INSTEP_FORWARD_SCALE = 0.7
SIDE_FOOT_FORWARD_SCALE = 0.6
MASK_ASSOCIATION_IOU_THRESHOLD = 0.3
BACKSWING_MIN_FRAMES_FROM_START = 5
BALL_SIZE_SMOOTHING_WINDOW = 3
BALL_RADIUS_SHRINK_THRESHOLD_PX = 1.0
BALL_SHRINK_RECOVERY_FACTOR = 0.5
MAX_CONSECUTIVE_PREDICTIONS = 2
PREDICTED_BALL_CONFIDENCE_DECAY = 0.6
KICK_TO_SUPPORT_SPEED_RATIO_THRESHOLD = 2.5
MAX_INTER_FOOT_DISTANCE_AT_IMPACT = 200
INITIAL_FRAMES_FOR_BALL_SIZE_EST = 30
BALL_SIZE_EST_MIN_CONFIDENCE = 0.4
BALL_SIZE_EST_OUTLIER_PERCENTILE = 0.1
OCCLUSION_IOU_THRESHOLD = 0.1
OCCLUSION_MIN_NEW_BALL_CONF = 0.25
FOOT_SPEED_SCORE_WEIGHT = -0.8
FOOT_SPEED_PEAK_BONUS = -10.0
FOOT_SPEED_PEAK_WINDOW = 1
MINIMUM_DISTANCE_WINDOW = 2
ABSOLUTE_DISTANCE_THRESHOLD = 12.0  # original =30.0
IMPACT_AV_CHANGE_BONUS = -10.0
SUPPORTING_FOOT_STABILITY_WINDOW_SIZE = 5
MAX_FRAMES_FOR_MAX_BALL_SPEED_SEARCH = 30
SEGMENTATION_VIS_COLOR = (255, 150, 150, 150)
FOOT_VIS_COLOR = (150, 255, 150, 120)

# --- 속도 단위 변환 계수 ---
CM_S_TO_KM_H = 0.036

# --- 전역 변수 초기화 (점수 결과 저장용) ---
score_result_global = None
analysis_dataframe_global = pd.DataFrame()

# --- 사용자 입력 및 파일 경로 설정 ---
video_path = None
main_kicking_foot_preference = None

#  1. 여기에 분석하고 싶은 영상 파일 경로를 직접 입력하세요.
video_path = "/Users/heojaeeun/Documents/soccer-kick-analyzer/tests/custom_test_data/test1(insde_bad).mp4"
# video_path = "/Users/heojaeeun/Documents/soccer-kick-analyzer/tests/data/new3-2.mov"
# 2. 분석할 주발을 선택하세요. ('left', 'right', 'auto')
main_kicking_foot_preference = "auto"

# --- 아래는 파일 존재 여부만 확인하는 코드이므로 수정할 필요 없습니다. ---
print(f"테스트 영상: '{video_path}'")
print(f"주발 설정: '{main_kicking_foot_preference}'")

if not os.path.exists(video_path):
    error_message = f"에러: '{video_path}' 파일을 찾을 수 없습니다. 파일 이름이나 경로를 확인해주세요."
    print(error_message)
    # 노트북에서는 에러를 명확히 보여주는 것이 좋으므로 raise를 사용합니다.
    raise FileNotFoundError(error_message)
else:
    print("영상 파일을 성공적으로 찾았습니다. 분석을 시작합니다.")

# Ultralytics YOLO 임포트
try:
    from ultralytics import YOLO

    print("YOLO imported successfully.")
except ImportError:
    print("ultralytics package not found. Please install it: pip install ultralytics")
    exit()
except Exception as e:
    print(f"Error importing YOLO: {e}")
    exit()

# --- 출력 비디오 경로 설정 ---
video_name_no_ext = os.path.splitext(os.path.basename(video_path))[0]
timestamp = time.strftime("%Y%m%d-%H%M%S")
output_video_filename = f"{video_name_no_ext}_analysis_result_{timestamp}.mp4"
output_video_path = os.path.join(os.getcwd(), output_video_filename)
print(f"Result video will be saved to: {output_video_path}")

# ----------------------------------------- 유틸리티 함수 ---------------------------------------------

REFERENCE_FPS = 30.0  # 속도 정규화를 위한 기준 FPS (30.0 또는 60.0 추천)


# ⚽️ 영상에서 측정한 속도를 실제 단위(km/h)로 바꾸되, FPS 차이도 고려해서 정규화해주는 함수
def convert_speed_to_kmh_normalized(px_per_frame, pixel_scale, current_effective_fps):
    """
    측정된 px/frame 속도를 기준 FPS로 정규화하고, 물리적 단위(km/h)로 변환합니다.
    이것이 바로 '소프트웨어적 FPS 정규화'의 핵심 구현입니다.

    Args:
        px_per_frame (float): 현재 유효 FPS에서 측정된 프레임당 픽셀 이동량
        pixel_scale (float): cm/pixel 변환 계수
        current_effective_fps (float): 분석에 실제 사용된 유효 FPS (원본FPS / 샘플링 간격)

    Returns:
        float: km/h 단위의 최종 보정된 속도
    """
    if current_effective_fps <= 1e-6 or pd.isna(px_per_frame):
        return 0.0

    # 1. 기준 FPS(REFERENCE_FPS)로 속도를 정규화
    # 공식: 측정값 * (실제 분석 FPS / 기준 FPS) -> 모든 영상의 속도를 기준 FPS 영상에서 측정한 것처럼 환산
    normalized_px_per_reference_frame = px_per_frame * (
        current_effective_fps / REFERENCE_FPS
    )

    # 2. 정규화된 속도(px/기준프레임)를 cm/s로 변환
    # 이제 속도는 기준 FPS(REFERENCE_FPS)에 대한 값이므로, 계산 시에도 기준 FPS를 사용
    speed_cm_per_sec = normalized_px_per_reference_frame * pixel_scale * REFERENCE_FPS

    # 3. cm/s를 km/h로 변환 (CM_S_TO_KM_H = 0.036)
    speed_km_per_hour = speed_cm_per_sec * CM_S_TO_KM_H

    return speed_km_per_hour


# ⚽️ 공의 픽셀 반지름과 실제 지름(cm)을 바탕으로,
# 현재 영상의 픽셀-센티미터 환산 계수 (cm/pixel)를 동적으로 추정하는 함수 (기초)
def calculate_dynamic_pixel_to_cm_scale(ball_radius_px, real_ball_diameter_cm_param):
    if ball_radius_px and ball_radius_px > 1e-3 and real_ball_diameter_cm_param > 0:
        detected_ball_diameter_px = ball_radius_px * 2.0
        if detected_ball_diameter_px > 1e-3:
            return real_ball_diameter_cm_param / detected_ball_diameter_px
    return None


# [OPTIONAL] 추후에 발 기반 스케일 추정을 도입해 향후 공 검출 실패 시 fallback용 사용해 보자 !
# if pixel_to_cm is None and use_foot_as_fallback:
#     foot_scale = calculate_pixel_to_cm_from_foot(pose_landmarks, side, frame_shape)
#     pixel_to_cm = get_safe_pixel_to_cm_scale(foot_scale)


# ⚽️ 비정상적인 픽셀-센티미터 스케일 값을 감지하고 기본값으로 대체
#  경고를 발생시켜 사용자에게 결과 왜곡 가능성을 알리는 함수 (보완)
import warnings


def get_safe_pixel_to_cm_scale(scale_val):
    if scale_val is None or pd.isna(scale_val) or scale_val <= 1e-6:
        warnings.warn(
            f"[경고] scale_val={scale_val} → 기본값({DEFAULT_PIXEL_TO_CM_SCALE})으로 대체됨. 결과 왜곡 주의!",
            RuntimeWarning,
        )
        return DEFAULT_PIXEL_TO_CM_SCALE
    return scale_val


# ⚽️ 벡터가 가리키는 방향을 기준으로 공 표면 어디에 임팩트가 있었는지 시각적 위치 분류를 해주는 함수
# 재은아 이거 보정해야한다...
def get_ball_contact_region(vec_x, vec_y):
    angle = np.degrees(np.arctan2(vec_y, vec_x))
    if -22.5 <= angle < 22.5:
        return "Right"
    if 22.5 <= angle < 67.5:
        return "Top-Right"
    if 67.5 <= angle < 112.5:
        return "Top"
    if 112.5 <= angle < 157.5:
        return "Top-Left"
    if 157.5 <= angle or angle < -157.5:
        return "Left"
    if -157.5 <= angle < -112.5:
        return "Bottom-Left"
    if -112.5 <= angle < -67.5:
        return "Bottom"
    if -67.5 <= angle < -22.5:
        return "Bottom-Right"
    return "Center"


# ⚽️ 두 개의 사각형(바운딩 박스) box1, box2 사이의 IoU (교집합/합집합 비율)를 계산하는 함수
# 의의: 단일 공을 계속 추적하기 위함
# 의문...굳이 필요?
def calculate_iou(box1, box2):
    if not (box1 and box2 and len(box1) == 4 and len(box2) == 4):
        return 0.0
    x1i = max(box1[0], box2[0])
    y1i = max(box1[1], box2[1])
    x2i = min(box1[2], box2[2])
    y2i = min(box1[3], box2[3])
    inter_area = max(0, x2i - x1i) * max(0, y2i - y1i)
    if inter_area == 0:
        return 0.0
    area1 = (box1[2] - box1[0]) * (box1[3] - box1[1])
    area2 = (box2[2] - box2[0]) * (box2[3] - box2[1])
    union_area = area1 + area2 - inter_area
    if union_area < 1e-6:
        return 0.0
    return inter_area / union_area


# ⚽️ MediaPipe Pose의 결과에서 특정 관절(lm_enum)의 프레임 내 좌표(x, y)를 추출한다.
# 조건: 해당 관절의 감지 신뢰도가 충분히 높을 때만 반환한다.
def get_specific_landmark_position(landmarks, lm_enum, frame_shape):
    if not landmarks or not landmarks.landmark:
        return None
    if not (
        isinstance(lm_enum, mp.solutions.pose.PoseLandmark)
        and 0 <= lm_enum.value < len(landmarks.landmark)
    ):
        return None
    lm = landmarks.landmark[lm_enum.value]
    if lm.visibility < MP_POSE_MIN_DETECTION_CONFIDENCE:
        return None
    return int(lm.x * frame_shape[1]), int(lm.y * frame_shape[0])


# ⚽️ 지정된 관절(lm_enum)의 (x, y, z) 좌표를 반환
# visibility와 z값의 유효성을 함께 검사하며, 조건을 만족하지 않으면 None을 반환
def get_landmark_with_z(landmarks, lm_enum, frame_shape, z_valid_range=(-1.0, 1.0)):
    if not landmarks or not landmarks.landmark:
        return None
    if not (
        isinstance(lm_enum, mp_pose.PoseLandmark)
        and 0 <= lm_enum.value < len(landmarks.landmark)
    ):
        return None
    lm = landmarks.landmark[lm_enum.value]
    if lm.visibility < MP_POSE_MIN_DETECTION_CONFIDENCE:
        return None
    if not (z_valid_range[0] <= lm.z <= z_valid_range[1]):
        return None
    return (int(lm.x * frame_shape[1]), int(lm.y * frame_shape[0]), lm.z)


# ⚽️ MediaPipe 관절 정보를 기반으로 발의 방향, 길이, 안/바깥쪽 위치 등을 정밀 추정하는 함수
def enhance_foot_model(pose_landmarks, side, frame_shape):
    # 설정값
    Z_VALID_RANGE = (-1.0, 1.0)
    USE_HEEL_CORRECTION = True

    # 기본 landmark 선택
    ankle_enum = (
        mp_pose.PoseLandmark.RIGHT_ANKLE
        if side == "right"
        else mp_pose.PoseLandmark.LEFT_ANKLE
    )
    toe_enum = (
        mp_pose.PoseLandmark.RIGHT_FOOT_INDEX
        if side == "right"
        else mp_pose.PoseLandmark.LEFT_FOOT_INDEX
    )

    # z 유효성 포함하여 좌표 + z값 추출
    ankle = get_landmark_with_z(pose_landmarks, ankle_enum, frame_shape, Z_VALID_RANGE)
    toe = get_landmark_with_z(pose_landmarks, toe_enum, frame_shape, Z_VALID_RANGE)

    base_px = {}
    if ankle:
        base_px["ankle"] = ankle[:2]
        base_px["ankle_z"] = ankle[2]
    if toe:
        base_px["toe"] = toe[:2]
        base_px["toe_z"] = toe[2]
    if "ankle" not in base_px or "toe" not in base_px:
        return base_px

    ax, ay = base_px["ankle"]
    tx, ty = base_px["toe"]
    ankle_coord = np.array([ax, ay])
    v = np.array([tx - ax, ty - ay])
    nv = np.linalg.norm(v)

    if nv < 1e-6:
        base_px.update({"direction_vector": np.array([0, 0]), "foot_length_px": 0.0})
        return base_px

    direction_vector = v / nv

    # 🔄 발끝 보정 벡터 평균화 (heel, pinky/index 사용)
    if USE_HEEL_CORRECTION:
        heel_enum = (
            mp_pose.PoseLandmark.RIGHT_HEEL
            if side == "right"
            else mp_pose.PoseLandmark.LEFT_HEEL
        )
        pinky_enum = (
            mp_pose.PoseLandmark.RIGHT_FOOT_INDEX + 1
            if side == "right"
            else mp_pose.PoseLandmark.LEFT_FOOT_INDEX + 1
        )
        index_enum = toe_enum  # 기본 toe

        heel = get_landmark_with_z(
            pose_landmarks, heel_enum, frame_shape, Z_VALID_RANGE
        )
        pinky = get_landmark_with_z(
            pose_landmarks, pinky_enum, frame_shape, Z_VALID_RANGE
        )

        if heel and pinky:
            pinky_vec = np.array(pinky[:2]) - np.array(heel[:2])
            pinky_vec /= np.linalg.norm(pinky_vec) + 1e-6
            direction_vector = 0.7 * direction_vector + 0.3 * pinky_vec
            direction_vector /= np.linalg.norm(direction_vector)

    # 길이 및 수직 벡터 계산
    foot_length_px = nv
    foot_width_px = FOOT_WIDTH_SCALE * foot_length_px
    perp_vector = np.array([-direction_vector[1], direction_vector[0]])

    # 발 등, 안쪽/바깥쪽 좌표 계산
    instep_coord = ankle_coord + INSTEP_FORWARD_SCALE * v
    side_base_coord = ankle_coord + SIDE_FOOT_FORWARD_SCALE * v

    if side == "right":
        inside_coord = side_base_coord - perp_vector * foot_width_px / 2
        outside_coord = side_base_coord + perp_vector * foot_width_px / 2
    else:
        inside_coord = side_base_coord + perp_vector * foot_width_px / 2
        outside_coord = side_base_coord - perp_vector * foot_width_px / 2

    base_px.update(
        {
            "direction_vector": direction_vector,
            "foot_length_px": foot_length_px,
            "instep": tuple(map(int, instep_coord)),
            "inside": tuple(map(int, inside_coord)),
            "outside": tuple(map(int, outside_coord)),
            "instep_z": (toe[2] + ankle[2]) / 2 if "toe_z" in base_px else None,
        }
    )

    return base_px


# ⚽️ 주어진 다각형 영역을 이미지 위에 반투명한 색으로 마스킹하여 시각적으로 표시하는 시각화 함수
def draw_segmentation_mask(image, mask_points, color=(0, 255, 0, 100)):
    if mask_points is not None and len(mask_points) > 0:
        overlay = image.copy()
        alpha_channel = (
            color[3] / 255.0
            if len(color) == 4 and color[3] > 1
            else (color[3] if len(color) == 4 and 0 <= color[3] <= 1 else 0.4)
        )
        cv2.fillPoly(overlay, [mask_points.astype(np.int32)], color[:3])
        cv2.addWeighted(overlay, alpha_channel, image, 1 - alpha_channel, 0, image)


# ⚽️ MediaPipe Pose landmark들을 바탕으로 “발의 외곽 다각형(Polygon)“을 구성하는 함수
def get_foot_polygon(pose_landmarks, side, frame_shape, enhanced_foot_parts_input=None):
    if not pose_landmarks:
        return None
    foot_points_for_hull = []
    ankle_lm = (
        mp_pose.PoseLandmark.LEFT_ANKLE
        if side == "left"
        else mp_pose.PoseLandmark.RIGHT_ANKLE
    )
    heel_lm = (
        mp_pose.PoseLandmark.LEFT_HEEL
        if side == "left"
        else mp_pose.PoseLandmark.RIGHT_HEEL
    )
    foot_index_lm = (
        mp_pose.PoseLandmark.LEFT_FOOT_INDEX
        if side == "left"
        else mp_pose.PoseLandmark.RIGHT_FOOT_INDEX
    )
    ankle_pos = get_specific_landmark_position(pose_landmarks, ankle_lm, frame_shape)
    heel_pos = get_specific_landmark_position(pose_landmarks, heel_lm, frame_shape)
    foot_index_pos = get_specific_landmark_position(
        pose_landmarks, foot_index_lm, frame_shape
    )
    if ankle_pos:
        foot_points_for_hull.append(ankle_pos)
    if heel_pos:
        foot_points_for_hull.append(heel_pos)
    if foot_index_pos:
        foot_points_for_hull.append(foot_index_pos)
    enhanced_parts = enhanced_foot_parts_input
    if enhanced_parts is None and USE_ENHANCED_FOOT_MODEL:
        enhanced_parts = enhance_foot_model(pose_landmarks, side, frame_shape)
    if enhanced_parts:
        for part_name, pos_px in enhanced_parts.items():
            if (
                part_name in ["instep", "inside", "outside", "toe", "ankle"]
                and isinstance(pos_px, tuple)
                and len(pos_px) == 2
            ):
                if pos_px not in foot_points_for_hull:
                    foot_points_for_hull.append(pos_px)
    if len(foot_points_for_hull) >= 3:
        try:
            return cv2.convexHull(np.array(foot_points_for_hull, dtype=np.int32))
        except Exception:
            if len(foot_points_for_hull) >= 3:
                return np.array(foot_points_for_hull, dtype=np.int32).reshape(
                    (-1, 1, 2)
                )
    return None


# ⚽️ 공이 가려졌다가 다시 등장했는지를 판단하는 로직
# → 이전 프레임 후보군들과의 IOU(겹침 비율)가 낮고, confidence가 높은 새 후보가 있다면
# → 이를 "새롭게 나타난 공"으로 간주하여 반환함
# “움직이는 공”과 “가만히 있는 공”을 구별하지 못하는 근본적인 한계
# def handle_occlusion_with_new_appearance(current_candidates, previous_candidates_list):
#     if not current_candidates or not previous_candidates_list:
#         return None
#     newly_appeared_candidates = []
#     for curr_cand in current_candidates:
#         max_iou_with_prev = 0.0
#         for prev_cand in previous_candidates_list:
#             iou = calculate_iou(curr_cand["box"], prev_cand["box"])
#             if iou > max_iou_with_prev:
#                 max_iou_with_prev = iou
#         if max_iou_with_prev < OCCLUSION_IOU_THRESHOLD:
#             if curr_cand["confidence"] > OCCLUSION_MIN_NEW_BALL_CONF:
#                 newly_appeared_candidates.append(curr_cand)
#     if newly_appeared_candidates:
#         best_new_candidate = max(
#             newly_appeared_candidates, key=lambda x_new: x_new["confidence"]
#         )
#         print(
#             f"✨ Occlusion Handler: Found new candidate (possibly recovered)! Center: {best_new_candidate['center']}, Conf: {best_new_candidate['confidence']:.2f} ✨"
#         )
#         return best_new_candidate
#     return None


# ⚽️ (new)위의  handle_occlusion_with_new_appearance 의 대안 함수
def find_valid_ball_candidate(
    current_candidates,
    previous_position,
    previous_velocity,
    min_confidence=0.5,
    max_position_shift=150,  # px
    min_motion_similarity=0.7,  # cosine similarity
):
    """
    공이 가려졌다 다시 나타났을 때 유효한 후보를 선별하는 함수.
    속도 방향, 위치 변화, confidence 등을 모두 고려함.
    """

    def cosine_similarity(v1, v2):
        norm1 = np.linalg.norm(v1)
        norm2 = np.linalg.norm(v2)
        if norm1 == 0 or norm2 == 0:
            return 0.0
        return np.dot(v1, v2) / (norm1 * norm2)

    best_candidate = None
    best_score = -1

    for cand in current_candidates:
        center = np.array(cand["center"])
        confidence = cand.get("confidence", 0.0)

        # 위치 변화량
        displacement = center - previous_position
        shift = np.linalg.norm(displacement)

        # 이동 방향 유사도
        direction_similarity = cosine_similarity(displacement, previous_velocity)

        # 기준에 맞는지 판단
        if (
            confidence >= min_confidence
            and shift <= max_position_shift
            and direction_similarity >= min_motion_similarity
        ):
            # 정합성 점수 = 방향 유사도 * confidence
            score = direction_similarity * confidence
            if score > best_score:
                best_score = score
                best_candidate = cand

    return best_candidate  # 없으면 None 반환


# ⚽️ 현재 인덱스 기준 주어진 반경 내에서 해당 값이 최소값인지 확인
def is_distance_minimum_in_window(
    analysis_df_arg, current_idx, window_radius, col_name="distance_smoothed"
):
    start_idx = max(0, current_idx - window_radius)
    end_idx = min(len(analysis_df_arg) - 1, current_idx + window_radius)
    if not (0 <= current_idx < len(analysis_df_arg)):
        return False
    current_dist_val = analysis_df_arg.loc[current_idx, col_name]
    if pd.isna(current_dist_val):
        return False
    window_values = analysis_df_arg.loc[start_idx:end_idx, col_name].dropna()
    if window_values.empty:
        return True
    return current_dist_val <= window_values.min()


# ⚽️ 킥 분석에서 추출된 후보 프레임들을 시각적으로 검토할 수 있도록 전체 그림, 포즈, 공, 발 부위를 그려주는 함수
def visualize_candidates(
    frames_list, poses_list, candidates_list, ball_detections_list_for_viz
):
    n = len(candidates_list)
    if n == 0:
        print("No candidate frames to visualize.")
        return
    cols = min(5, n)
    rows = (n + cols - 1) // cols
    plt.figure(figsize=(4 * cols, 3 * rows))
    for i_cand, cand_dict in enumerate(candidates_list):
        if "data_row" not in cand_dict or cand_dict["data_row"] is None:
            continue
        current_data_row = cand_dict["data_row"]
        original_idx_val = None
        if isinstance(current_data_row, pd.Series):
            original_idx_val = current_data_row.get("original_idx")
        elif isinstance(current_data_row, dict):
            original_idx_val = current_data_row.get("original_idx")
        if original_idx_val is None:
            continue
        try:
            idx = int(original_idx_val)
        except ValueError:
            continue
        if not (
            0 <= idx < len(frames_list)
            and 0 <= idx < len(poses_list)
            and 0 <= idx < len(ball_detections_list_for_viz)
        ):
            continue
        img_to_show = frames_list[idx].copy()
        current_pose_data = poses_list[idx]
        if current_pose_data:
            mp.solutions.drawing_utils.draw_landmarks(
                img_to_show,
                current_pose_data,
                mp_pose.POSE_CONNECTIONS,
                mp.solutions.drawing_utils.DrawingSpec(
                    color=(0, 255, 0), thickness=2, circle_radius=2
                ),
                mp.solutions.drawing_utils.DrawingSpec(color=(255, 0, 0), thickness=2),
            )
            kicking_foot_cand = None
            if isinstance(current_data_row, pd.Series):
                kicking_foot_cand = current_data_row.get("kicking_foot")
            elif isinstance(current_data_row, dict):
                kicking_foot_cand = current_data_row.get("kicking_foot")
            if kicking_foot_cand and kicking_foot_cand != "N/A":
                foot_poly_cand = get_foot_polygon(
                    current_pose_data, kicking_foot_cand, img_to_show.shape
                )
                if foot_poly_cand is not None:
                    draw_segmentation_mask(
                        img_to_show, foot_poly_cand, color=FOOT_VIS_COLOR
                    )
        ball_data_for_viz = ball_detections_list_for_viz[idx]
        if ball_data_for_viz:
            is_predicted_ball = ball_data_for_viz.get("is_predicted", False)
            mask_points_viz = ball_data_for_viz.get("mask_points")
            if (
                USE_BALL_SEGMENTATION
                and mask_points_viz is not None
                and not is_predicted_ball
            ):
                draw_segmentation_mask(
                    img_to_show, mask_points_viz, color=SEGMENTATION_VIS_COLOR
                )
            else:
                ball_center_val = ball_data_for_viz.get("ball_center")
                ball_radius_val = ball_data_for_viz.get("ball_radius")
                if (
                    ball_center_val is not None
                    and ball_radius_val is not None
                    and ball_radius_val > 0
                ):
                    ball_color = (255, 165, 0) if is_predicted_ball else (0, 0, 255)
                    cv2.circle(
                        img_to_show,
                        tuple(map(int, ball_center_val)),
                        int(ball_radius_val),
                        ball_color,
                        2,
                    )
        if DEBUG_MODE:
            plt.subplot(rows, cols, i_cand + 1)
            plt.imshow(cv2.cvtColor(img_to_show, cv2.COLOR_BGR2RGB))
            score_val = cand_dict.get("score", float("nan"))
            title_text = f"Idx:{idx}, Score:{score_val:.1f}"
            is_ball_pred_title = False
            if isinstance(current_data_row, pd.Series):
                is_ball_pred_title = current_data_row.get("is_ball_predicted", False)
            elif isinstance(current_data_row, dict):
                is_ball_pred_title = current_data_row.get("is_ball_predicted", False)
            if is_ball_pred_title:
                title_text += " (BallPred)"
            if cand_dict.get("is_peak", False):
                title_text += " (SpdPk!)"
            if cand_dict.get("is_current_frame_av_impact", False):
                title_text += " (AVImp!)"
            plt.title(title_text)
            plt.axis("off")


if DEBUG_MODE:
    plt.tight_layout()
    plt.show()


# ⚽️ Kalman Filter 기반 공 위치 예측 클래스
class KalmanBallTracker:
    def __init__(self):
        # 상태 벡터: [x, y, vx, vy]
        self.x = np.zeros((4, 1))

        # 상태 전이 행렬 (등속운동 가정)
        self.F = np.array(
            [
                [1, 0, 1, 0],
                [0, 1, 0, 1],
                [0, 0, 1, 0],
                [0, 0, 0, 1],
            ]
        )

        # 측정 행렬 (위치만 측정 가능)
        self.H = np.array(
            [
                [1, 0, 0, 0],
                [0, 1, 0, 0],
            ]
        )

        self.P = np.eye(4) * 500  # 상태 추정 오차 공분산
        self.R = np.eye(2) * 25  # 측정 오차 공분산
        self.Q = np.eye(4) * 1e-2  # 프로세스 잡음 공분산

    def reset(self, initial_position):
        self.x = np.array(
            [
                [initial_position[0]],
                [initial_position[1]],
                [0.0],
                [0.0],
            ]
        )
        self.P = np.eye(4) * 500

    def predict(self):
        self.x = np.dot(self.F, self.x)
        self.P = np.dot(self.F, np.dot(self.P, self.F.T)) + self.Q
        return tuple(self.x[:2].flatten())

    def update(self, z):
        z = np.reshape(z, (2, 1))
        y = z - np.dot(self.H, self.x)
        S = np.dot(self.H, np.dot(self.P, self.H.T)) + self.R
        K = np.dot(np.dot(self.P, self.H.T), np.linalg.inv(S))
        self.x = self.x + np.dot(K, y)
        I = np.eye(4)
        self.P = (I - np.dot(K, self.H)) @ self.P


# def interpolate_point_data(df, point_column_name, target_idx, window_radius=5):
#     start_idx = max(0, int(target_idx) - window_radius)
#     end_idx = min(len(df) - 1, int(target_idx) + window_radius)
#     window_df = df.loc[start_idx:end_idx].copy()
#     x_coords = window_df.apply(
#         lambda row: (
#             row[point_column_name][0]
#             if isinstance(row[point_column_name], tuple)
#             else np.nan
#         ),
#         axis=1,
#     )
#     y_coords = window_df.apply(
#         lambda row: (
#             row[point_column_name][1]
#             if isinstance(row[point_column_name], tuple)
#             else np.nan
#         ),
#         axis=1,
#     )
#     interp_data_x = pd.DataFrame(
#         {"idx": window_df["original_idx"], "coord": x_coords}
#     ).dropna()
#     interp_data_y = pd.DataFrame(
#         {"idx": window_df["original_idx"], "coord": y_coords}
#     ).dropna()
#     if len(interp_data_x) < 2 or len(interp_data_y) < 2:
#         return None
#     try:
#         kind = "cubic" if len(interp_data_x) >= 4 else "linear"
#         interp_func_x = interp1d(
#             interp_data_x["idx"],
#             interp_data_x["coord"],
#             kind=kind,
#             bounds_error=False,
#             fill_value="extrapolate",
#         )
#         kind = "cubic" if len(interp_data_y) >= 4 else "linear"
#         interp_func_y = interp1d(
#             interp_data_y["idx"],
#             interp_data_y["coord"],
#             kind=kind,
#             bounds_error=False,
#             fill_value="extrapolate",
#         )
#         new_x = interp_func_x(target_idx)
#         new_y = interp_func_y(target_idx)
#         return (float(new_x), float(new_y))
#     except Exception:
#         return None
from scipy.interpolate import interp1d
from sklearn.metrics import r2_score


def get_confidence_level(r2_x, r2_y):
    r2_min = min(r2_x, r2_y)
    if r2_min >= 0.9:
        return "high"
    elif r2_min >= 0.6:
        return "medium"
    else:
        return "low"


# ⚽️ 목적: 좌표가 None인 경우에 쓸만한 추정값을 보간으로 채워넣기 위함 + fallback 로직
def interpolate_point_data_with_quality_and_fallback(
    df, point_column_name, target_idx, window_radius=5, fallback_strategy=True
):
    start_idx = max(0, int(target_idx) - window_radius)
    end_idx = min(len(df) - 1, int(target_idx) + window_radius)
    window_df = df.loc[start_idx:end_idx].copy()

    x_coords = window_df.apply(
        lambda row: (
            row[point_column_name][0]
            if isinstance(row[point_column_name], tuple)
            else np.nan
        ),
        axis=1,
    )
    y_coords = window_df.apply(
        lambda row: (
            row[point_column_name][1]
            if isinstance(row[point_column_name], tuple)
            else np.nan
        ),
        axis=1,
    )

    interp_data_x = pd.DataFrame(
        {"idx": window_df["original_idx"], "coord": x_coords}
    ).dropna()
    interp_data_y = pd.DataFrame(
        {"idx": window_df["original_idx"], "coord": y_coords}
    ).dropna()

    if len(interp_data_x) < 2 or len(interp_data_y) < 2:
        if DEBUG_MODE:
            print(f"[DEBUG] 보간 불가: 데이터 수 부족 (idx={target_idx})")
        return {"interpolated_point": None, "quality": None}

    try:
        kind = "cubic" if len(interp_data_x) >= 4 else "linear"

        interp_func_x = interp1d(
            interp_data_x["idx"],
            interp_data_x["coord"],
            kind=kind,
            bounds_error=False,
            fill_value="extrapolate",
        )
        interp_func_y = interp1d(
            interp_data_y["idx"],
            interp_data_y["coord"],
            kind=kind,
            bounds_error=False,
            fill_value="extrapolate",
        )

        new_x = float(interp_func_x(target_idx))
        new_y = float(interp_func_y(target_idx))

        # R² 계산
        r2_x = r2_score(interp_data_x["coord"], interp_func_x(interp_data_x["idx"]))
        r2_y = r2_score(interp_data_y["coord"], interp_func_y(interp_data_y["idx"]))
        confidence = get_confidence_level(r2_x, r2_y)

        if DEBUG_MODE:
            print(
                f"[DEBUG] idx={target_idx}, 보간=({new_x:.1f}, {new_y:.1f}), R²=({r2_x:.2f}, {r2_y:.2f}), 신뢰도={confidence}"
            )

        # 신뢰도 낮고 fallback 전략 사용
        if confidence == "low" and fallback_strategy:
            fallback = None
            for offset in range(1, window_radius + 1):
                for neighbor in [target_idx - offset, target_idx + offset]:
                    if 0 <= neighbor < len(df):
                        neighbor_val = df.iloc[neighbor][point_column_name]
                        if isinstance(neighbor_val, tuple):
                            fallback = neighbor_val
                            break
                if fallback is not None:
                    break
            if DEBUG_MODE:
                print(f"[DEBUG] → Fallback 사용됨: {fallback}")
            return {
                "interpolated_point": fallback,
                "quality": {"r2_x": r2_x, "r2_y": r2_y, "level": confidence},
                "fallback_used": True,
            }

        return {
            "interpolated_point": (new_x, new_y),
            "quality": {"r2_x": r2_x, "r2_y": r2_y, "level": confidence},
            "fallback_used": False,
        }

    except Exception as e:
        if DEBUG_MODE:
            print(f"[DEBUG] 예외 발생: {e}")
        return {"interpolated_point": None, "quality": None, "error": str(e)}


# ⚽️ 목적: 임팩트가 발생한 정확한 순간(정밀 프레임)을 찾아내기 위함
def find_refined_impact_with_scipy(df, impact_idx, window_radius=7, density_factor=50):
    """
    [수정된 버전] 실제 물리적 충돌 순간을 찾는 함수
    기존 SciPy 수학적 최소값 대신 물리적 접촉을 우선 감지
    """
    print("🔬 물리적 충돌 감지 시작 (기존 SciPy 함수 업그레이드)...")

    # === 1단계: 물리적 충돌 후보 탐색 ===
    start_idx = max(0, impact_idx - window_radius)
    end_idx = min(len(df), impact_idx + window_radius)

    window_df = df.iloc[start_idx:end_idx].copy()

    physical_candidates = []

    # 각 프레임에서 실제 접촉 가능성 검사
    for i, (_, row) in enumerate(window_df.iterrows()):
        actual_idx = start_idx + i

        # 기본 데이터 확인
        foot_pos = row.get("foot_pos")
        ball_center = row.get("ball_center")
        ball_radius = row.get("ball_radius", 11)
        foot_speed = row.get("kicking_foot_speed", 0)

        if not foot_pos or not ball_center or foot_speed < 0.5:
            continue

        # 실제 거리 계산
        distance = np.hypot(foot_pos[0] - ball_center[0], foot_pos[1] - ball_center[1])

        # 물리적 접촉 임계값 (공 반지름 + 발 두께)
        contact_threshold = ball_radius * 1.4  # 40% 여유

        # === 물리적 접촉 조건 확인 ===
        is_in_contact = distance <= contact_threshold

        if is_in_contact:
            print(
                f"   물리적 접촉 후보: Frame {actual_idx}, 거리={distance:.1f}px ≤ 임계값={contact_threshold:.1f}px"
            )

            # 발 속도 조건 (최소 임팩트 속도)
            min_impact_speed = 1.0  # px/frame
            if foot_speed < min_impact_speed:
                print(f"       속도 부족: {foot_speed:.2f} < {min_impact_speed}")
                continue

            # 접근 방향 확인
            approach_velocity = row.get("foot_ball_approach_velocity", 0)
            is_approaching = approach_velocity < -0.1  # 음수 = 접근

            # 간단한 접촉 비율 계산
            contact_ratio = max(0, (contact_threshold - distance) / contact_threshold)

            # 물리적 점수 계산
            contact_score = (
                max(0, (contact_threshold - distance) / contact_threshold) * 100
            )
            speed_score = min(foot_speed * 10, 50)
            contact_bonus = contact_ratio * 50  # 최대 50점
            approach_bonus = 30 if is_approaching else 0

            physical_score = (
                contact_score + speed_score + contact_bonus + approach_bonus
            )

            print(
                f"      물리 점수: {physical_score:.1f} (접촉:{contact_score:.1f} + 속도:{speed_score:.1f} + 비율:{contact_bonus:.1f} + 접근:{approach_bonus})"
            )

            physical_candidates.append(
                {
                    "original_idx": row["original_idx"],
                    "distance": distance,
                    "foot_speed": foot_speed,
                    "physical_score": physical_score,
                    "contact_ratio": contact_ratio,
                    "is_real_contact": True,
                }
            )

    # === 2단계: 물리적 충돌 후보가 있으면 최고 점수 선택 ===
    if physical_candidates:
        best_candidate = max(physical_candidates, key=lambda x: x["physical_score"])

        print(f"실제 충돌 순간 발견!")
        print(f"   Original Index: {best_candidate['original_idx']}")
        print(f"   접촉 거리: {best_candidate['distance']:.2f}px")
        print(f"   발 속도: {best_candidate['foot_speed']:.2f}px/fr")
        print(f"   물리 점수: {best_candidate['physical_score']:.1f}")

        return {
            "refined_original_idx": best_candidate["original_idx"],
            "min_interpolated_distance": best_candidate["distance"],
            "physical_impact_detected": True,
            "impact_method": "physical_contact",
        }

    # === 3단계: 물리적 충돌이 없으면 기존 SciPy 방법으로 Fallback ===
    print("물리적 충돌을 찾지 못했습니다. 기존 수학적 방법으로 fallback...")

    # 기존 SciPy 보간 로직
    interp_data = window_df[["original_idx", "distance_smoothed"]].dropna()
    interp_data = interp_data[np.isfinite(interp_data["distance_smoothed"])]
    interp_data = interp_data.drop_duplicates(subset=["original_idx"])

    if len(interp_data) < 2:
        print("SciPy 보간 실패: 데이터 부족")
        return None

    x = interp_data["original_idx"].values
    y = interp_data["distance_smoothed"].values
    kind = "cubic" if len(x) >= 4 else "linear"

    try:
        interp_func = interp1d(
            x, y, kind=kind, bounds_error=False, fill_value="extrapolate"
        )

        fine_x_step = 1.0 / max(1, density_factor)
        fine_x = np.arange(x.min(), x.max() + fine_x_step, fine_x_step)
        interpolated_y = interp_func(fine_x)

        min_idx = np.argmin(interpolated_y)
        refined_idx = fine_x[min_idx]
        min_distance = interpolated_y[min_idx]

        # === 4단계: SciPy 결과의 물리적 타당성 검증 ===
        print(f"🔍 SciPy 결과 검증: Idx={refined_idx:.2f}, Dist={min_distance:.2f}px")

        # 거리 임계값 확인
        avg_ball_radius = window_df["ball_radius"].mean()
        max_acceptable_distance = avg_ball_radius * 2.0  # 공 지름 정도까지만 허용

        if min_distance > max_acceptable_distance:
            print(
                f"SciPy 결과가 물리적으로 타당하지 않음: {min_distance:.1f}px > {max_acceptable_distance:.1f}px"
            )
            print("가장 가까운 프레임을 대신 사용")

            # 가장 가까운 거리의 실제 프레임 찾기
            closest_idx = window_df["distance_smoothed"].idxmin()
            closest_row = window_df.loc[closest_idx]

            return {
                "refined_original_idx": closest_row["original_idx"],
                "min_interpolated_distance": closest_row["distance_smoothed"],
                "physical_impact_detected": False,
                "impact_method": "closest_frame_fallback",
                "scipy_rejected": True,
            }

        print(f"SciPy 결과 물리적 타당성 통과")

        return {
            "refined_original_idx": refined_idx,
            "min_interpolated_distance": min_distance,
            "physical_impact_detected": False,
            "impact_method": "scipy_mathematical",
        }

    except Exception as e:
        print(f"SciPy 보간 중 에러: {e}")
        return None


# 1단계: 실제 물리적 접촉 우선 탐지
# 2단계: 접촉 없으면 SciPy 수학적 방법 사용
# 3단계: SciPy 결과의 물리적 타당성 검증
# 4단계: 타당하지 않으면 가장 가까운 실제 프레임 사용


# ⚽️ 보간(interpolation)된 시점(refined idx)의 발과 공의 위치를 두 프레임을 알파 블렌딩하여 생성된 이미지 위에 시각화
def visualize_interpolated_moment(
    frames_list, interp_details, interp_points, refined_info, pixel_to_cm_scale
):
    print("\n--- ✨ Generating Visual Interpolation Frame... ✨ ---")
    if not interp_details or not interp_points:
        print("🟡 V-Interp: Insufficient data for visualization.")
        return
    refined_idx = interp_details.get("refined_original_idx")
    if refined_idx is None:
        print("🟡 V-Interp: Refined index not found.")
        return
    prev_frame_idx = int(refined_idx)
    next_frame_idx = prev_frame_idx + 1
    alpha = refined_idx - prev_frame_idx
    if not (
        0 <= prev_frame_idx < len(frames_list)
        and 0 <= next_frame_idx < len(frames_list)
    ):
        print("🟡 V-Interp: Frame indices are out of bounds.")
        return
    frame1 = frames_list[prev_frame_idx].copy()
    frame2 = frames_list[next_frame_idx].copy()
    blended_frame = cv2.addWeighted(frame1, 1 - alpha, frame2, alpha, 0)
    foot_pos = interp_points.get("foot_pos")
    ball_pos = interp_points.get("ball_center")
    if not foot_pos or not ball_pos:
        print("🟡 V-Interp: Interpolated foot or ball position not found.")
        return
    foot_pos_int = tuple(map(int, foot_pos))
    ball_pos_int = tuple(map(int, ball_pos))
    cv2.circle(blended_frame, foot_pos_int, 12, (0, 255, 255), -1)
    cv2.circle(blended_frame, foot_pos_int, 12, (0, 0, 0), 2)
    cv2.circle(blended_frame, ball_pos_int, 12, (255, 0, 255), -1)
    cv2.circle(blended_frame, ball_pos_int, 12, (0, 0, 0), 2)
    cv2.line(blended_frame, foot_pos_int, ball_pos_int, (255, 255, 255), 2, cv2.LINE_AA)
    dist_px = np.hypot(foot_pos[0] - ball_pos[0], foot_pos[1] - ball_pos[1])
    dist_cm = dist_px * pixel_to_cm_scale
    contact_region = get_ball_contact_region(
        ball_pos[0] - foot_pos[0], ball_pos[1] - foot_pos[1]
    )
    info_texts = [
        f"Interpolated Moment (Refined Idx: {refined_idx:.2f})",
        f"Foot-Ball Dist: {dist_cm:.2f} cm ({dist_px:.2f} px)",
        f"Calculated Contact Region: {contact_region}",
    ]
    y_pos = 30
    for text in info_texts:
        cv2.putText(
            blended_frame,
            text,
            (10, y_pos),
            cv2.FONT_HERSHEY_SIMPLEX,
            0.7,
            (0, 0, 0),
            4,
            cv2.LINE_AA,
        )
        cv2.putText(
            blended_frame,
            text,
            (10, y_pos),
            cv2.FONT_HERSHEY_SIMPLEX,
            0.7,
            (0, 255, 255),
            2,
            cv2.LINE_AA,
        )
        y_pos += 30
    if DEBUG_MODE:
        plt.figure(figsize=(12, 9))
        plt.imshow(cv2.cvtColor(blended_frame, cv2.COLOR_BGR2RGB))
        plt.title("Visually Interpolated Impact Moment")
        plt.axis("off")
        plt.show()


# ⚽️ [V3.1] 발 전체 흐름 (무릎~발목~발끝)의 궤적과 구조적 관계를 분석하여 킥 유형(in-step, toe, inside)을 추론
def determine_kick_part_by_whole_leg_flow_v3_1(
    current_idx,
    kicking_foot_side,
    ball_center,
    pose_results_list,
    all_processed_frames,
    window_size=7,
    min_points=3,
    DEBUG_MODE=False,
):
    """

    - 단일 랜드마크(발끝) 대신, 전체 발 움직임(벡터 흐름)을 분석하여 더 정확한 판단
    - 2D 기반이나, 3D 구조적 관계 일부 포함 (향후 확장 가능)
    """
    try:
        if current_idx < min_points or ball_center is None:
            return "unknown"

        start_idx = max(0, current_idx - window_size)
        ankle_enum = (
            mp.solutions.pose.PoseLandmark.RIGHT_ANKLE
            if kicking_foot_side == "right"
            else mp.solutions.pose.PoseLandmark.LEFT_ANKLE
        )
        knee_enum = (
            mp.solutions.pose.PoseLandmark.RIGHT_KNEE
            if kicking_foot_side == "right"
            else mp.solutions.pose.PoseLandmark.LEFT_KNEE
        )
        toe_enum = (
            mp.solutions.pose.PoseLandmark.RIGHT_FOOT_INDEX
            if kicking_foot_side == "right"
            else mp.solutions.pose.PoseLandmark.LEFT_FOOT_INDEX
        )

        ankle_positions, toe_positions, knee_positions = [], [], []

        for i in range(start_idx, current_idx):
            if i < len(pose_results_list) and pose_results_list[i]:
                frame_shape = all_processed_frames[i].shape
                ankle = get_specific_landmark_position(
                    pose_results_list[i], ankle_enum, frame_shape
                )
                toe = get_specific_landmark_position(
                    pose_results_list[i], toe_enum, frame_shape
                )
                knee = get_specific_landmark_position(
                    pose_results_list[i], knee_enum, frame_shape
                )
                if ankle and toe and knee:
                    ankle_positions.append(ankle)
                    toe_positions.append(toe)
                    knee_positions.append(knee)

        if len(ankle_positions) < 2 or len(knee_positions) < 2:
            return "unknown"

        ankle_velocities = np.diff(np.array(ankle_positions), axis=0)
        avg_ankle_vector = np.mean(ankle_velocities, axis=0)

        if np.linalg.norm(avg_ankle_vector) < 1.0:
            return "unknown"

        foot_pos_pre_impact = ankle_positions[-1]
        foot_to_ball_vector = np.array(ball_center) - np.array(foot_pos_pre_impact)

        norm_v1 = np.linalg.norm(avg_ankle_vector)
        norm_v2 = np.linalg.norm(foot_to_ball_vector)
        if norm_v1 < 1e-6 or norm_v2 < 1e-6:
            return "unknown"

        unit_v1 = avg_ankle_vector / norm_v1
        unit_v2 = foot_to_ball_vector / norm_v2
        angle_deg = np.degrees(np.arccos(np.clip(np.dot(unit_v1, unit_v2), -1.0, 1.0)))

        # 발 구조 분석 (무릎-발목 vs 발목-발끝 각도)
        final_knee = np.array(knee_positions[-1])
        final_ankle = np.array(ankle_positions[-1])
        final_toe = np.array(toe_positions[-1])
        vec_thigh = final_ankle - final_knee
        vec_foot = final_toe - final_ankle

        norm_thigh = np.linalg.norm(vec_thigh)
        norm_foot = np.linalg.norm(vec_foot)
        if norm_thigh < 1e-6 or norm_foot < 1e-6:
            return "unknown"

        thigh_foot_angle = np.degrees(
            np.arccos(
                np.clip(
                    np.dot(vec_thigh, vec_foot) / (norm_thigh * norm_foot), -1.0, 1.0
                )
            )
        )

        if DEBUG_MODE:
            print(
                f"🏹 Direction Angle: {angle_deg:.2f}°, Knee-Ankle-Toe Angle: {thigh_foot_angle:.2f}°"
            )

        # 킥 유형 판단
        if angle_deg < 45.0:
            if 130 < thigh_foot_angle < 170:
                return "instep"
            elif thigh_foot_angle < 110:
                return "toe"
            else:
                return "inside"
        elif thigh_foot_angle < 105:
            return "toe"
        else:
            return "inside"

    except Exception as e:
        print(f"ERROR in determine_kick_part_by_whole_leg_flow_v3_1: {e}")
        return "unknown"


# --- 선수 각도 데이터 및 비교 함수 ---
player_data = {
    "messi": {
        "backswing": {
            "R_hip": {"mean": 160.9468, "std": 9.532324783},
            "R_knee": {"mean": 170.78125, "std": 10.46103882},
            "R_ankle": {"mean": 145.489, "std": 3.524245262},
            "L_hip": {"mean": 171.7248571, "std": 11.36573581},
            "L_knee": {"mean": 71.0198, "std": 17.49788997},
            "L_ankle": {"mean": 153.983, "std": 11.84122082},
        },
        "impact": {
            "R_hip": {"mean": 149.321, "std": 14.2431464},
            "R_knee": {"mean": 168.0067143, "std": 6.840434483},
            "R_ankle": {"mean": 142.6363333, "std": 15.73840486},
            "L_hip": {"mean": 126.016, "std": 8.853215461},
            "L_knee": {"mean": 161.4106, "std": 13.47662239},
            "L_ankle": {"mean": 136.899, "std": 19.65173808},
        },
    },
    "ronaldo": {
        "backswing": {
            "R_hip": {"mean": 156.0967, "std": 11.39067},
            "R_knee": {"mean": 61.15573, "std": 18.01905},
            "R_ankle": {"mean": 146.4122, "std": 28.38302},
            "L_hip": {"mean": 154.2442, "std": 6.301532},
            "L_knee": {"mean": 175.1804, "std": 5.957248},
            "L_ankle": {"mean": 155.9201, "std": 20.688},
        },
        "impact": {
            "R_hip": {"mean": 118.0275, "std": 9.277196},
            "R_knee": {"mean": 168.6488, "std": 8.127587},
            "R_ankle": {"mean": 138.005, "std": 19.74243},
            "L_hip": {"mean": 140.3548, "std": 9.359457},
            "L_knee": {"mean": 169.4105, "std": 8.598768},
            "L_ankle": {"mean": 146.0102, "std": 15.86986},
        },
    },
}


def calculate_z_score(user_value, mean, std):
    if pd.isna(user_value) or pd.isna(mean) or pd.isna(std):
        return float("inf")
    if std == 0:
        return 0 if user_value == mean else float("inf")
    return abs((user_value - mean) / std)


def z_score_to_similarity(z_score):
    if z_score == float("inf"):
        return 0
    return np.exp(-0.5 * z_score**2)


def calculate_all_limbs_angle_comparison_score(
    user_L_angles, user_R_angles, player_model_name, phase
):
    player_stats_phase = player_data[player_model_name][phase]
    all_joint_similarities = {}
    all_joint_z_scores = {}
    for side_prefix, user_angles_side in [("L_", user_L_angles), ("R_", user_R_angles)]:
        if user_angles_side is None:
            continue
        for joint_name, user_angle_val in user_angles_side.items():
            if joint_name not in ["hip", "knee", "ankle"]:
                continue
            player_joint_key = f"{side_prefix}{joint_name}"
            if player_joint_key in player_stats_phase:
                mean_val = player_stats_phase[player_joint_key]["mean"]
                std_val = player_stats_phase[player_joint_key]["std"]
                z = calculate_z_score(user_angle_val, mean_val, std_val)
                sim = z_score_to_similarity(z)
                all_joint_similarities[player_joint_key] = sim
                all_joint_z_scores[player_joint_key] = z
    player_dominant_foot_prefix = "L_" if player_model_name == "messi" else "R_"
    supporting_foot_prefix = "R_" if player_model_name == "messi" else "L_"
    weights = {
        f"{player_dominant_foot_prefix}knee": 0.30,
        f"{player_dominant_foot_prefix}hip": 0.20,
        f"{player_dominant_foot_prefix}ankle": 0.10,
        f"{supporting_foot_prefix}knee": 0.20,
        f"{supporting_foot_prefix}hip": 0.10,
        f"{supporting_foot_prefix}ankle": 0.10,
    }
    weighted_similarity_sum = 0
    total_weight_sum = 0
    for joint_key, similarity_val in all_joint_similarities.items():
        if joint_key in weights and pd.notna(similarity_val):
            weighted_similarity_sum += similarity_val * weights[joint_key]
            total_weight_sum += weights[joint_key]
    final_overall_similarity = 0
    if total_weight_sum > 0:
        final_overall_similarity = weighted_similarity_sum / total_weight_sum
    score = final_overall_similarity * 10
    if score >= 6.8:
        adjusted_score = 8 + (score - 6.8) * (2 / 3.2)
    elif score >= 3.7:
        adjusted_score = 5 + (score - 3.7) * (3 / 3.1)
    elif score >= 1.4:
        adjusted_score = 2 + (score - 1.4) * (3 / 2.3)
    else:
        adjusted_score = score * (2 / 1.4)
    adjusted_score = min(10, max(0, adjusted_score))
    return {
        "score": round(adjusted_score, 1),
        "all_joint_similarities": all_joint_similarities,
        "all_joint_z_scores": all_joint_z_scores,
        "weighted_overall_similarity": final_overall_similarity,
        "comparison_player": player_model_name,
        "phase": phase,
    }


# --- 점수 계산 함수 ---
def calculate_kick_score(
    impact_info,
    pixel_to_cm_scale,
    fps=30.0,
    max_foot_speed_px_fr=None,
    analysis_df_for_angles=None,
    impact_df_idx_for_angles=-1,
    backswing_df_idx_for_angles=-1,
):
    if not impact_info:
        print("Warning: No impact_info provided to calculate_kick_score.")
        return None
    total_score = 0
    MAX_POSSIBLE_SCORE = 100
    score_details = {
        "impact_evaluation": {"subtotal": 0, "details": {}},
        "backswing_evaluation": {"subtotal": 0, "details": {}},
    }
    scale = get_safe_pixel_to_cm_scale(pixel_to_cm_scale)
    category_impact = "impact_evaluation"
    kicking_foot_part = impact_info.get("kicking_foot_part", "N/A")
    foot_part_score = (
        10
        if kicking_foot_part in ["instep", "inside"]
        else 3 if kicking_foot_part != "N/A" else 0
    )
    score_details[category_impact]["details"]["hitting_foot_part"] = {
        "value": kicking_foot_part,
        "score": foot_part_score,
        "max_score": 10,
    }
    score_details[category_impact]["subtotal"] += foot_part_score
    contact_region = impact_info.get("contact_region_on_ball", "N/A")
    ball_contact_score = (
        10
        if contact_region in ["Bottom", "Center"]
        else (
            8
            if contact_region in ["Bottom-Left", "Bottom-Right"]
            else (
                5
                if contact_region in ["Left", "Right"]
                else 3 if contact_region in ["Top", "Top-Left", "Top-Right"] else 0
            )
        )
    )
    score_details[category_impact]["details"]["ball_contact_point"] = {
        "value": contact_region,
        "score": ball_contact_score,
        "max_score": 10,
    }
    score_details[category_impact]["subtotal"] += ball_contact_score
    support_dist_px = impact_info.get("dist_ball_to_supporting_foot_ankle")
    support_dist_score = 0
    support_dist_cm_str = "N/A"
    if pd.notna(support_dist_px) and scale > 0:
        support_dist_cm = support_dist_px * scale
        support_dist_cm_str = f"{support_dist_cm:.1f}cm"
        if 10 <= support_dist_cm < 25:
            support_dist_score = 10
        elif 25 <= support_dist_cm < 40:
            support_dist_score = 7
        elif support_dist_cm < 10 or support_dist_cm >= 40:
            support_dist_score = 4
        else:
            support_dist_score = 0
    score_details[category_impact]["details"]["support_foot_ball_distance"] = {
        "value": support_dist_cm_str,
        "score": support_dist_score,
        "max_score": 10,
    }
    score_details[category_impact]["subtotal"] += support_dist_score
    ball_max_speed_px_fr = impact_info.get("max_ball_speed_px_fr", 0)
    ball_initial_speed_score = 0
    ball_initial_speed_str = "N/A"
    if (
        pd.notna(ball_max_speed_px_fr)
        and ball_max_speed_px_fr > 0
        and scale > 0
        and fps > 0
    ):
        ball_speed_km_h = convert_speed_to_kmh_normalized(
            ball_max_speed_px_fr, scale, fps
        )
        # 기존 cm/s 계산은 문자열 포맷팅에만 사용되므로 그대로 두거나, 필요 시 수정
        ball_speed_cm_s = (ball_speed_km_h / CM_S_TO_KM_H) if CM_S_TO_KM_H > 0 else 0
        ball_initial_speed_str = (
            f"{ball_speed_km_h:.1f}km/h ({ball_speed_cm_s:.1f}cm/s)"
        )
        if ball_speed_km_h >= 100:
            ball_initial_speed_score = 10
        elif ball_speed_km_h >= 85:
            ball_initial_speed_score = 8
        elif ball_speed_km_h >= 70:
            ball_initial_speed_score = 6
        elif ball_speed_km_h >= 50:
            ball_initial_speed_score = 4
        elif ball_speed_km_h > 0:
            ball_initial_speed_score = 2
    score_details[category_impact]["details"]["ball_initial_speed"] = {
        "value": ball_initial_speed_str,
        "score": ball_initial_speed_score,
        "max_score": 10,
    }
    score_details[category_impact]["subtotal"] += ball_initial_speed_score
    impact_player_angle_score = 0
    impact_player_angle_metric_val = "N/A (분석 불가)"
    if (
        USE_PLAYER_ANGLE_COMPARISON
        and analysis_df_for_angles is not None
        and impact_df_idx_for_angles != -1
    ):
        user_L_angles_imp, user_R_angles_imp = {}, {}
        if "refined_idx" in impact_info:
            df_row_impact_series = analysis_df_for_angles[
                analysis_df_for_angles["original_idx"] <= impact_info["refined_idx"]
            ].iloc[-1]
            user_L_angles_imp = {
                "hip": impact_info.get("left_hip_angle"),
                "knee": impact_info.get("left_knee_angle"),
                "ankle": impact_info.get("left_ankle_angle"),
            }
            user_R_angles_imp = {
                "hip": impact_info.get("right_hip_angle"),
                "knee": impact_info.get("right_knee_angle"),
                "ankle": impact_info.get("right_ankle_angle"),
            }
        elif impact_df_idx_for_angles in analysis_df_for_angles.index:
            df_row_impact = analysis_df_for_angles.loc[impact_df_idx_for_angles]
            user_L_angles_imp = {
                "hip": df_row_impact.get("left_hip_angle"),
                "knee": df_row_impact.get("left_knee_angle"),
                "ankle": df_row_impact.get("left_ankle_angle"),
            }
            user_R_angles_imp = {
                "hip": df_row_impact.get("right_hip_angle"),
                "knee": df_row_impact.get("right_knee_angle"),
                "ankle": df_row_impact.get("right_ankle_angle"),
            }
        user_kicking_foot = impact_info.get("kicking_foot")
        player_to_compare_imp = (
            "messi"
            if user_kicking_foot == "left"
            else ("ronaldo" if user_kicking_foot == "right" else None)
        )
        all_user_angles_valid_imp = all(
            pd.notna(angle_val)
            for angle_set in [user_L_angles_imp, user_R_angles_imp]
            if angle_set is not None
            for angle_val in angle_set.values()
        )
        if player_to_compare_imp and all_user_angles_valid_imp:
            angle_comp_result_impact = calculate_all_limbs_angle_comparison_score(
                user_L_angles_imp, user_R_angles_imp, player_to_compare_imp, "impact"
            )
            impact_player_angle_score = angle_comp_result_impact["score"]
            impact_player_angle_metric_val = f"{angle_comp_result_impact['score']:.1f}/10 vs {player_to_compare_imp.capitalize()}"
        elif not player_to_compare_imp:
            impact_player_angle_metric_val = "N/A (주발 정보 없음)"
        elif not all_user_angles_valid_imp:
            impact_player_angle_metric_val = "N/A (사용자 각도 부족)"
    score_details[category_impact]["details"]["impact_angle_comparison"] = {
        "value": impact_player_angle_metric_val,
        "score": impact_player_angle_score,
        "max_score": 10,
    }
    score_details[category_impact]["subtotal"] += impact_player_angle_score
    category_backswing = "backswing_evaluation"
    max_foot_swing_speed_score = 0
    max_foot_swing_speed_str = "N/A"
    if (
        max_foot_speed_px_fr is not None
        and pd.notna(max_foot_speed_px_fr)
        and max_foot_speed_px_fr > 0
        and scale > 0
        and fps > 0
    ):
        max_foot_speed_km_h = convert_speed_to_kmh_normalized(
            max_foot_speed_px_fr, scale, fps
        )
        # 기존 cm/s 계산은 문자열 포맷팅에만 사용되므로 그대로 두거나, 필요 시 수정
        max_foot_speed_cm_s = (
            (max_foot_speed_km_h / CM_S_TO_KM_H) if CM_S_TO_KM_H > 0 else 0
        )
        max_foot_swing_speed_str = (
            f"{max_foot_speed_km_h:.1f}km/h ({max_foot_speed_cm_s:.1f}cm/s)"
        )
        if max_foot_speed_km_h >= 108:
            max_foot_swing_speed_score = 10
        elif 90 <= max_foot_speed_km_h < 108:
            max_foot_swing_speed_score = 8
        elif 72 <= max_foot_speed_km_h < 90:
            max_foot_swing_speed_score = 6
        elif 54 <= max_foot_speed_km_h < 72:
            max_foot_swing_speed_score = 4
        elif max_foot_speed_km_h > 0:
            max_foot_swing_speed_score = 2
    score_details[category_backswing]["details"]["max_foot_swing_speed"] = {
        "value": max_foot_swing_speed_str,
        "score": max_foot_swing_speed_score,
        "max_score": 10,
    }
    score_details[category_backswing]["subtotal"] += max_foot_swing_speed_score
    foot_accel_px_fr2 = impact_info.get("kicking_foot_acceleration_scalar")
    kick_foot_kinematics_score = 0
    kick_foot_kinematics_str = "N/A"
    if pd.notna(foot_accel_px_fr2):
        kick_foot_kinematics_str = f"Accel (raw_px/fr²): {foot_accel_px_fr2:.1f}"
        if abs(foot_accel_px_fr2) < 25:
            kick_foot_kinematics_score = 10
        elif abs(foot_accel_px_fr2) < 50:
            kick_foot_kinematics_score = 7
        elif abs(foot_accel_px_fr2) < 75:
            kick_foot_kinematics_score = 4
        elif abs(foot_accel_px_fr2) < 100:
            kick_foot_kinematics_score = 2
        else:
            kick_foot_kinematics_score = 0
    score_details[category_backswing]["details"]["kick_foot_kinematics_change"] = {
        "value": kick_foot_kinematics_str,
        "score": kick_foot_kinematics_score,
        "max_score": 10,
    }
    score_details[category_backswing]["subtotal"] += kick_foot_kinematics_score
    backswing_knee_angle = impact_info.get("backswing_kicking_knee_angle")
    backswing_knee_score = 0
    backswing_knee_str = "N/A (데이터 없음)"

    if pd.notna(backswing_knee_angle):
        backswing_knee_str = f"{backswing_knee_angle:.1f}도"
        user_kicking_foot_for_bs_knee = impact_info.get("kicking_foot")
        target_data = None

        if user_kicking_foot_for_bs_knee == "left":
            target_data = player_data["messi"]["backswing"]["L_knee"]
        elif user_kicking_foot_for_bs_knee == "right":
            target_data = player_data["ronaldo"]["backswing"]["R_knee"]

        if target_data:
            mean_val = target_data["mean"]
            std_val = target_data["std"]
            z = abs((backswing_knee_angle - mean_val) / std_val) if std_val > 0 else 0

            # 완화된 z-score 기준 적용
            if z <= 0.75:
                backswing_knee_score = 10
            elif z <= 1.5:
                backswing_knee_score = 8
            elif z <= 2.5:
                backswing_knee_score = 6
            elif z <= 3.5:
                backswing_knee_score = 4
            else:
                backswing_knee_score = 2

        else:
            backswing_knee_str = f"{backswing_knee_angle:.1f}도 (주발 정보 없음)"

    score_details[category_backswing]["details"]["backswing_knee_angle_size"] = {
        "value": backswing_knee_str,
        "score": backswing_knee_score,
        "max_score": 10,
    }
    score_details[category_backswing]["subtotal"] += backswing_knee_score
    support_stability_px_fr = impact_info.get("supporting_foot_stability")
    support_stability_score = 0
    support_stability_str = "N/A"
    if pd.notna(support_stability_px_fr) and scale > 0:
        support_stability_cm_fr = support_stability_px_fr * scale
        support_stability_str = f"{support_stability_cm_fr:.2f}cm/frame"
        # # ✨ MediaPipe 노이즈를 고려한 현실적 기준
        # if support_stability_cm_fr < 2.0:  # 10점 (매우 안정적)
        #     support_stability_score = 10
        # elif support_stability_cm_fr < 5.0:  # 8점
        #     support_stability_score = 8
        # elif support_stability_cm_fr < 8.0:  # 6점
        #     support_stability_score = 6
        # elif support_stability_cm_fr < 12.0:  # 4점
        #     support_stability_score = 4
        # elif support_stability_cm_fr < 18.0:  # 2점
        #     support_stability_score = 2
        # else:  # 0점
        #     support_stability_score = 0
        if support_stability_cm_fr < 2.0:
            support_stability_score = 10
        elif support_stability_cm_fr < 18.0:
            # 선형 감소 (10 → 0점), 정수화
            support_stability_score = max(
                0, int(10 - ((support_stability_cm_fr - 2.0) * (10 / 16)))
            )
        else:
            support_stability_score = 0
    score_details[category_backswing]["details"]["support_foot_stability"] = {
        "value": support_stability_str,
        "score": support_stability_score,
        "max_score": 10,
    }
    score_details[category_backswing]["subtotal"] += support_stability_score
    backswing_player_angle_score = 0
    backswing_player_angle_metric_val = "N/A (분석 불가)"
    if (
        USE_PLAYER_ANGLE_COMPARISON
        and analysis_df_for_angles is not None
        and backswing_df_idx_for_angles != -1
        and backswing_df_idx_for_angles in analysis_df_for_angles.index
    ):
        df_row_backswing = analysis_df_for_angles.loc[backswing_df_idx_for_angles]
        user_L_angles_bs = {
            "hip": df_row_backswing.get("left_hip_angle"),
            "knee": df_row_backswing.get("left_knee_angle"),
            "ankle": df_row_backswing.get("left_ankle_angle"),
        }
        user_R_angles_bs = {
            "hip": df_row_backswing.get("right_hip_angle"),
            "knee": df_row_backswing.get("right_knee_angle"),
            "ankle": df_row_backswing.get("right_ankle_angle"),
        }
        user_kicking_foot = impact_info.get("kicking_foot")
        player_to_compare_bs = (
            "messi"
            if user_kicking_foot == "left"
            else ("ronaldo" if user_kicking_foot == "right" else None)
        )
        all_user_angles_valid_bs = all(
            pd.notna(angle_val)
            for angle_set in [user_L_angles_bs, user_R_angles_bs]
            if angle_set is not None
            for angle_val in angle_set.values()
        )
        if player_to_compare_bs and all_user_angles_valid_bs:
            angle_comp_result_backswing = calculate_all_limbs_angle_comparison_score(
                user_L_angles_bs, user_R_angles_bs, player_to_compare_bs, "backswing"
            )
            backswing_player_angle_score = angle_comp_result_backswing["score"]
            backswing_player_angle_metric_val = f"{angle_comp_result_backswing['score']:.1f}/10 vs {player_to_compare_bs.capitalize()}"
        elif not player_to_compare_bs:
            backswing_player_angle_metric_val = "N/A (주발 정보 없음)"
        elif not all_user_angles_valid_bs:
            backswing_player_angle_metric_val = "N/A (사용자 각도 부족)"
    score_details[category_backswing]["details"]["backswing_angle_comparison"] = {
        "value": backswing_player_angle_metric_val,
        "score": backswing_player_angle_score,
        "max_score": 10,
    }
    score_details[category_backswing]["subtotal"] += backswing_player_angle_score
    total_score = (
        score_details[category_impact]["subtotal"]
        + score_details[category_backswing]["subtotal"]
    )
    return {
        "total_score": total_score,
        "max_score": MAX_POSSIBLE_SCORE,
        "percentage": (
            (total_score / MAX_POSSIBLE_SCORE) * 100 if MAX_POSSIBLE_SCORE > 0 else 0
        ),
        "categories": score_details,
    }


def print_score_report(score_result):
    if not score_result:
        print("No score data available.")
        return
    print("\n" + "=" * 60)
    print("⚽ KICK PERFORMANCE SCORE REPORT ⚽".center(60))
    print("=" * 60)
    max_total_score = score_result.get("max_score", 100)
    print(
        f"\n🏆 TOTAL SCORE: {score_result['total_score']}/{max_total_score} ({score_result['percentage']:.1f}%)"
    )

    for category_name, category_data in score_result["categories"].items():
        category_title = category_name.replace("_", " ").title()
        current_category_max_score = sum(
            detail.get("max_score", 0) for detail in category_data["details"].values()
        )
        print(
            f"\n📊 {category_title}: {category_data['subtotal']}/{current_category_max_score} points"
        )
        line_len = max(
            30,
            len(category_title)
            + len(str(category_data["subtotal"]))
            + len(str(current_category_max_score))
            + 15,
        )
        print("-" * line_len)
        for detail_name, detail_data in category_data["details"].items():
            detail_title = detail_name.replace("_", " ").title()
            print(
                f"  • {detail_title}: {detail_data['score']}/{detail_data['max_score']} points"
            )
            print(f"    Value: {detail_data['value']}")

    percentage = score_result["percentage"]
    if percentage >= 90:
        grade = "A+ (Excellent)"
    elif percentage >= 80:
        grade = "A (Very Good)"
    elif percentage >= 70:
        grade = "B+ (Good)"
    elif percentage >= 60:
        grade = "B (Above Average)"
    elif percentage >= 50:
        grade = "C (Average)"
    elif percentage >= 40:
        grade = "D (Below Average)"
    else:
        grade = "F (Needs Improvement)"
    print(f"\n🎯 GRADE: {grade}")
    print("=" * 60)


def print_kick_analysis_summary(
    determined_impact_frame_idx,
    determined_impact_info_arg,
    interpolated_impact_details,
    current_sampling_rate_to_pass,
    source_video_fps,
    global_pixel_to_cm_scale,
):
    global score_result_global, analysis_dataframe_global  # ✨ 전역 데이터프레임 사용 ✨
    print("\n\n--- ⚽ Kick Analysis Summary ⚽ ---")
    if (
        determined_impact_frame_idx == -1
        or determined_impact_info_arg is None
        or not determined_impact_info_arg
    ):
        print(
            "Final Analysis: Impact moment could not be determined or insufficient information."
        )
        score_result_global = None
        print("\n--- Summary End ---")
        return

    print(f"\n[1] Main Impact Moment (Based on actual analyzed frame)")
    print(
        f"  - Impact Frame Number (Index in sampled list): {determined_impact_frame_idx}"
    )

    pixel_to_cm_scale_to_use = get_safe_pixel_to_cm_scale(
        determined_impact_info_arg.get("pixel_to_cm_scale", global_pixel_to_cm_scale)
    )
    print(
        f"  - Used Pixel-cm Scale: {pixel_to_cm_scale_to_use:.4f} cm/pixel (Based on ball diameter {REAL_BALL_DIAMETER_CM}cm)"
    )

    if determined_impact_info_arg.get("is_ball_info_corrected"):
        print(f"  ℹ️ Ball position at impact was corrected using pre-impact frame data.")

    print(
        f"  - Kicking Foot (Estimated): {determined_impact_info_arg.get('kicking_foot', 'N/A')}"
    )
    print(
        f"  - Kicking Foot Part (Estimated): {determined_impact_info_arg.get('kicking_foot_part', 'N/A')}"
    )
    print(
        f"  - Ball Contact Region (Estimated): On '{determined_impact_info_arg.get('contact_region_on_ball', 'N/A')}' part of the ball"
    )

    for key, label, unit_is_cm in [
        ("distance", "Kicking Foot-Ball Dist (Raw)", True),
        ("distance_smoothed", "Kicking Foot-Ball Dist (Smoothed)", True),
        ("dist_ball_to_supporting_foot_ankle", "Support Foot (Ankle)-Ball Dist", True),
    ]:
        val_px = determined_impact_info_arg.get(key)
        if isinstance(val_px, (int, float)) and pd.notna(val_px):
            cm_text = (
                f" (~ {val_px * pixel_to_cm_scale_to_use:.2f} cm)" if unit_is_cm else ""
            )
            print(f"  - {label}: {val_px:.2f} pixels{cm_text}")

    fps_to_use_for_speed_calc = (
        source_video_fps / current_sampling_rate_to_pass
        if current_sampling_rate_to_pass > 0
        else source_video_fps
    )
    approach_vel_px_fr = determined_impact_info_arg.get("foot_ball_approach_velocity")
    if isinstance(approach_vel_px_fr, (int, float)) and pd.notna(approach_vel_px_fr):
        speed_km_h = convert_speed_to_kmh_normalized(
            approach_vel_px_fr, pixel_to_cm_scale_to_use, fps_to_use_for_speed_calc
        )
        print(f"  - Foot-Ball Approach Vel: {approach_vel_px_fr:.2f} px/fr")
        if approach_vel_px_fr < 0:
            print(
                f"    (Approx. {-speed_km_h:.2f} km/h approach, Analysis FPS: {fps_to_use_for_speed_calc:.1f})"
            )
        else:
            print(
                f"    (Approx. {speed_km_h:.2f} km/h separation, Analysis FPS: {fps_to_use_for_speed_calc:.1f})"
            )

    for key, label in [
        ("foot_pos", "Kicking Foot Pos @ Impact (x,y)"),
        ("supporting_foot_ankle_pos", "Supporting Foot Ankle Pos @ Impact (x,y)"),
        ("ball_center", "Ball Center Pos @ Impact (x,y)"),
    ]:
        print(f"  - {label}: {determined_impact_info_arg.get(key, 'N/A')}")

    ball_rad_px = determined_impact_info_arg.get("ball_radius")
    if isinstance(ball_rad_px, (int, float)) and pd.notna(ball_rad_px):
        cm_text = f" (~ {ball_rad_px * pixel_to_cm_scale_to_use:.1f} cm)"
        print(f"  - Ball Radius @ Impact: {ball_rad_px:.1f} px{cm_text}")

    if determined_impact_info_arg.get(
        "is_ball_predicted", False
    ) and not determined_impact_info_arg.get("is_ball_info_corrected"):
        print(f"  ⚠️ Warning: Ball info at impact might be a predicted value.")
    if determined_impact_info_arg.get("occlusion_recovered", False):
        print(f"  ℹ️ Info: This ball info was recovered via occlusion handling logic.")

    max_ball_speed_val_px_fr = determined_impact_info_arg.get("max_ball_speed_px_fr")
    if pd.notna(max_ball_speed_val_px_fr):
        max_ball_speed_cm_s = (
            max_ball_speed_val_px_fr
            * pixel_to_cm_scale_to_use
            * fps_to_use_for_speed_calc
        )
        max_ball_speed_km_h = max_ball_speed_cm_s * CM_S_TO_KM_H
        print(
            f"  - Max Ball Speed After Impact (Est.): {max_ball_speed_val_px_fr:.2f} px/fr (~ {max_ball_speed_km_h:.2f} km/h)"
        )

    ball_speed_at_impact_px_fr = determined_impact_info_arg.get("ball_speed_at_impact")
    if pd.notna(ball_speed_at_impact_px_fr):
        ball_speed_at_cm_s = (
            ball_speed_at_impact_px_fr
            * pixel_to_cm_scale_to_use
            * fps_to_use_for_speed_calc
        )
        ball_speed_at_km_h = ball_speed_at_cm_s * CM_S_TO_KM_H
        print(
            f"  - Ball Speed @ Impact Moment (Est.): {ball_speed_at_impact_px_fr:.2f} px/fr (~ {ball_speed_at_km_h:.2f} km/h)"
        )

    backswing_knee_angle_val = determined_impact_info_arg.get(
        "backswing_kicking_knee_angle"
    )
    if pd.notna(backswing_knee_angle_val):
        print(
            f"  - Backswing Kicking Knee Angle: {backswing_knee_angle_val:.1f} degrees"
        )

    backswing_peak_dist_px = determined_impact_info_arg.get("backswing_peak_distance")
    if pd.notna(backswing_peak_dist_px):
        print(
            f"  - Backswing Peak Distance (Foot-Ball): {backswing_peak_dist_px:.2f} pixels (~ {backswing_peak_dist_px * pixel_to_cm_scale_to_use:.1f} cm)"
        )

    support_foot_stability_px_fr = determined_impact_info_arg.get(
        "supporting_foot_stability"
    )
    if pd.notna(support_foot_stability_px_fr):
        stability_cm_fr = support_foot_stability_px_fr * pixel_to_cm_scale_to_use
        stability_km_h = stability_cm_fr * fps_to_use_for_speed_calc * CM_S_TO_KM_H
        print(
            f"  - Supporting Foot Stability (Avg. Disp.): {support_foot_stability_px_fr:.2f} px/fr (~ {stability_cm_fr:.2f} cm/fr | ~ {stability_km_h:.4f} km/h)"
        )

    if interpolated_impact_details:
        print(f"\n[2] Frame Interpolation Analysis")
        refined_idx = interpolated_impact_details.get("refined_original_idx")
        min_dist_interp_px = interpolated_impact_details.get(
            "min_interpolated_distance"
        )
        if isinstance(refined_idx, (int, float)) and pd.notna(refined_idx):
            print(f"  - Refined Impact Moment (orig_idx): {refined_idx:.2f}")
        if isinstance(min_dist_interp_px, (int, float)) and pd.notna(
            min_dist_interp_px
        ):
            cm_text = f" (~ {min_dist_interp_px * pixel_to_cm_scale_to_use:.2f} cm)"
            print(
                f"  - Min. Interpolated Distance (Smoothed): {min_dist_interp_px:.2f} px{cm_text}"
            )
    elif USE_FRAME_INTERPOLATION:
        print("\n[2] Frame Interpolation: No result / Failed.")
    else:
        print("\n[2] Frame Interpolation: Disabled.")

    max_foot_speed_val_for_score = determined_impact_info_arg.get(
        "calculated_max_foot_swing_speed_px_fr", 0
    )
    if not pd.notna(max_foot_speed_val_for_score) or max_foot_speed_val_for_score == 0:
        if "kicking_foot_speed" in determined_impact_info_arg and pd.notna(
            determined_impact_info_arg.get("kicking_foot_speed")
        ):
            max_foot_speed_val_for_score = determined_impact_info_arg.get(
                "kicking_foot_speed", 0
            )

    # ✨ 점수 계산 시 analysis_dataframe_global과 인덱스 전달 ✨
    # DataFrame의 original_idx는 0부터 시작하므로, determined_impact_frame_idx (이것이 original_idx임)를 그대로 사용
    impact_df_idx = determined_impact_frame_idx
    backswing_df_idx = determined_impact_info_arg.get(
        "backswing_original_idx", -1
    )  # final_impact_info에 저장된 값

    score_result_global = calculate_kick_score(
        determined_impact_info_arg,
        pixel_to_cm_scale_to_use,
        fps=fps_to_use_for_speed_calc,
        max_foot_speed_px_fr=max_foot_speed_val_for_score,
        analysis_df_for_angles=analysis_dataframe_global,  # ✨ 전달
        impact_df_idx_for_angles=impact_df_idx,  # ✨ 전달
        backswing_df_idx_for_angles=backswing_df_idx,  # ✨ 전달
    )

    if score_result_global:
        print_score_report(score_result_global)
    print("\n--- Summary End ---")


# --- 미디어파이프 & 모델 로드 ---
mp_pose = mp.solutions.pose
pose_model = mp_pose.Pose(
    static_image_mode=False,
    model_complexity=1,
    min_detection_confidence=MP_POSE_MIN_DETECTION_CONFIDENCE,
    min_tracking_confidence=MP_POSE_MIN_TRACKING_CONFIDENCE,
)
yolo_model = None
try:
    primary_model_name = "yolov8m-seg.pt"
    fallback_model_name = "yolov8s-seg.pt"
    try:
        yolo_model = YOLO(primary_model_name)
        print(f"YOLO model ('{primary_model_name}') loaded successfully.")
    except Exception as e_primary_load:
        print(
            f"Failed to load primary YOLO model ('{primary_model_name}'): {e_primary_load}"
        )
        print(f"Attempting to load fallback model '{fallback_model_name}'...")
        try:
            yolo_model = YOLO(fallback_model_name)
            print(f"Fallback YOLO model ('{fallback_model_name}') loaded successfully.")
        except Exception as e_fallback_load:
            print(
                f"Failed to load fallback YOLO model ('{fallback_model_name}'): {e_fallback_load}"
            )
            print("Could not load any YOLO model. Exiting program.")
            exit()
except Exception as e_yolo_load:
    print(f"Unexpected error loading YOLO model: {e_yolo_load}")
    exit()

# --- 비디오 캡처 객체 생성 ---
cap = cv2.VideoCapture(video_path)
if not cap.isOpened():
    print(f"Failed to open video: {video_path}")
    exit()

original_video_fps = cap.get(cv2.CAP_PROP_FPS)
total_frames_in_video = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

if not original_video_fps or pd.isna(original_video_fps) or original_video_fps <= 0:
    original_video_fps = 30.0
    print(
        f"Warning: Could not get FPS from video or FPS is invalid. Using default {original_video_fps} FPS."
    )
else:
    print(f"Original video FPS: {original_video_fps:.2f}")

# --- 전역 스케일 계산 ---
# ⚽️ 공의 크기를 분석해서, 영상의 ‘픽셀 단위 길이’를 실제 cm 단위로 변환하는 스케일(비례계수)을 자동 추정
pixel_to_cm_scale_global = None
print(
    f"\nAttempting to measure stable ball size from the first {INITIAL_FRAMES_FOR_BALL_SIZE_EST} frames..."
)
print(
    "=> New Method: Finding the most STATIONARY ball in the initial sequence."
)  # 새로운 방식임을 명시

temp_cap_for_scale = cv2.VideoCapture(video_path)
# ✨ [수정] 공의 시간대별 위치, 반지름 정보를 모두 저장할 리스트
ball_data_over_time = []

frames_processed_for_scale_count = 0
initial_sampling_interval_scale = 1
if (
    SAMPLING_RATE_TARGET_FRAMES > 0
    and total_frames_in_video > INITIAL_FRAMES_FOR_BALL_SIZE_EST * 2
):
    base_main_sampling = max(1, total_frames_in_video // SAMPLING_RATE_TARGET_FRAMES)
    initial_sampling_interval_scale = max(
        1, base_main_sampling // 2 if base_main_sampling > 1 else 1
    )

temp_frame_idx_scale = 0
while (
    temp_cap_for_scale.isOpened()
    and frames_processed_for_scale_count < INITIAL_FRAMES_FOR_BALL_SIZE_EST
):
    ret_temp, frame_temp = temp_cap_for_scale.read()
    if not ret_temp:
        break

    if temp_frame_idx_scale % initial_sampling_interval_scale == 0:
        yolo_preds_temp = yolo_model.predict(
            source=frame_temp,
            classes=[32],
            conf=BALL_SIZE_EST_MIN_CONFIDENCE,
            verbose=False,
        )
        if yolo_preds_temp and yolo_preds_temp[0].boxes:
            # 여러 공이 감지될 경우 가장 큰 공을 선택
            best_ball_in_frame = None
            max_radius_in_frame = 0

            for box_info in yolo_preds_temp[0].boxes:
                if int(box_info.cls) == 32:
                    x1, y1, x2, y2 = box_info.xyxy[0].cpu().numpy().astype(int)
                    rad_val = max(5, int(min(x2 - x1, y2 - y1) / 2.0))
                    if rad_val > max_radius_in_frame:
                        max_radius_in_frame = rad_val
                        best_ball_in_frame = {
                            "radius": rad_val,
                            "center": ((x1 + x2) / 2.0, (y1 + y2) / 2.0),
                        }

            if best_ball_in_frame:
                # ✨ [수정] 프레임 인덱스와 함께 공의 반지름, 중심점 정보를 저장
                ball_data_over_time.append(
                    {
                        "frame_index": frames_processed_for_scale_count,
                        "radius": best_ball_in_frame["radius"],
                        "center": best_ball_in_frame["center"],
                    }
                )

        frames_processed_for_scale_count += 1
    temp_frame_idx_scale += 1
temp_cap_for_scale.release()

# ✨ [수정] 가장 안정적인 공을 찾는 로직 시작
if len(ball_data_over_time) >= 2:  # 최소 2개 이상의 데이터가 있어야 움직임 비교 가능
    displacements = []
    # 연속된 탐지 사이의 변위(움직임) 계산
    for i in range(1, len(ball_data_over_time)):
        prev_center = np.array(ball_data_over_time[i - 1]["center"])
        curr_center = np.array(ball_data_over_time[i]["center"])
        distance = np.linalg.norm(curr_center - prev_center)
        displacements.append(distance)

    if displacements:
        # 가장 움직임이 적었던 순간(프레임)을 찾습니다.
        min_disp_index = np.argmin(displacements)

        # 변위는 두 프레임 사이의 값이므로, 안정적인 쌍 중 두 번째 프레임의 데이터를 사용
        most_stable_ball_data = ball_data_over_time[min_disp_index + 1]
        stable_ball_radius_px = most_stable_ball_data["radius"]

        pixel_to_cm_scale_global = calculate_dynamic_pixel_to_cm_scale(
            stable_ball_radius_px, REAL_BALL_DIAMETER_CM
        )

        if pixel_to_cm_scale_global:
            print(
                f"Stable ball found in frame {most_stable_ball_data['frame_index']} (movement: {min(displacements):.2f}px)."
            )
            print(
                f"Using radius {stable_ball_radius_px:.2f}px from most stable frame. Calculated global scale: {pixel_to_cm_scale_global:.4f} cm/pixel"
            )
        else:
            print(
                "Failed to calculate global scale from stable ball radius. Will use default scale."
            )
            pixel_to_cm_scale_global = DEFAULT_PIXEL_TO_CM_SCALE
    else:
        # 변위 계산이 불가능한 경우(데이터 부족), 첫 번째로 찾은 공을 사용
        stable_ball_radius_px = ball_data_over_time[0]["radius"]
        pixel_to_cm_scale_global = calculate_dynamic_pixel_to_cm_scale(
            stable_ball_radius_px, REAL_BALL_DIAMETER_CM
        )
        print("Could not calculate displacements, using the first detected ball.")

elif len(ball_data_over_time) == 1:
    # 공이 하나만 감지된 경우, 해당 공의 데이터를 사용
    stable_ball_radius_px = ball_data_over_time[0]["radius"]
    pixel_to_cm_scale_global = calculate_dynamic_pixel_to_cm_scale(
        stable_ball_radius_px, REAL_BALL_DIAMETER_CM
    )
    print("Only one ball detected, using its data for scale calculation.")

else:
    print(
        "Ball detection failed or no data collected in early frames. Will use default scale."
    )
    pixel_to_cm_scale_global = DEFAULT_PIXEL_TO_CM_SCALE

pixel_to_cm_scale_global = get_safe_pixel_to_cm_scale(pixel_to_cm_scale_global)
print(f"Final global scale to be applied: {pixel_to_cm_scale_global:.4f} cm/pixel")

# --- ⭐️⭐️ 메인 영상 처리 루프 ⭐️⭐️ ---

all_processed_frames, pose_results_list, ball_detections_list = [], [], []
raw_ball_detections_list = []

sampling_interval_calc = 1
if (
    SAMPLING_RATE_TARGET_FRAMES > 0
    and total_frames_in_video > SAMPLING_RATE_TARGET_FRAMES
):
    sampling_interval_calc = max(
        1, total_frames_in_video // SAMPLING_RATE_TARGET_FRAMES
    )

print(
    f"\nTotal frames: {total_frames_in_video}, Sampling interval: {sampling_interval_calc}, Expected frames to process: {total_frames_in_video // sampling_interval_calc if sampling_interval_calc > 0 else total_frames_in_video}"
)

current_frame_idx_in_video, processed_sampled_frame_count = 0, 0
video_processing_start_time = time.time()
print("Starting video frame processing...")
cap.set(cv2.CAP_PROP_POS_FRAMES, 0)
analysis_dataframe = pd.DataFrame()  # 로컬 변수로 우선 사용, 나중에 전역에 할당
kalman_tracker = KalmanBallTracker()

while cap.isOpened():
    ret, current_frame_image_orig = cap.read()
    if not ret:
        break

    current_frame_image = current_frame_image_orig
    if manual_rotate_code is not None:  # manual_rotate_code가 설정되었으면 회전 적용
        current_frame_image = cv2.rotate(current_frame_image_orig, manual_rotate_code)

    if (
        SAMPLING_RATE_TARGET_FRAMES > 0
        and processed_sampled_frame_count
        >= (
            total_frames_in_video // sampling_interval_calc
            if sampling_interval_calc > 0
            else total_frames_in_video
        )
        and sampling_interval_calc > 1
    ):
        break

    if current_frame_idx_in_video % sampling_interval_calc == 0:
        all_processed_frames.append(current_frame_image.copy())

        rgb_frame_for_pose = cv2.cvtColor(current_frame_image, cv2.COLOR_BGR2RGB)
        pose_detection_result = pose_model.process(rgb_frame_for_pose)
        pose_results_list.append(pose_detection_result.pose_landmarks)

        yolo_predictions = yolo_model.predict(
            source=current_frame_image,
            classes=[32],
            conf=YOLO_CONF_THRESHOLD,
            iou=YOLO_IOU_THRESHOLD,
            verbose=False,
        )

        detected_balls_in_current_frame = []
        ball_masks_from_segmentation_this_frame = []

        if yolo_predictions and yolo_predictions[0].boxes:
            boxes = yolo_predictions[0].boxes
            masks = yolo_predictions[0].masks

            for i_box, box_info in enumerate(boxes):
                if int(box_info.cls) == 32:
                    x1, y1, x2, y2 = box_info.xyxy[0].cpu().numpy().astype(int)
                    conf_score = float(box_info.conf[0].cpu().numpy())
                    cx, cy, rad_val = (
                        (x1 + x2) / 2.0,
                        (y1 + y2) / 2.0,
                        max(5, int(min(x2 - x1, y2 - y1) / 2.0)),
                    )

                    ball_entry = {
                        "box": [x1, y1, x2, y2],
                        "center": (int(cx), int(cy)),
                        "radius": rad_val,
                        "confidence": conf_score,
                        "mask_points": None,
                    }

                    if (
                        USE_BALL_SEGMENTATION
                        and masks is not None
                        and i_box < len(masks.xy)
                    ):
                        mask_polygon_normalized = masks.xy[i_box]
                        mask_polygon_scaled = (mask_polygon_normalized).astype(np.int32)
                        ball_entry["mask_points"] = mask_polygon_scaled
                        ball_masks_from_segmentation_this_frame.append(
                            {
                                "box": [x1, y1, x2, y2],
                                "mask_points": mask_polygon_scaled,
                                "confidence": conf_score,
                            }
                        )

                    detected_balls_in_current_frame.append(ball_entry)

        raw_ball_detections_list.append(detected_balls_in_current_frame.copy())

        selected_ball_for_this_frame = None
        tracked_ball_candidate = None

        if (
            USE_PREVIOUS_BALL_INFO_TRACKING
            and ball_detections_list
            and ball_detections_list[-1] is not None
        ):
            last_known_ball_info = ball_detections_list[-1]
            min_tracking_score_val = float("inf")

            for ball_cand_item in detected_balls_in_current_frame:
                curr_center_np, curr_radius_val = (
                    np.array(ball_cand_item["center"]),
                    ball_cand_item["radius"],
                )
                if (
                    last_known_ball_info.get("center") is None
                    or last_known_ball_info.get("radius") is None
                ):
                    continue

                prev_center_np, prev_radius_val = (
                    np.array(last_known_ball_info["center"]),
                    last_known_ball_info["radius"],
                )

                dist_s_calc = np.linalg.norm(curr_center_np - prev_center_np)
                rad_diff_ratio_calc = (
                    abs(curr_radius_val - prev_radius_val) / prev_radius_val
                    if prev_radius_val > 1e-6
                    else float("inf")
                )

                if dist_s_calc > BALL_TRACKING_MAX_DIST_FROM_PREV or (
                    prev_radius_val > 1e-6
                    and rad_diff_ratio_calc > BALL_TRACKING_MAX_RADIUS_DIFF_RATIO
                ):
                    continue

                track_s_calc = (
                    dist_s_calc * BALL_TRACKING_DIST_WEIGHT
                    + abs(curr_radius_val - prev_radius_val)
                    * BALL_TRACKING_RADIUS_WEIGHT
                    + (1.0 - ball_cand_item["confidence"])
                    * abs(BALL_TRACKING_CONF_WEIGHT)
                )

                if track_s_calc < min_tracking_score_val:
                    min_tracking_score_val = track_s_calc
                    tracked_ball_candidate = ball_cand_item

            if tracked_ball_candidate is not None:
                selected_ball_for_this_frame = tracked_ball_candidate
                selected_ball_for_this_frame["is_predicted"] = False
                selected_ball_for_this_frame["predicted_count"] = 0

        if selected_ball_for_this_frame is None and detected_balls_in_current_frame:
            curr_pose_lm = pose_results_list[-1]
            foot_selected_ball = None
            if curr_pose_lm:
                foot_lms = [
                    fp
                    for fp in [
                        get_specific_landmark_position(
                            curr_pose_lm, lme, current_frame_image.shape
                        )
                        for lme in [
                            mp_pose.PoseLandmark.RIGHT_FOOT_INDEX,
                            mp_pose.PoseLandmark.LEFT_FOOT_INDEX,
                        ]
                    ]
                    if fp
                ]
                if foot_lms:
                    max_p_score = float("-inf")
                    for ball_c_item in detected_balls_in_current_frame:
                        min_d_f = min(
                            [
                                np.linalg.norm(
                                    np.array(ball_c_item["center"]) - np.array(fp_c)
                                )
                                for fp_c in foot_lms
                            ]
                        )
                        p_score = (
                            ball_c_item["confidence"]
                            - BALL_SELECTION_ALPHA_FOOT_DIST * min_d_f
                        )
                        if p_score > max_p_score:
                            max_p_score = p_score
                            foot_selected_ball = ball_c_item

            if foot_selected_ball is not None:
                selected_ball_for_this_frame = foot_selected_ball
            else:
                selected_ball_for_this_frame = max(
                    detected_balls_in_current_frame, key=lambda xb: xb["confidence"]
                )

            if selected_ball_for_this_frame is not None:
                selected_ball_for_this_frame["is_predicted"] = False
                selected_ball_for_this_frame["predicted_count"] = 0

        # if (
        #     selected_ball_for_this_frame is None
        #     and USE_OCCLUSION_HANDLING_PAPER_LOGIC
        #     and len(raw_ball_detections_list) > 1
        #     and raw_ball_detections_list[-2] is not None
        # ):
        #     occlusion_handled_ball = handle_occlusion_with_new_appearance( # 이거 안쓰는 함수
        #         detected_balls_in_current_frame, raw_ball_detections_list[-2]
        #     )
        #     if occlusion_handled_ball is not None:
        #         selected_ball_for_this_frame = occlusion_handled_ball
        #         selected_ball_for_this_frame["is_predicted"] = False
        #         selected_ball_for_this_frame["predicted_count"] = 0
        #         selected_ball_for_this_frame["occlusion_recovered"] = True

        if (
            selected_ball_for_this_frame is None
            and USE_OCCLUSION_HANDLING_PAPER_LOGIC
            and len(raw_ball_detections_list) > 1
            and raw_ball_detections_list[-2] is not None
            and len(ball_detections_list) >= 2
        ):
            prev_ball = ball_detections_list[-2]
            last_ball = ball_detections_list[-1]
            if prev_ball.get("center") and last_ball.get("center"):
                velocity = np.array(last_ball["center"]) - np.array(prev_ball["center"])
                occlusion_handled_ball = find_valid_ball_candidate(
                    current_candidates=detected_balls_in_current_frame,
                    previous_position=np.array(last_ball["center"]),
                    previous_velocity=velocity,
                )
                if occlusion_handled_ball is not None:
                    selected_ball_for_this_frame = (
                        occlusion_handled_ball  # 대안함수 써서 교체한 ver
                    )
                    selected_ball_for_this_frame["is_predicted"] = False
                    selected_ball_for_this_frame["predicted_count"] = 0
                    selected_ball_for_this_frame["occlusion_recovered"] = True

        # 공이 가려져서 YOLO로 감지되지 않았을 경우”,최근 프레임에서의 위치와 속도를 이용해 다음 위치를 예측해 selected_ball_for_this_frame에 할당
        # 칼만 필터로 변환
        # if (
        #     selected_ball_for_this_frame is None
        #     and USE_PREDICTIVE_TRACKING
        #     and USE_PREVIOUS_BALL_INFO_TRACKING
        #     and ball_detections_list
        #     and ball_detections_list[-1] is not None
        # ):
        #     last_ball_info = ball_detections_list[-1]
        #     last_predicted_count = last_ball_info.get("predicted_count", 0)

        #     if last_predicted_count < MAX_CONSECUTIVE_PREDICTIONS:
        #         predicted_center_x, predicted_center_y = last_ball_info["center"]
        #         velocity_x, velocity_y = 0, 0

        #         if (
        #             len(ball_detections_list) >= 2
        #             and ball_detections_list[-2] is not None
        #             and not last_ball_info.get("is_predicted", False)
        #             and not ball_detections_list[-2].get("is_predicted", False)
        #             and ball_detections_list[-2].get("center") is not None
        #             and last_ball_info.get("center") is not None
        #         ):
        #             prev_center = ball_detections_list[-2]["center"]
        #             current_center = last_ball_info["center"]
        #             velocity_x = current_center[0] - prev_center[0]
        #             velocity_y = current_center[1] - prev_center[1]

        #         predicted_center_x += velocity_x
        #         predicted_center_y += velocity_y
        #         radius = last_ball_info["radius"]

        #         box_width = (
        #             (last_ball_info["box"][2] - last_ball_info["box"][0])
        #             if last_ball_info.get("box") and len(last_ball_info["box"]) == 4
        #             else 2 * radius
        #         )
        #         box_height = (
        #             (last_ball_info["box"][3] - last_ball_info["box"][1])
        #             if last_ball_info.get("box") and len(last_ball_info["box"]) == 4
        #             else 2 * radius
        #         )
        #         pred_x1 = int(predicted_center_x - box_width / 2)
        #         pred_y1 = int(predicted_center_y - box_height / 2)
        #         pred_x2 = int(predicted_center_x + box_width / 2)
        #         pred_y2 = int(predicted_center_y + box_height / 2)

        #         selected_ball_for_this_frame = {
        #             "box": [pred_x1, pred_y1, pred_x2, pred_y2],
        #             "center": (int(predicted_center_x), int(predicted_center_y)),
        #             "radius": radius,
        #             "confidence": last_ball_info.get(
        #                 "confidence", YOLO_CONF_THRESHOLD * 0.8
        #             )
        #             * PREDICTED_BALL_CONFIDENCE_DECAY,
        #             "mask_points": None,
        #             "is_predicted": True,
        #             "predicted_count": last_predicted_count + 1,
        #         }
        # ... 루프 내부에서
        if selected_ball_for_this_frame is None and USE_PREDICTIVE_TRACKING:
            # 이전 프레임에서 탐지된 공이 있는 경우 업데이트
            if ball_detections_list and ball_detections_list[-1] is not None:
                last_ball_info = ball_detections_list[-1]
                center = last_ball_info.get("center")

                if center:
                    kalman_tracker.update(center)
                    predicted_center = kalman_tracker.predict()
                    radius = last_ball_info.get("radius", 10)

                    # 예측된 박스 생성
                    selected_ball_for_this_frame = {
                        "box": [
                            int(predicted_center[0] - radius),
                            int(predicted_center[1] - radius),
                            int(predicted_center[0] + radius),
                            int(predicted_center[1] + radius),
                        ],
                        "center": predicted_center,
                        "radius": radius,
                        "confidence": last_ball_info.get("confidence", 0.5) * 0.9,
                        "mask_points": None,
                        "is_predicted": True,
                        "predicted_count": last_ball_info.get("predicted_count", 0) + 1,
                    }
        # ________
        if (
            selected_ball_for_this_frame
            and not selected_ball_for_this_frame.get("is_predicted", False)
            and USE_BALL_SEGMENTATION
            and yolo_model is not None
            and ball_masks_from_segmentation_this_frame
        ):
            best_mask_iou = -1.0
            associated_mask_points = None
            sbf_box_from_detection = selected_ball_for_this_frame.get("box")

            if sbf_box_from_detection:
                for seg_info in ball_masks_from_segmentation_this_frame:
                    if seg_info.get("box"):
                        iou = calculate_iou(sbf_box_from_detection, seg_info["box"])
                        if iou > best_mask_iou:
                            best_mask_iou = iou
                            associated_mask_points = seg_info["mask_points"]

                if best_mask_iou >= MASK_ASSOCIATION_IOU_THRESHOLD:
                    selected_ball_for_this_frame["mask_points"] = associated_mask_points
                else:
                    selected_ball_for_this_frame["mask_points"] = None
            else:
                selected_ball_for_this_frame["mask_points"] = None
        elif (
            selected_ball_for_this_frame
            and "mask_points" not in selected_ball_for_this_frame
        ):
            selected_ball_for_this_frame["mask_points"] = None

        ball_detections_list.append(selected_ball_for_this_frame)
        processed_sampled_frame_count += 1

        if processed_sampled_frame_count % 50 == 0:
            print(
                f"     ... Processed {processed_sampled_frame_count} frames (Original frame: {current_frame_idx_in_video})"
            )

    current_frame_idx_in_video += 1

cap.release()
# out.release()
cv2.destroyAllWindows()
print("✅ 영상 저장 완료")
print(
    f"Video frame processing complete: {processed_sampled_frame_count} frames processed in ({time.time()-video_processing_start_time:.2f}s)"
)

# === 여기서 모든 프레임의 크기를 통일 ===
# cv2.VideoWriter() 사용 시, 모든 프레임은 동일한 해상도여야 저장 가능
max_h = max(f.shape[0] for f in all_processed_frames)
max_w = max(f.shape[1] for f in all_processed_frames)


def resize_to_fixed_shape(frame, target_shape):
    return cv2.resize(
        frame, (target_shape[1], target_shape[0]), interpolation=cv2.INTER_LINEAR
    )


target_shape = (max_h, max_w)
all_processed_frames = [
    resize_to_fixed_shape(f, target_shape) for f in all_processed_frames
]
# --- 데이터프레임 생성 및 처리 ---
if DEBUG_MODE:
    print("Creating DataFrame for impact analysis...")
impact_analysis_start_time = time.time()
frame_analysis_data_list = []


def calculate_angle_for_df(p1, p2, p3):  # 데이터프레임용
    # None 또는 NaN 값이 포함된 좌표일 경우 바로 nan 반환
    if (
        p1 is None
        or p2 is None
        or p3 is None
        or any([not isinstance(p, (list, tuple, np.ndarray)) for p in (p1, p2, p3)])
        or any(
            [
                (
                    np.isnan(p).any()
                    if isinstance(p, np.ndarray)
                    else np.isnan(np.array(p)).any()
                )
                for p in (p1, p2, p3)
            ]
        )
    ):
        return np.nan

    p1, p2, p3 = np.array(p1), np.array(p2), np.array(p3)
    v1 = p1 - p2
    v2 = p3 - p2

    norm_v1 = np.linalg.norm(v1)
    norm_v2 = np.linalg.norm(v2)

    if norm_v1 < 1e-6 or norm_v2 < 1e-6:
        return np.nan

    cosine_angle = np.dot(v1, v2) / (norm_v1 * norm_v2)
    angle = np.arccos(np.clip(cosine_angle, -1.0, 1.0))
    return np.degrees(angle)


for i, (pose_data_item, ball_data_item) in enumerate(
    zip(pose_results_list, ball_detections_list)
):
    current_frame_img_shape = all_processed_frames[i].shape

    # 주요 랜드마크 좌표 추출
    r_shoulder_pos = get_specific_landmark_position(
        pose_data_item, mp_pose.PoseLandmark.RIGHT_SHOULDER, current_frame_img_shape
    )
    l_shoulder_pos = get_specific_landmark_position(
        pose_data_item, mp_pose.PoseLandmark.LEFT_SHOULDER, current_frame_img_shape
    )
    r_hip_pos = get_specific_landmark_position(
        pose_data_item, mp_pose.PoseLandmark.RIGHT_HIP, current_frame_img_shape
    )
    l_hip_pos = get_specific_landmark_position(
        pose_data_item, mp_pose.PoseLandmark.LEFT_HIP, current_frame_img_shape
    )
    r_knee_pos = get_specific_landmark_position(
        pose_data_item, mp_pose.PoseLandmark.RIGHT_KNEE, current_frame_img_shape
    )
    l_knee_pos = get_specific_landmark_position(
        pose_data_item, mp_pose.PoseLandmark.LEFT_KNEE, current_frame_img_shape
    )
    r_ankle_pos = get_specific_landmark_position(
        pose_data_item, mp_pose.PoseLandmark.RIGHT_ANKLE, current_frame_img_shape
    )
    l_ankle_pos = get_specific_landmark_position(
        pose_data_item, mp_pose.PoseLandmark.LEFT_ANKLE, current_frame_img_shape
    )
    r_foot_idx_pos = get_specific_landmark_position(
        pose_data_item, mp_pose.PoseLandmark.RIGHT_FOOT_INDEX, current_frame_img_shape
    )
    l_foot_idx_pos = get_specific_landmark_position(
        pose_data_item, mp_pose.PoseLandmark.LEFT_FOOT_INDEX, current_frame_img_shape
    )

    frame_entry = {
        "original_idx": i,
        "is_ball_predicted": (
            ball_data_item.get("is_predicted", False)
            if isinstance(ball_data_item, dict)
            else False
        ),
        "occlusion_recovered": (
            ball_data_item.get("occlusion_recovered", False)
            if isinstance(ball_data_item, dict)
            else False
        ),
        "right_foot_index_pos": r_foot_idx_pos,
        "left_foot_index_pos": l_foot_idx_pos,
        "right_ankle_pos": r_ankle_pos,
        "left_ankle_pos": l_ankle_pos,
        "pixel_to_cm_scale": pixel_to_cm_scale_global,
        # ✨ 엉덩이, 무릎, 발목 각도 계산 추가 ✨
        "right_hip_angle": calculate_angle_for_df(
            r_shoulder_pos, r_hip_pos, r_knee_pos
        ),
        "left_hip_angle": calculate_angle_for_df(l_shoulder_pos, l_hip_pos, l_knee_pos),
        "right_knee_angle": calculate_angle_for_df(r_hip_pos, r_knee_pos, r_ankle_pos),
        "left_knee_angle": calculate_angle_for_df(l_hip_pos, l_knee_pos, l_ankle_pos),
        "right_ankle_angle": calculate_angle_for_df(
            r_knee_pos, r_ankle_pos, r_foot_idx_pos
        ),
        "left_ankle_angle": calculate_angle_for_df(
            l_knee_pos, l_ankle_pos, l_foot_idx_pos
        ),
    }

    dist_r_foot_ball, dist_l_foot_ball = np.nan, np.nan
    if isinstance(ball_data_item, dict) and ball_data_item.get("center") is not None:
        ball_center_coords_df = ball_data_item["center"]
        if r_foot_idx_pos:
            dist_r_foot_ball = np.hypot(
                r_foot_idx_pos[0] - ball_center_coords_df[0],
                r_foot_idx_pos[1] - ball_center_coords_df[1],
            )
        if l_foot_idx_pos:
            dist_l_foot_ball = np.hypot(
                l_foot_idx_pos[0] - ball_center_coords_df[0],
                l_foot_idx_pos[1] - ball_center_coords_df[1],
            )

    frame_entry["min_dist_right_foot_to_ball"] = dist_r_foot_ball
    frame_entry["min_dist_left_foot_to_ball"] = dist_l_foot_ball

    if (
        not pose_data_item
        or not isinstance(ball_data_item, dict)
        or ball_data_item.get("center") is None
        or ball_data_item.get("radius") is None
        or ball_data_item["radius"] <= 0
    ):
        frame_analysis_data_list.append(frame_entry)
        continue

    ball_center_coords_df, ball_radius_val_df = (
        ball_data_item["center"],
        ball_data_item["radius"],
    )
    closest_foot_details = None
    current_kicking_foot_name = np.nan
    current_kicking_foot_part_name = "toe"
    current_foot_part_coords = None
    current_min_dist_val = np.nan
    foot_candidates_info = []

    if USE_ENHANCED_FOOT_MODEL:
        for side_name_enh in ["right", "left"]:
            foot_enh_info = enhance_foot_model(
                pose_data_item, side_name_enh, current_frame_img_shape
            )
            if foot_enh_info:
                for part_key_name, part_coords_px in foot_enh_info.items():
                    if part_key_name in ("direction_vector", "foot_length_px") or not (
                        isinstance(part_coords_px, tuple) and len(part_coords_px) == 2
                    ):
                        continue
                    dist_to_ball = np.hypot(
                        part_coords_px[0] - ball_center_coords_df[0],
                        part_coords_px[1] - ball_center_coords_df[1],
                    )
                    foot_candidates_info.append(
                        (dist_to_ball, side_name_enh, part_key_name, part_coords_px)
                    )
    else:
        if pd.notna(dist_r_foot_ball) and r_foot_idx_pos:
            foot_candidates_info.append(
                (dist_r_foot_ball, "right", "toe", r_foot_idx_pos)
            )
        if pd.notna(dist_l_foot_ball) and l_foot_idx_pos:
            foot_candidates_info.append(
                (dist_l_foot_ball, "left", "toe", l_foot_idx_pos)
            )

    if foot_candidates_info:
        # 1. 어느 발이 공과 가장 가까운지 판단하여 '차는 발(kicking_foot)'과 기본 좌표만 결정합니다.
        candidates_to_check = foot_candidates_info
        if main_kicking_foot_preference != "auto":
            preferred_candidates = [
                c for c in foot_candidates_info if c[1] == main_kicking_foot_preference
            ]
            if preferred_candidates:
                candidates_to_check = preferred_candidates

        closest_foot_details = min(
            candidates_to_check, key=lambda x: x[0] if pd.notna(x[0]) else float("inf")
        )

        if closest_foot_details and pd.notna(closest_foot_details[0]):
            (
                current_min_dist_val,
                current_kicking_foot_name,
                _,  # 부위(part)에 대한 초기 추측은 이제 사용하지 않고 무시합니다.
                current_foot_part_coords,
            ) = closest_foot_details

            # 2. 결정된 '차는 발' 정보를 바탕으로, 새로운 함수를 호출하여 타격 부위를 최종 결정합니다.
            current_kicking_foot_part_name = determine_kick_part_by_whole_leg_flow_v3_1(
                processed_sampled_frame_count,  # -> current_idx
                current_kicking_foot_name,  # -> kicking_foot_side
                ball_center_coords_df,  # -> ball_center
                pose_results_list,  # -> pose_results_list
                # pose_data_item,  # -> current_pose
                all_processed_frames,
            )
        else:  # 만약 가장 가까운 발조차 찾지 못했다면
            current_kicking_foot_name = "N/A"
            current_kicking_foot_part_name = "N/A"
            current_foot_part_coords = None
            current_min_dist_val = np.nan

    frame_entry.update(
        {
            "distance": current_min_dist_val,
            "kicking_foot": current_kicking_foot_name,
            "kicking_foot_part": current_kicking_foot_part_name,
            "ball_radius": ball_radius_val_df,
            "ball_center": ball_center_coords_df,
            "ball_box": ball_data_item.get("box"),
            "foot_pos": current_foot_part_coords,
        }
    )
    frame_analysis_data_list.append(frame_entry)

expected_cols = [
    "original_idx",
    "distance",
    "kicking_foot",
    "kicking_foot_part",
    "ball_radius",
    "ball_center",
    "ball_box",
    "foot_pos",
    "pixel_to_cm_scale",
    "distance_smoothed",
    "foot_ball_approach_velocity",
    "foot_ball_approach_acceleration",
    "foot_pos_x",
    "foot_pos_y",
    "kicking_foot_velocity_x",
    "kicking_foot_velocity_y",
    "kicking_foot_speed",
    "kicking_foot_acceleration_scalar",
    "is_ball_predicted",
    "occlusion_recovered",
    "right_foot_index_pos",
    "left_foot_index_pos",
    "min_dist_right_foot_to_ball",
    "min_dist_left_foot_to_ball",
    "right_ankle_pos",
    "left_ankle_pos",
    # ✨ 추가된 각도 컬럼 ✨
    "right_hip_angle",
    "left_hip_angle",
    "right_knee_angle",
    "left_knee_angle",
    "right_ankle_angle",
    "left_ankle_angle",
]
if USE_DIFFERENTIAL_FOOT_SPEED_CHECK:
    expected_cols.extend(["right_foot_speed", "left_foot_speed"])
if USE_INTER_FOOT_DISTANCE_CHECK:
    expected_cols.append("inter_foot_distance")
if USE_BALL_SHRINK_DETECTION:
    expected_cols.extend(["ball_radius_smoothed", "ball_radius_change"])
if USE_BACKSWING_APPROACH_PATTERN:
    expected_cols.extend(
        ["min_dist_right_foot_to_ball_smoothed", "min_dist_left_foot_to_ball_smoothed"]
    )
if CALCULATE_SUPPORTING_FOOT_STABILITY:
    expected_cols.extend(["supporting_foot_ankle_pos", "supporting_foot_displacement"])

analysis_dataframe = pd.DataFrame(frame_analysis_data_list)


for col in expected_cols:
    if col not in analysis_dataframe.columns:
        analysis_dataframe[col] = np.nan

numeric_cols_to_convert = [
    "ball_radius",
    "distance",
    "min_dist_right_foot_to_ball",
    "min_dist_left_foot_to_ball",
    # ✨ 추가된 각도 컬럼 ✨
    "right_hip_angle",
    "left_hip_angle",
    "right_knee_angle",
    "left_knee_angle",
    "right_ankle_angle",
    "left_ankle_angle",
]
for col in numeric_cols_to_convert:
    if col in analysis_dataframe.columns:
        analysis_dataframe[col] = pd.to_numeric(
            analysis_dataframe[col], errors="coerce"
        )

if analysis_dataframe.empty:
    print("Error: Analysis DataFrame is empty. Cannot proceed with further analysis.")
    exit()

analysis_dataframe = analysis_dataframe.sort_values("original_idx").reset_index(
    drop=True
)
analysis_dataframe["pixel_to_cm_scale"] = analysis_dataframe[
    "pixel_to_cm_scale"
].fillna(pixel_to_cm_scale_global)
analysis_dataframe["ball_radius"] = (
    analysis_dataframe["ball_radius"].ffill().bfill().fillna(10.0)
)
analysis_dataframe["distance"] = analysis_dataframe["distance"].fillna(float("inf"))
analysis_dataframe["min_dist_right_foot_to_ball"] = analysis_dataframe[
    "min_dist_right_foot_to_ball"
].fillna(float("inf"))
analysis_dataframe["min_dist_left_foot_to_ball"] = analysis_dataframe[
    "min_dist_left_foot_to_ball"
].fillna(float("inf"))

actual_smoothing_win_size = (
    min(SMOOTHING_WINDOW_SIZE, len(analysis_dataframe))
    if len(analysis_dataframe) > 1
    else 1
)


# ⚽️ 킥 동작 분석의 핵심 시계열 지표들을 계산
# 거리 → 속도 → 가속도, 발의 위치 변화량 → 속도 계산, 양발 비교용 속도
if (
    "distance" in analysis_dataframe.columns
    and not analysis_dataframe["distance"].isna().all()
):
    analysis_dataframe["distance_smoothed"] = (
        analysis_dataframe["distance"]
        .rolling(window=actual_smoothing_win_size, center=True, min_periods=1)
        .mean()
        if USE_DISTANCE_SMOOTHING and actual_smoothing_win_size > 1
        else analysis_dataframe["distance"]
    )
else:
    analysis_dataframe["distance_smoothed"] = np.nan

if USE_BACKSWING_APPROACH_PATTERN:
    for side in ["right", "left"]:
        dist_col = f"min_dist_{side}_foot_to_ball"
        smooth_col = f"min_dist_{side}_foot_to_ball_smoothed"
        if (
            dist_col in analysis_dataframe.columns
            and not analysis_dataframe[dist_col].isna().all()
        ):
            analysis_dataframe[smooth_col] = (
                analysis_dataframe[dist_col]
                .rolling(window=actual_smoothing_win_size, center=True, min_periods=1)
                .mean()
            )
        else:
            analysis_dataframe[smooth_col] = np.nan

if (
    "distance_smoothed" in analysis_dataframe.columns
    and not analysis_dataframe["distance_smoothed"].isna().all()
):
    analysis_dataframe["foot_ball_approach_velocity"] = analysis_dataframe[
        "distance_smoothed"
    ].diff()
    if (
        "foot_ball_approach_velocity" in analysis_dataframe.columns
        and not analysis_dataframe["foot_ball_approach_velocity"].isna().all()
    ):
        analysis_dataframe["foot_ball_approach_acceleration"] = analysis_dataframe[
            "foot_ball_approach_velocity"
        ].diff()
    else:
        analysis_dataframe["foot_ball_approach_acceleration"] = np.nan
else:
    analysis_dataframe["foot_ball_approach_velocity"] = np.nan
    analysis_dataframe["foot_ball_approach_acceleration"] = np.nan

analysis_dataframe["foot_pos_x"] = analysis_dataframe["foot_pos"].apply(
    lambda p: p[0] if isinstance(p, tuple) and len(p) == 2 else np.nan
)
analysis_dataframe["foot_pos_y"] = analysis_dataframe["foot_pos"].apply(
    lambda p: p[1] if isinstance(p, tuple) and len(p) == 2 else np.nan
)

temp_df_kick_foot = analysis_dataframe[["foot_pos_x", "foot_pos_y"]].dropna()
if not temp_df_kick_foot.empty and len(temp_df_kick_foot) > 1:
    vx_foot = temp_df_kick_foot["foot_pos_x"].diff()
    vy_foot = temp_df_kick_foot["foot_pos_y"].diff()
    analysis_dataframe.loc[temp_df_kick_foot.index, "kicking_foot_velocity_x"] = vx_foot
    analysis_dataframe.loc[temp_df_kick_foot.index, "kicking_foot_velocity_y"] = vy_foot
    analysis_dataframe.loc[temp_df_kick_foot.index, "kicking_foot_speed"] = np.sqrt(
        vx_foot**2 + vy_foot**2
    )
    analysis_dataframe["kicking_foot_speed"] = pd.to_numeric(
        analysis_dataframe["kicking_foot_speed"], errors="coerce"
    )

    valid_speed_indices_for_accel = analysis_dataframe["kicking_foot_speed"].notna()
    if (
        valid_speed_indices_for_accel.any()
        and len(analysis_dataframe.loc[valid_speed_indices_for_accel]) > 1
    ):
        analysis_dataframe.loc[
            valid_speed_indices_for_accel, "kicking_foot_acceleration_scalar"
        ] = analysis_dataframe.loc[
            valid_speed_indices_for_accel, "kicking_foot_speed"
        ].diff()
else:
    analysis_dataframe["kicking_foot_velocity_x"] = np.nan
    analysis_dataframe["kicking_foot_velocity_y"] = np.nan
    analysis_dataframe["kicking_foot_speed"] = np.nan
    analysis_dataframe["kicking_foot_acceleration_scalar"] = np.nan

if USE_DIFFERENTIAL_FOOT_SPEED_CHECK:
    for foot_side in ["right", "left"]:
        pos_col = f"{foot_side}_foot_index_pos"
        speed_col = f"{foot_side}_foot_speed"
        x_col = f"{foot_side}_foot_index_x"
        y_col = f"{foot_side}_foot_index_y"

        analysis_dataframe[x_col] = analysis_dataframe[pos_col].apply(
            lambda p: p[0] if isinstance(p, tuple) and len(p) == 2 else np.nan
        )
        analysis_dataframe[y_col] = analysis_dataframe[pos_col].apply(
            lambda p: p[1] if isinstance(p, tuple) and len(p) == 2 else np.nan
        )

        temp_df_foot = analysis_dataframe[[x_col, y_col]].dropna()
        if not temp_df_foot.empty and len(temp_df_foot) > 1:
            vx = temp_df_foot[x_col].diff()
            vy = temp_df_foot[y_col].diff()
            analysis_dataframe.loc[temp_df_foot.index, speed_col] = np.sqrt(
                vx**2 + vy**2
            )
        else:
            analysis_dataframe[speed_col] = np.nan
        analysis_dataframe[speed_col] = pd.to_numeric(
            analysis_dataframe[speed_col], errors="coerce"
        )

# CALCULATE_SUPPORTING_FOOT_STABILITY 섹션을 다음과 같이 수정:

if CALCULATE_SUPPORTING_FOOT_STABILITY:

    def get_supporting_foot_ankle_pos(row):
        kicking_foot = row["kicking_foot"]
        if pd.isna(kicking_foot):
            return np.nan
        supporting_foot = "left" if kicking_foot == "right" else "right"
        return row[f"{supporting_foot}_ankle_pos"]

    analysis_dataframe["supporting_foot_ankle_pos"] = analysis_dataframe.apply(
        get_supporting_foot_ankle_pos, axis=1
    )

    analysis_dataframe["supp_foot_x"] = analysis_dataframe[
        "supporting_foot_ankle_pos"
    ].apply(lambda p: p[0] if isinstance(p, tuple) and len(p) == 2 else np.nan)
    analysis_dataframe["supp_foot_y"] = analysis_dataframe[
        "supporting_foot_ankle_pos"
    ].apply(lambda p: p[1] if isinstance(p, tuple) and len(p) == 2 else np.nan)

    # ✨ 강화된 스무딩 (기존 3프레임 → 9프레임)
    SMOOTHING_WINDOW_STABILITY = 9
    analysis_dataframe["supp_foot_x_smooth"] = (
        analysis_dataframe["supp_foot_x"]
        .rolling(window=SMOOTHING_WINDOW_STABILITY, center=True, min_periods=1)
        .mean()
    )
    analysis_dataframe["supp_foot_y_smooth"] = (
        analysis_dataframe["supp_foot_y"]
        .rolling(window=SMOOTHING_WINDOW_STABILITY, center=True, min_periods=1)
        .mean()
    )

    # ✨ 아웃라이어 제거 (변위가 50px 이상인 프레임 제거)
    temp_df_supp_foot = analysis_dataframe[
        ["supp_foot_x_smooth", "supp_foot_y_smooth"]
    ].dropna()
    if not temp_df_supp_foot.empty and len(temp_df_supp_foot) > 1:
        dx = temp_df_supp_foot["supp_foot_x_smooth"].diff()
        dy = temp_df_supp_foot["supp_foot_y_smooth"].diff()
        displacement = np.sqrt(dx**2 + dy**2)

        # 아웃라이어 제거 (50px 이상 변위는 노이즈로 간주)
        displacement = displacement.where(displacement < 50, np.nan)

        analysis_dataframe.loc[
            temp_df_supp_foot.index, "supporting_foot_displacement"
        ] = displacement
    else:
        analysis_dataframe["supporting_foot_displacement"] = np.nan

if USE_INTER_FOOT_DISTANCE_CHECK:

    def calc_dist_between_points(p1_series, p2_series):
        distances = []
        for p1, p2 in zip(p1_series, p2_series):
            if (
                isinstance(p1, tuple)
                and isinstance(p2, tuple)
                and len(p1) == 2
                and len(p2) == 2
            ):
                distances.append(np.hypot(p1[0] - p2[0], p1[1] - p2[1]))
            else:
                distances.append(np.nan)
        return pd.Series(distances, index=p1_series.index)

    analysis_dataframe["inter_foot_distance"] = calc_dist_between_points(
        analysis_dataframe["left_ankle_pos"], analysis_dataframe["right_ankle_pos"]
    )
    analysis_dataframe["inter_foot_distance"] = pd.to_numeric(
        analysis_dataframe["inter_foot_distance"], errors="coerce"
    ).fillna(float("inf"))

if USE_BALL_SHRINK_DETECTION:
    if "ball_radius" in analysis_dataframe.columns:
        analysis_dataframe["ball_radius_smoothed"] = (
            analysis_dataframe["ball_radius"]
            .rolling(window=BALL_SIZE_SMOOTHING_WINDOW, center=True, min_periods=1)
            .mean()
        )
        analysis_dataframe["ball_radius_change"] = analysis_dataframe[
            "ball_radius_smoothed"
        ].diff()
    else:
        analysis_dataframe["ball_radius_smoothed"] = np.nan
        analysis_dataframe["ball_radius_change"] = np.nan

cols_to_fillna_zero = [
    "foot_ball_approach_velocity",
    "foot_ball_approach_acceleration",
    "kicking_foot_velocity_x",
    "kicking_foot_velocity_y",
    "kicking_foot_speed",
    "kicking_foot_acceleration_scalar",
    "foot_pos_x",
    "foot_pos_y",
]
if USE_DIFFERENTIAL_FOOT_SPEED_CHECK:
    cols_to_fillna_zero.extend(["right_foot_speed", "left_foot_speed"])
if CALCULATE_SUPPORTING_FOOT_STABILITY:
    cols_to_fillna_zero.append("supporting_foot_displacement")
if USE_BALL_SHRINK_DETECTION:
    if "ball_radius_change" in analysis_dataframe.columns:
        cols_to_fillna_zero.append("ball_radius_change")
    if "ball_radius_smoothed" in analysis_dataframe.columns:
        analysis_dataframe["ball_radius_smoothed"] = (
            analysis_dataframe["ball_radius_smoothed"].ffill().bfill().fillna(10.0)
        )
if USE_BACKSWING_APPROACH_PATTERN:
    for col_bs in [
        "min_dist_right_foot_to_ball_smoothed",
        "min_dist_left_foot_to_ball_smoothed",
    ]:
        if col_bs in analysis_dataframe.columns:
            analysis_dataframe[col_bs] = (
                analysis_dataframe[col_bs].ffill().bfill().fillna(float("inf"))
            )
if (
    USE_INTER_FOOT_DISTANCE_CHECK
    and "inter_foot_distance" in analysis_dataframe.columns
):
    analysis_dataframe["inter_foot_distance"] = analysis_dataframe[
        "inter_foot_distance"
    ].fillna(float("inf"))

for col_fill_zero in cols_to_fillna_zero:
    if col_fill_zero in analysis_dataframe.columns:
        analysis_dataframe[col_fill_zero] = analysis_dataframe[col_fill_zero].fillna(
            0.0
        )


df_ball_kinematics = pd.DataFrame()
if (
    "ball_center" in analysis_dataframe.columns
    and not analysis_dataframe["ball_center"].isna().all()
):
    temp_ball_kin_df = analysis_dataframe[
        ["original_idx", "ball_center", "is_ball_predicted"]
    ].copy()
    valid_ball_center_rows = temp_ball_kin_df["ball_center"].notna()

    temp_ball_kin_df.loc[valid_ball_center_rows, "ball_x"] = temp_ball_kin_df.loc[
        valid_ball_center_rows, "ball_center"
    ].apply(lambda p: p[0] if isinstance(p, tuple) and len(p) == 2 else np.nan)
    temp_ball_kin_df.loc[valid_ball_center_rows, "ball_y"] = temp_ball_kin_df.loc[
        valid_ball_center_rows, "ball_center"
    ].apply(lambda p: p[1] if isinstance(p, tuple) and len(p) == 2 else np.nan)

    valid_indices_for_kinematics = temp_ball_kin_df[
        ~temp_ball_kin_df["is_ball_predicted"]
        & temp_ball_kin_df["ball_x"].notna()
        & temp_ball_kin_df["ball_y"].notna()
    ].index

    if len(valid_indices_for_kinematics) > 1:
        vx_ball_valid = temp_ball_kin_df.loc[
            valid_indices_for_kinematics, "ball_x"
        ].diff()
        vy_ball_valid = temp_ball_kin_df.loc[
            valid_indices_for_kinematics, "ball_y"
        ].diff()
        temp_ball_kin_df.loc[valid_indices_for_kinematics, "ball_velocity_x"] = (
            vx_ball_valid
        )
        temp_ball_kin_df.loc[valid_indices_for_kinematics, "ball_velocity_y"] = (
            vy_ball_valid
        )

        valid_velocities = (
            temp_ball_kin_df["ball_velocity_x"].notna()
            & temp_ball_kin_df["ball_velocity_y"].notna()
        )
        temp_ball_kin_df.loc[valid_velocities, "ball_speed"] = np.sqrt(
            temp_ball_kin_df.loc[valid_velocities, "ball_velocity_x"] ** 2
            + temp_ball_kin_df.loc[valid_velocities, "ball_velocity_y"] ** 2
        )

        valid_speed_for_accel_indices = (
            temp_ball_kin_df.loc[valid_indices_for_kinematics, "ball_speed"]
            .dropna()
            .index
        )
        if len(valid_speed_for_accel_indices) > 1:
            temp_ball_kin_df.loc[valid_speed_for_accel_indices, "ball_acceleration"] = (
                temp_ball_kin_df.loc[valid_speed_for_accel_indices, "ball_speed"].diff()
            )

    for col_kin in [
        "ball_velocity_x",
        "ball_velocity_y",
        "ball_speed",
        "ball_acceleration",
    ]:
        if col_kin not in temp_ball_kin_df.columns:
            temp_ball_kin_df[col_kin] = np.nan
    temp_ball_kin_df.fillna(
        {
            "ball_velocity_x": 0,
            "ball_velocity_y": 0,
            "ball_speed": 0,
            "ball_acceleration": 0,
        },
        inplace=True,
    )

    df_ball_kinematics = temp_ball_kin_df[
        ["original_idx", "ball_speed", "ball_acceleration", "is_ball_predicted"]
    ].copy()

# =====================================================================================
# ✨ [최종 v5] 무릎 각도에 가중치를 둔, 가장 정확한 백스윙 점수 계산 로직
# =====================================================================================
print("Calculating SEPARATE backswing scores for each foot (Knee Angle Weighted)...")

frame_height = all_processed_frames[0].shape[0] if all_processed_frames else 600

for side in ["left", "right"]:
    pos_col = f"{side}_foot_index_pos"
    y_col = f"{side}_foot_index_y"
    knee_angle_col = f"{side}_knee_angle"
    distance_col = f"min_dist_{side}_foot_to_ball_smoothed"
    score_temporal_col = f"{side}_backswing_score_temporal"

    if pos_col in analysis_dataframe.columns:
        analysis_dataframe[y_col] = analysis_dataframe[pos_col].apply(
            lambda p: p[1] if isinstance(p, tuple) else np.nan
        )
    else:
        analysis_dataframe[score_temporal_col] = 0
        print(
            f"🔴 Fatal Warning: Column '{pos_col}' not found. Cannot calculate backswing score for '{side}' foot."
        )
        continue

    if y_col in analysis_dataframe.columns:
        foot_pos_y_smoothed = (
            analysis_dataframe[y_col]
            .rolling(window=actual_smoothing_win_size, center=True, min_periods=1)
            .mean()
            .fillna(method="bfill")
            .fillna(method="ffill")
        )

        height_normalized = ((frame_height - foot_pos_y_smoothed) / frame_height).clip(
            0, 1
        )

        max_distance = (
            analysis_dataframe[distance_col].replace([np.inf, -np.inf], np.nan).max()
        )
        distance_normalized = (
            (analysis_dataframe[distance_col] / max_distance).clip(0, 1)
            if pd.notna(max_distance) and max_distance > 0
            else 0
        )

        retreat_velocity = -analysis_dataframe[distance_col].diff().fillna(0)
        lift_velocity = -foot_pos_y_smoothed.diff().fillna(0)
        max_retreat_vel = retreat_velocity.max()
        max_lift_vel = lift_velocity.max()

        retreat_velocity_normalized = (
            np.maximum(retreat_velocity / max_retreat_vel, 0).clip(0, 1)
            if pd.notna(max_retreat_vel) and max_retreat_vel > 0
            else 0
        )
        lift_velocity_normalized = (
            np.maximum(lift_velocity / max_lift_vel, 0).clip(0, 1)
            if pd.notna(max_lift_vel) and max_lift_vel > 0
            else 0
        )

        knee_angle_normalized = (
            (180 - analysis_dataframe[knee_angle_col].fillna(180)) / 180
        ).clip(0, 1)

        # ✨✨✨ 가중치 수정: 무릎 각도의 중요도를 대폭 높임 ✨✨✨
        analysis_dataframe[f"{side}_backswing_score"] = (
            distance_normalized * 0.10  # 거리 10%
            + height_normalized * 0.10  # 높이 10%
            + retreat_velocity_normalized * 0.05  # 후퇴 속도 5%
            + lift_velocity_normalized * 0.05  # 상승 속도 5%
            + knee_angle_normalized * 0.70  # ✨ 무릎 각도 70%
        ) * 100000

        analysis_dataframe[score_temporal_col] = (
            analysis_dataframe[f"{side}_backswing_score"]
            .rolling(window=5, center=True, min_periods=1)
            .mean()
        )
        print(
            f"✅ Calculated independent backswing score for '{side}' foot with new weights."
        )
    else:
        analysis_dataframe[score_temporal_col] = 0
        print(f"🔴 Warning: Missing data for '{side}' foot backswing calculation.")
# =====================================================================================

# --- ✨ 정교한 백스윙 점수 계산 끝 ✨ ---
# ✨ 생성된 analysis_dataframe을 전역 변수에 할당 ✨
analysis_dataframe_global = analysis_dataframe.copy()


# --- 임팩트 분석 ---
impact_candidate_list = []
final_impact_idx = -1
# final_impact_info = None
final_interpolated_details = None

if analysis_dataframe_global.empty:  # ✨ 전역 변수 사용
    print("Error: Analysis DataFrame is empty. Cannot proceed with impact analysis.")
else:
    right_backswing_peak_df_idx, left_backswing_peak_df_idx = -1, -1

    print("\n--- DETAILED IMPACT CANDIDATE CONDITION CHECK ---")
    for (
        df_row_idx,
        df_current_row,
    ) in analysis_dataframe_global.iterrows():  # ✨ 전역 변수 사용 ✨
        # --- 임팩트 분석 ---

        # <<< 바로 여기에 전체 코드를 추가 또는 교체합니다.
        # 💡 해결 방안 1: 예측된 공 정보는 임팩트 후보에서 제외
        if df_current_row.get("is_ball_predicted", False):
            continue  # 현재 프레임이 예측된 정보라면, 임팩트 후보로 고려하지 않고 건너뜀

        current_foot_pos = df_current_row.get("foot_pos")
        if not (isinstance(current_foot_pos, tuple) and len(current_foot_pos) == 2):
            continue
        # <<< 여기까지 추가 또는 교체

        raw_distance = df_current_row.get("distance", float("inf"))
        ball_rad_check = df_current_row.get("ball_radius", 10.0)
        ball_rad_check = (
            ball_rad_check if pd.notna(ball_rad_check) and ball_rad_check > 0 else 10.0
        )
        current_ball_box = df_current_row.get("ball_box")

        prev_approach_vel_val = (
            analysis_dataframe_global.loc[df_row_idx - 1, "foot_ball_approach_velocity"]
            if df_row_idx > 0
            and (df_row_idx - 1) in analysis_dataframe_global.index
            and "foot_ball_approach_velocity" in analysis_dataframe_global.columns
            and pd.notna(
                analysis_dataframe_global.loc[
                    df_row_idx - 1, "foot_ball_approach_velocity"
                ]
            )
            else 0.0
        )

        current_approach_vel_val = df_current_row.get(
            "foot_ball_approach_velocity", 0.0
        )
        current_approach_vel_val = (
            current_approach_vel_val if pd.notna(current_approach_vel_val) else 0.0
        )

        current_kicking_foot_speed_val = df_current_row.get("kicking_foot_speed", 0.0)
        current_kicking_foot_speed_val = (
            current_kicking_foot_speed_val
            if pd.notna(current_kicking_foot_speed_val)
            else 0.0
        )

        is_window_minimum_dist_smooth = is_distance_minimum_in_window(
            analysis_dataframe_global,
            df_row_idx,
            MINIMUM_DISTANCE_WINDOW,
            "distance_smoothed",
        )
        is_very_close_raw = (
            raw_distance < ABSOLUTE_DISTANCE_THRESHOLD
            if pd.notna(raw_distance)
            else False
        )

        is_very_close_proximity = (
            raw_distance < ball_rad_check * BETA_PROXIMITY_SCALE
            if pd.notna(raw_distance) and pd.notna(ball_rad_check)
            else False
        )

        iou_value = 0.0
        if isinstance(current_foot_pos, tuple) and len(current_foot_pos) == 2:
            foot_center_x, foot_center_y = current_foot_pos
            foot_bounding_box = [
                foot_center_x - FOOT_BOX_RADIUS,
                foot_center_y - FOOT_BOX_RADIUS,
                foot_center_x + FOOT_BOX_RADIUS,
                foot_center_y + FOOT_BOX_RADIUS,
            ]
            if (
                current_ball_box
                and isinstance(current_ball_box, list)
                and len(current_ball_box) == 4
            ):
                iou_value = calculate_iou(foot_bounding_box, current_ball_box)
        is_significant_iou = iou_value > IOU_CONTACT_THRESHOLD
        # --- DETAILED IMPACT CANDIDATE CONDITION CHECK ---
        print("\\n--- DETAILED IMPACT CANDIDATE CONDITION CHECK ---")
        for df_row_idx, df_current_row in analysis_dataframe_global.iterrows():
            # ... (기존 코드)

            is_significant_iou = iou_value > IOU_CONTACT_THRESHOLD

            # 💡 디버깅 코드 추가
            if df_row_idx > 20 and df_row_idx < 50:  # 특정 프레임 구간만 확인
                print(
                    f"F:{df_row_idx} | "
                    f"Dist:{raw_distance:.1f} (Thresh:{ABSOLUTE_DISTANCE_THRESHOLD}) | "
                    f"IoU:{iou_value:.2f} (Thresh:{IOU_CONTACT_THRESHOLD}) | "
                    f"CloseRaw:{is_very_close_raw} | "
                    f"CloseProx:{is_very_close_proximity} | "
                    f"SigIoU:{is_significant_iou}"
                )

        base_impact_conditions = (
            (is_window_minimum_dist_smooth or is_very_close_raw)
            and is_very_close_proximity
            and is_significant_iou
        )

        is_current_frame_av_impact = False
        if pd.notna(prev_approach_vel_val) and pd.notna(current_approach_vel_val):
            if (
                prev_approach_vel_val < -KICK_FOOT_APPROACH_VEL_THRESHOLD * 0.5
                and current_approach_vel_val >= -KICK_FOOT_APPROACH_VEL_THRESHOLD * 0.1
            ):
                is_current_frame_av_impact = True

        is_differential_speed_ok = True
        if USE_DIFFERENTIAL_FOOT_SPEED_CHECK:
            is_differential_speed_ok = False
            actual_kicking_foot_for_diff = df_current_row.get("kicking_foot")
            kick_foot_speed_diff_val, support_foot_speed_diff_val = np.nan, np.nan

            if actual_kicking_foot_for_diff == "right":
                kick_foot_speed_diff_val = df_current_row.get("right_foot_speed", 0.0)
                support_foot_speed_diff_val = df_current_row.get("left_foot_speed", 0.0)
            elif actual_kicking_foot_for_diff == "left":
                kick_foot_speed_diff_val = df_current_row.get("left_foot_speed", 0.0)
                support_foot_speed_diff_val = df_current_row.get(
                    "right_foot_speed", 0.0
                )

            kick_foot_speed_diff_val = (
                kick_foot_speed_diff_val if pd.notna(kick_foot_speed_diff_val) else 0.0
            )
            support_foot_speed_diff_val = (
                support_foot_speed_diff_val
                if pd.notna(support_foot_speed_diff_val)
                else 0.0
            )

            if (
                pd.notna(actual_kicking_foot_for_diff)
                and actual_kicking_foot_for_diff != "N/A"
            ):
                if (
                    kick_foot_speed_diff_val
                    > support_foot_speed_diff_val
                    * KICK_TO_SUPPORT_SPEED_RATIO_THRESHOLD
                ):
                    is_differential_speed_ok = True
                elif support_foot_speed_diff_val < (
                    MIN_KICKING_FOOT_SPEED_AT_IMPACT * 0.1 / pixel_to_cm_scale_global
                    if pixel_to_cm_scale_global > 0
                    else 1.0
                ) and kick_foot_speed_diff_val >= (
                    MIN_KICKING_FOOT_SPEED_AT_IMPACT * 0.5 / pixel_to_cm_scale_global
                    if pixel_to_cm_scale_global > 0
                    else 2.0
                ):
                    is_differential_speed_ok = True

        is_feet_relatively_close = True
        if USE_INTER_FOOT_DISTANCE_CHECK:
            is_feet_relatively_close = False
            inter_foot_distance_val = df_current_row.get(
                "inter_foot_distance", float("inf")
            )
            if (
                pd.notna(inter_foot_distance_val)
                and inter_foot_distance_val < MAX_INTER_FOOT_DISTANCE_AT_IMPACT
            ):
                is_feet_relatively_close = True

        dynamic_conditions_list = [is_differential_speed_ok, is_feet_relatively_close]

        all_impact_conditions_met = base_impact_conditions and all(
            dynamic_conditions_list
        )

        if all_impact_conditions_met:
            candidate_score_val = (
                (raw_distance if pd.notna(raw_distance) else 9999)
                - iou_value * 10
                + current_kicking_foot_speed_val * FOOT_SPEED_SCORE_WEIGHT
            )
            if is_current_frame_av_impact:
                candidate_score_val += IMPACT_AV_CHANGE_BONUS

            impact_candidate_list.append(
                {
                    "data_row": df_current_row,
                    "score": candidate_score_val,
                    "iou": iou_value,
                    "df_idx": df_row_idx,
                    "is_current_frame_av_impact": is_current_frame_av_impact,
                }
            )

    if impact_candidate_list:
        if "kicking_foot_speed" in analysis_dataframe_global.columns:
            for cand_item in impact_candidate_list:
                idx_cand = cand_item["df_idx"]
                is_peak_cand = False
                current_speed_cand = analysis_dataframe_global.loc[
                    idx_cand, "kicking_foot_speed"
                ]
                if pd.isna(current_speed_cand):
                    cand_item["is_peak"] = False
                    continue

                start_check_cand = max(0, idx_cand - FOOT_SPEED_PEAK_WINDOW)
                end_check_cand = min(
                    len(analysis_dataframe_global) - 1,
                    idx_cand + FOOT_SPEED_PEAK_WINDOW,
                )
                window_speeds_cand = analysis_dataframe_global.loc[
                    start_check_cand:end_check_cand, "kicking_foot_speed"
                ].dropna()

                if (
                    not window_speeds_cand.empty
                    and current_speed_cand >= window_speeds_cand.max()
                ):
                    is_peak_cand = True

                if is_peak_cand:
                    cand_item["score"] += FOOT_SPEED_PEAK_BONUS
                    cand_item["is_peak"] = True
                else:
                    cand_item["is_peak"] = False

        impact_candidate_list.sort(key=lambda x_cand: x_cand["score"])

        # =====================================================================================
        # ✨ [최종 v6] '최소 무릎 각도'를 최종 기준으로 사용하는 가장 완벽한 로직
        # =====================================================================================
        if USE_BACKSWING_APPROACH_PATTERN:
            print(
                "\n--- Enhanced Backswing Detection (Knee Angle Priority Logic v2) ---"
            )

            if impact_candidate_list:
                impact_df_idx = impact_candidate_list[0]["df_idx"]
                search_window_size = 50
                search_start_idx = max(0, impact_df_idx - search_window_size)

                search_df = analysis_dataframe_global.loc[
                    search_start_idx:impact_df_idx
                ].copy()
                print(
                    f"Backswing search window: frame {search_start_idx} to {impact_df_idx}"
                )

                KNEE_ANGLE_THRESHOLD = 110

                # --- 왼쪽 발 백스윙 찾기 ---
                if (
                    "left_knee_angle" in search_df.columns
                    and search_df["left_knee_angle"].notna().any()
                ):
                    left_candidates = search_df[
                        search_df["left_knee_angle"] < KNEE_ANGLE_THRESHOLD
                    ]
                    if not left_candidates.empty:
                        # ✨✨✨ 변경점: 점수(idxmax) 대신 무릎 각도(idxmin)를 기준으로 선택 ✨✨✨
                        best_left_idx = left_candidates["left_knee_angle"].idxmin()
                        left_backswing_peak_df_idx = best_left_idx
                        print(
                            f"✅ Left foot backswing peak candidate found at index: {best_left_idx} (based on min knee angle)"
                        )

                # --- 오른쪽 발 백스윙 찾기 ---
                if (
                    "right_knee_angle" in search_df.columns
                    and search_df["right_knee_angle"].notna().any()
                ):
                    right_candidates = search_df[
                        search_df["right_knee_angle"] < KNEE_ANGLE_THRESHOLD
                    ]
                    if not right_candidates.empty:
                        # ✨✨✨ 변경점: 점수(idxmax) 대신 무릎 각도(idxmin)를 기준으로 선택 ✨✨✨
                        best_right_idx = right_candidates["right_knee_angle"].idxmin()
                        right_backswing_peak_df_idx = best_right_idx
                        print(
                            f"✅ Right foot backswing peak candidate found at index: {best_right_idx} (based on min knee angle)"
                        )
            else:
                print(
                    "🟡 Warning: No impact candidates found. Skipping backswing detection."
                )

        print(
            f"DEBUG: Final backswing candidates - Right: {right_backswing_peak_df_idx}, Left: {left_backswing_peak_df_idx}"
        )
        # =====================================================================================

    print("impact_candidate_list: ", impact_candidate_list)
    # 기존의 if impact_candidate_list: 부터 시작하는 블록 전체를
    # 아래 코드로 교체해주세요.

    if impact_candidate_list:
        # --- [섹션 A] : 기존 코드 (최초 임팩트 후보 선정) ---
        best_candidate_final = impact_candidate_list[0]
        final_impact_info = best_candidate_final["data_row"].to_dict()
        final_impact_idx = int(final_impact_info["original_idx"])
        best_candidate_df_idx_val = best_candidate_final.get("df_idx")

        # --- [섹션 B] : ★★★ 여기가 핵심 수정 및 디버깅 코드 추가 부분 ★★★ ---
        # 1. 확정된 임팩트 정보에서 '차는 발'이 어느 쪽인지 가져옵니다.
        k_foot_impact = final_impact_info.get("kicking_foot")

        # 3. 차는 발에 맞는 백스윙 인덱스를 선택하여 변수에 할당합니다.
        backswing_peak_idx_used_df = -1
        if k_foot_impact == "right" and right_backswing_peak_df_idx != -1:
            backswing_peak_idx_used_df = right_backswing_peak_df_idx

        elif k_foot_impact == "left" and left_backswing_peak_df_idx != -1:
            backswing_peak_idx_used_df = left_backswing_peak_df_idx

        if backswing_peak_idx_used_df != -1:
            final_impact_info["backswing_original_idx"] = backswing_peak_idx_used_df

        # --- [섹션 C] : 기존에 가지고 계신 디버깅 코드 ---
        # 이제 backswing_peak_idx_used_df 변수가 제대로 된 값을 가지므로, 이 블록이 정상 작동할 것입니다.
        if backswing_peak_idx_used_df != -1:
            try:
                backswing_peak_row = analysis_dataframe_global.loc[
                    backswing_peak_idx_used_df
                ]
                original_frame_index = backswing_peak_row.get("original_idx", "N/A")
                backswing_score = backswing_peak_row.get(
                    "backswing_score_temporal", "N/A"
                )
                # k_foot_impact 변수는 [섹션 B]에서 이미 정의되었으므로 그대로 사용 가능합니다.
                kicking_knee_angle = backswing_peak_row.get(
                    f"{k_foot_impact}_knee_angle", "N/A"
                )
                foot_ball_distance = backswing_peak_row.get(
                    f"min_dist_{k_foot_impact}_foot_to_ball_smoothed", "N/A"
                )

                print("\n✅ [DEBUG] 백스윙 정점 프레임의 상세 정보:")
                print(f"  - 원본 샘플링 프레임 번호: {original_frame_index}")
                if isinstance(backswing_score, (int, float)):
                    print(f"  - 백스윙 종합 점수: {backswing_score:.2f}")
                if isinstance(kicking_knee_angle, (int, float)):
                    print(f"  - 차는 발 무릎 각도: {kicking_knee_angle:.2f}도")
                if isinstance(foot_ball_distance, (int, float)):
                    print(f"  - 발-공 거리 (Smoothed): {foot_ball_distance:.2f}px")

            except KeyError as e:
                print(f"🔴 [DEBUG] 에러: 백스윙 정보를 가져오는 중 키 에러 발생 - {e}")
            except Exception as e:
                print(f"🔴 [DEBUG] 에러: 백스윙 정보를 가져오는 중 예외 발생 - {e}")
        else:
            # 이 경고 메시지는 이제 나타나지 않아야 합니다.
            print("🟡 [DEBUG] 경고: 최종 백스윙 정점을 찾지 못했습니다 (인덱스: -1).")

        print("[DEBUG] -------------------------------------------------\n")

        # --- 이후 로직은 그대로 이어집니다 ---

        k_foot_impact = final_impact_info.get("kicking_foot")
        # backswing_peak_idx_used_df = globals().get("backswing_peak_idx_used_df", -1)
        if (
            USE_BACKSWING_APPROACH_PATTERN
            and backswing_peak_idx_used_df != -1
            and 0 <= backswing_peak_idx_used_df < len(analysis_dataframe_global)
        ):  # ✨ 전역 변수 사용 ✨
            dist_col_bs = f"min_dist_{k_foot_impact}_foot_to_ball_smoothed"
            if dist_col_bs in analysis_dataframe_global.columns:
                final_impact_info["backswing_peak_distance"] = (
                    analysis_dataframe_global.loc[
                        backswing_peak_idx_used_df, dist_col_bs
                    ]
                )

            knee_angle_col_bs = f"{k_foot_impact}_knee_angle"
            if knee_angle_col_bs in analysis_dataframe_global.columns:
                final_impact_info["backswing_kicking_knee_angle"] = (
                    analysis_dataframe_global.loc[
                        backswing_peak_idx_used_df, knee_angle_col_bs
                    ]
                )
            else:
                final_impact_info["backswing_kicking_knee_angle"] = np.nan
        else:
            final_impact_info["backswing_peak_distance"] = np.nan
            final_impact_info["backswing_kicking_knee_angle"] = np.nan

        if CALCULATE_SUPPORTING_FOOT_STABILITY:

            def get_supporting_foot_ankle_pos(row):
                kicking_foot = row["kicking_foot"]
                if pd.isna(kicking_foot):
                    return np.nan
                supporting_foot = "left" if kicking_foot == "right" else "right"
                return row.get(f"{supporting_foot}_ankle_pos", np.nan)

            analysis_dataframe["supporting_foot_ankle_pos"] = analysis_dataframe.apply(
                get_supporting_foot_ankle_pos, axis=1
            )

            analysis_dataframe["supp_foot_x"] = analysis_dataframe[
                "supporting_foot_ankle_pos"
            ].apply(lambda p: p[0] if isinstance(p, tuple) and len(p) == 2 else np.nan)
            analysis_dataframe["supp_foot_y"] = analysis_dataframe[
                "supporting_foot_ankle_pos"
            ].apply(lambda p: p[1] if isinstance(p, tuple) and len(p) == 2 else np.nan)

            # 🟡 분석 구간 필터링: backswing ~ impact 구간만
            backswing_idx = final_impact_info.get("backswing_original_idx", -1)
            impact_idx = final_impact_info.get("original_idx", -1)

            if backswing_idx != -1 and impact_idx != -1:
                # 백스윙~임팩트 프레임 구간만 추출
                mask_stability = (
                    analysis_dataframe["original_idx"] >= backswing_idx
                ) & (analysis_dataframe["original_idx"] <= impact_idx)
                temp_df_supp_foot = analysis_dataframe.loc[
                    mask_stability, ["supp_foot_x", "supp_foot_y"]
                ].dropna()

                SMOOTHING_WINDOW_STABILITY = 9
                supp_foot_x_smooth = (
                    temp_df_supp_foot["supp_foot_x"]
                    .rolling(
                        window=SMOOTHING_WINDOW_STABILITY, center=True, min_periods=1
                    )
                    .mean()
                )
                supp_foot_y_smooth = (
                    temp_df_supp_foot["supp_foot_y"]
                    .rolling(
                        window=SMOOTHING_WINDOW_STABILITY, center=True, min_periods=1
                    )
                    .mean()
                )

                dx = supp_foot_x_smooth.diff()
                dy = supp_foot_y_smooth.diff()
                displacement = np.sqrt(dx**2 + dy**2)

                # 아웃라이어 제거 (50px 이상은 무시)
                displacement = displacement.where(displacement < 50, np.nan)

                # 결과 반영
                analysis_dataframe.loc[
                    temp_df_supp_foot.index, "supporting_foot_displacement"
                ] = displacement

                # 스무딩된 좌표도 저장 (옵션)
                analysis_dataframe.loc[
                    temp_df_supp_foot.index, "supp_foot_x_smooth"
                ] = supp_foot_x_smooth
                analysis_dataframe.loc[
                    temp_df_supp_foot.index, "supp_foot_y_smooth"
                ] = supp_foot_y_smooth

            else:
                analysis_dataframe["supporting_foot_displacement"] = np.nan
                analysis_dataframe["supp_foot_x_smooth"] = np.nan
                analysis_dataframe["supp_foot_y_smooth"] = np.nan
        if (
            USE_FRAME_INTERPOLATION
            and final_impact_idx != -1
            and best_candidate_df_idx_val is not None
            and best_candidate_df_idx_val in analysis_dataframe_global.index
        ):
            try:
                s_interp_idx = max(
                    0, best_candidate_df_idx_val - INTERPOLATION_WINDOW_RADIUS
                )
                e_interp_idx = min(
                    len(analysis_dataframe_global) - 1,
                    best_candidate_df_idx_val + INTERPOLATION_WINDOW_RADIUS,
                )
                win_df_interp = analysis_dataframe_global.loc[s_interp_idx:e_interp_idx]

                if (
                    len(win_df_interp) >= 2
                    and "distance_smoothed" in win_df_interp.columns
                    and "original_idx" in win_df_interp.columns
                ):
                    win_df_interp_cleaned = win_df_interp[
                        ["original_idx", "distance_smoothed"]
                    ].dropna()
                    win_df_interp_cleaned = win_df_interp_cleaned[
                        np.isfinite(win_df_interp_cleaned["distance_smoothed"])
                    ]

                    if len(win_df_interp_cleaned) >= 2:
                        x_interp = win_df_interp_cleaned["original_idx"].values
                        y_interp = win_df_interp_cleaned["distance_smoothed"].values
                        ux_interp, u_idx_interp = np.unique(x_interp, return_index=True)
                        if len(ux_interp) >= 2:
                            x_interp_u, y_interp_u = ux_interp, y_interp[u_idx_interp]
                            kind_interp = (
                                "cubic"
                                if len(x_interp_u) >= 4
                                else ("quadratic" if len(x_interp_u) == 3 else "linear")
                            )
                            interp_func = interp1d(
                                x_interp_u,
                                y_interp_u,
                                kind=kind_interp,
                                bounds_error=False,
                                fill_value="extrapolate",
                            )

                            fine_x_s_val = 1.0 / max(1, INTERPOLATION_DENSITY_FACTOR)
                            fx_interp_val = np.arange(
                                x_interp_u.min(),
                                x_interp_u.max() + fine_x_s_val,
                                fine_x_s_val,
                            )

                            if len(fx_interp_val) > 0:
                                iy_interp_val = interp_func(fx_interp_val)
                                min_iy_interp_idx_val = np.argmin(iy_interp_val)
                                min_iy_interp_val, idx_r_interp_val = (
                                    iy_interp_val[min_iy_interp_idx_val],
                                    fx_interp_val[min_iy_interp_idx_val],
                                )
                                final_interpolated_details = {
                                    "refined_original_idx": idx_r_interp_val,
                                    "min_interpolated_distance": min_iy_interp_val,
                                }
                                print(
                                    f"  Interpolation successful: Refined Idx={idx_r_interp_val:.2f}, Min. Dist={min_iy_interp_val:.2f}px"
                                )
            except Exception as e_int:
                print(f"  Interpolation error: {e_int}")
                traceback.print_exc(limit=1)

    elif not analysis_dataframe_global.empty:
        print(
            "Warning: No impact candidates found by primary logic. Fallback: Selecting frame with minimum raw 'distance'."
        )
        fallback_df = analysis_dataframe_global.dropna(subset=["distance"])
        if not fallback_df.empty:
            best_fallback_idx_in_df = fallback_df["distance"].idxmin()
            if best_fallback_idx_in_df in analysis_dataframe_global.index:
                final_impact_info = analysis_dataframe_global.loc[
                    best_fallback_idx_in_df
                ].to_dict()
                final_impact_idx = int(final_impact_info["original_idx"])
                print(
                    f"Fallback successful: Selected frame original_idx={final_impact_idx} (df_idx={best_fallback_idx_in_df}) based on minimum raw distance."
                )
                final_impact_info["is_ball_info_corrected"] = False
                fp_contact_fb, bc_contact_fb = final_impact_info.get(
                    "foot_pos"
                ), final_impact_info.get("ball_center")
                if (
                    fp_contact_fb
                    and bc_contact_fb
                    and isinstance(fp_contact_fb, tuple)
                    and isinstance(bc_contact_fb, tuple)
                ):
                    final_impact_info["contact_region_on_ball"] = (
                        get_ball_contact_region(
                            bc_contact_fb[0] - fp_contact_fb[0],
                            bc_contact_fb[1] - fp_contact_fb[1],
                        )
                    )
                else:
                    final_impact_info["contact_region_on_ball"] = "N/A"

                sfa_pos_fb = final_impact_info.get("supporting_foot_ankle_pos")
                if (
                    sfa_pos_fb
                    and bc_contact_fb
                    and isinstance(sfa_pos_fb, tuple)
                    and isinstance(bc_contact_fb, tuple)
                ):
                    final_impact_info["dist_ball_to_supporting_foot_ankle"] = np.hypot(
                        sfa_pos_fb[0] - bc_contact_fb[0],
                        sfa_pos_fb[1] - bc_contact_fb[1],
                    )
                else:
                    final_impact_info["dist_ball_to_supporting_foot_ankle"] = np.nan

                if not df_ball_kinematics.empty:
                    sr_at_impact_fb = df_ball_kinematics[
                        (df_ball_kinematics["original_idx"] == final_impact_idx)
                        & (~df_ball_kinematics["is_ball_predicted"])
                    ]
                    final_impact_info["ball_speed_at_impact"] = (
                        sr_at_impact_fb["ball_speed"].iloc[0]
                        if not sr_at_impact_fb.empty
                        else np.nan
                    )
                    final_impact_info["max_ball_speed_px_fr"] = final_impact_info[
                        "ball_speed_at_impact"
                    ]
                    final_impact_info["ball_speed_after_impact"] = final_impact_info[
                        "max_ball_speed_px_fr"
                    ]
                else:
                    final_impact_info["ball_speed_at_impact"] = np.nan
                    final_impact_info["max_ball_speed_px_fr"] = np.nan
                    final_impact_info["ball_speed_after_impact"] = np.nan

                final_impact_info.setdefault("backswing_peak_distance", np.nan)
                final_impact_info.setdefault("backswing_kicking_knee_angle", np.nan)
                final_impact_info.setdefault("supporting_foot_stability", np.nan)
                final_impact_info.setdefault("kicking_foot_acceleration_scalar", np.nan)
                final_impact_info.setdefault(
                    "calculated_max_foot_swing_speed_px_fr",
                    final_impact_info.get("kicking_foot_speed", 0),
                )
                final_impact_info.setdefault(
                    "backswing_original_idx", -1
                )  # Fallback 시 백스윙 인덱스 없음

    if final_impact_idx == -1:
        print("Critical Error: Could not determine impact moment even with fallback.")
        final_impact_info = {}

print(
    f"\nImpact analysis logic completed ({time.time()-impact_analysis_start_time:.2f}s)"
)

if final_impact_info is None:
    final_impact_info = {}

max_foot_speed_for_score_px_fr = 0
if (
    final_impact_idx != -1
    and isinstance(analysis_dataframe_global, pd.DataFrame)
    and not analysis_dataframe_global.empty
):
    # ... (코드 후반부) ...
    # 1. final_impact_info에서 백스윙 정점의 인덱스를 가져옵니다.
    #    만약 찾지 못했다면 안전하게 0부터 시작합니다.
    backswing_df_idx = final_impact_info.get("backswing_original_idx", 0)
    if backswing_df_idx == -1:  # -1로 저장된 경우를 대비
        backswing_df_idx = 0

    # 2. 검색 시작 지점을 '0'이 아닌, '백스윙 정점 인덱스'로 변경합니다.
    search_start_idx_df = backswing_df_idx

    # 'search_end_idx_df'는 기존과 동일하게 임팩트 프레임의 인덱스입니다.
    search_end_idx_df = (
        analysis_dataframe_global[
            analysis_dataframe_global["original_idx"] == final_impact_idx
        ].index[0]
        # ...
    )

    # 이제 loc[...]는 '백스윙 정점'부터 '임팩트'까지의 데이터만 가져옵니다.
    foot_speeds_for_score = analysis_dataframe_global.loc[
        search_start_idx_df:search_end_idx_df, "kicking_foot_speed"
    ].dropna()

    if not foot_speeds_for_score.empty:
        max_foot_speed_for_score_px_fr = foot_speeds_for_score.max()
# ...
elif final_impact_idx != -1 and final_impact_info:
    max_foot_speed_for_score_px_fr = final_impact_info.get("kicking_foot_speed", 0)
    if not pd.notna(max_foot_speed_for_score_px_fr):
        max_foot_speed_for_score_px_fr = 0

final_impact_info["calculated_max_foot_swing_speed_px_fr"] = (
    max_foot_speed_for_score_px_fr
)

if final_impact_idx != -1 and final_impact_info is not None and final_impact_info:
    if (
        "contact_region_on_ball" not in final_impact_info
        or pd.isna(final_impact_info.get("contact_region_on_ball"))
        or final_impact_info.get("contact_region_on_ball") == "N/A"
    ):
        fp_contact_final, bc_contact_final = final_impact_info.get(
            "foot_pos"
        ), final_impact_info.get("ball_center")
        if (
            fp_contact_final
            and bc_contact_final
            and isinstance(fp_contact_final, tuple)
            and isinstance(bc_contact_final, tuple)
            and len(fp_contact_final) == 2
            and len(bc_contact_final) == 2
        ):
            final_impact_info["contact_region_on_ball"] = get_ball_contact_region(
                bc_contact_final[0] - fp_contact_final[0],
                bc_contact_final[1] - fp_contact_final[1],
            )
        else:
            final_impact_info["contact_region_on_ball"] = "N/A"
    # 백스윙 시점의 공 반지름을 저장할 변수 초기화
    backswing_ball_radius = np.nan

    # 백스윙 정점의 인덱스를 가져옵니다.
    backswing_df_idx = final_impact_info.get("backswing_original_idx", -1)

    if backswing_df_idx != -1:
        # 데이터프레임에서 백스윙 정점 프레임의 정보를 가져옵니다.
        backswing_row = analysis_dataframe_global[
            analysis_dataframe_global["original_idx"] == backswing_df_idx
        ]
        if not backswing_row.empty:
            # 해당 프레임의 공 반지름 값을 가져옵니다.
            backswing_ball_radius = backswing_row.iloc[0].get("ball_radius")

    # final_impact_info에 백스윙 시점의 공 반지름 저장
    final_impact_info["backswing_ball_radius"] = backswing_ball_radius
    print(
        f"[DEBUG] Backswing radius stored in final_impact_info: {final_impact_info.get('backswing_ball_radius')}"
    )
    # ... (이후 print_kick_analysis_summary 등 호출) ...
elif final_impact_idx == -1:
    print(
        "No impact detected, cannot generate detailed final summary or video annotations related to impact."
    )
    if final_impact_info is None:
        final_impact_info = {}
    final_impact_info["contact_region_on_ball"] = "N/A"

# ... (이후 print_kick_analysis_summary 등 호출) ...
sampling_rate_for_summary = (
    sampling_interval_calc
    if "sampling_interval_calc" in locals() and sampling_interval_calc > 0
    else 1
)

import matplotlib.pyplot as plt

plt.figure(figsize=(15, 6))

# 양쪽 발의 백스윙 점수 그래프 그리기
if "left_backswing_score_temporal" in analysis_dataframe_global.columns:
    plt.plot(
        analysis_dataframe_global["original_idx"],
        analysis_dataframe_global["left_backswing_score_temporal"],
        marker=".",
        linestyle="--",
        color="cyan",
        label="Left Backswing Score",
    )

if "right_backswing_score_temporal" in analysis_dataframe_global.columns:
    plt.plot(
        analysis_dataframe_global["original_idx"],
        analysis_dataframe_global["right_backswing_score_temporal"],
        marker=".",
        linestyle="--",
        color="magenta",
        label="Right Backswing Score",
    )

# 최종 결정된 백스윙 정점 표시
if "backswing_peak_idx_used_df" in locals() and backswing_peak_idx_used_df != -1:
    peak_x = analysis_dataframe_global.loc[backswing_peak_idx_used_df, "original_idx"]

    # 최종 결정된 발에 맞는 점수 컬럼에서 y값 찾기
    final_kicking_foot = (
        final_impact_info.get("kicking_foot")
        if "final_impact_info" in locals()
        else None
    )
    if final_kicking_foot:
        peak_y_col = f"{final_kicking_foot}_backswing_score_temporal"
        if peak_y_col in analysis_dataframe_global.columns:
            peak_y = analysis_dataframe_global.loc[
                backswing_peak_idx_used_df, peak_y_col
            ]
            plt.scatter(
                peak_x,
                peak_y,
                color="red",
                s=150,
                zorder=5,
                label=f"Detected Peak (Foot: {final_kicking_foot}, Idx: {backswing_peak_idx_used_df})",
            )

plt.title("Backswing Score Over Time (Left vs Right)")
plt.xlabel("Frame Index")
plt.ylabel("Calculated Backswing Score")
plt.legend()
plt.grid(True)
plt.show()


#  보간 전 ❗️디버그용❗️ 요약 출력
if DEBUG_MODE:
    print_kick_analysis_summary(
        final_impact_idx,
        final_impact_info,
        None,  # 아직 보간 전이므로 None
        sampling_rate_for_summary,
        original_video_fps,
        pixel_to_cm_scale_global,
    )

# (✨ 수정 후 새로운 코드)

# 보간 처리
final_interpolated_details = None
if USE_FRAME_INTERPOLATION and final_impact_idx != -1:
    final_interpolated_details = find_refined_impact_with_scipy(
        analysis_dataframe_global,
        final_impact_idx,
        window_radius=INTERPOLATION_WINDOW_RADIUS,
        density_factor=INTERPOLATION_DENSITY_FACTOR,
    )

    # --- 👇👇👇 여기가 문제를 해결하는 핵심 수정 부분입니다 👇👇👇 ---
    # 1. 보간에 성공하고, 새로운 'refined_original_idx' 키가 있는지 확인합니다.
    if (
        final_interpolated_details
        and "refined_original_idx" in final_interpolated_details
    ):
        refined_idx_float = final_interpolated_details["refined_original_idx"]

        # 2. 보간으로 얻은 소수점 인덱스를 가장 가까운 실제 프레임 인덱스(정수)로 변환합니다.
        new_impact_idx = int(round(refined_idx_float))

        if new_impact_idx != final_impact_idx:
            print(
                f"\n✅ [UPDATE] Impact frame has been refined! Changing from index {final_impact_idx} to {new_impact_idx}."
            )

            # 해결책 1: 덮어쓰기 전에 백스윙 정보 '저장'
            saved_backswing_idx = final_impact_info.get("backswing_original_idx", -1)

            final_impact_idx = new_impact_idx
            # 이 줄에서 final_impact_info가 새로운 정보로 덮어쓰기 됩니다.
            final_impact_info = analysis_dataframe_global.loc[
                final_impact_idx
            ].to_dict()

            # 해결책 2: 저장해두었던 백스윙 정보를 '복원'
            if saved_backswing_idx != -1:
                final_impact_info["backswing_original_idx"] = saved_backswing_idx
            fp_contact_check = final_impact_info.get("foot_pos")
            bc_contact_check = final_impact_info.get("ball_center")
            if (
                fp_contact_check
                and bc_contact_check
                and isinstance(fp_contact_check, tuple)
                and isinstance(bc_contact_check, tuple)
            ):
                final_impact_info["contact_region_on_ball"] = get_ball_contact_region(
                    bc_contact_check[0] - fp_contact_check[0],
                    bc_contact_check[1] - bc_contact_check[1],
                )
            else:
                final_impact_info["contact_region_on_ball"] = "N/A"
    # --- 👆👆👆 여기까지가 핵심 수정 부분입니다 👆👆👆 ---

if impact_candidate_list:
    # ======================================================================================
    # === ✨ 단계 1: 초기 임팩트 후보 선정 및 정밀 분석을 통한 최종 임팩트 프레임 확정 ✨ ===
    # ======================================================================================

    impact_candidate_list.sort(key=lambda x: x["score"])
    best_candidate_initial = impact_candidate_list[0]
    final_impact_idx = best_candidate_initial["df_idx"]
    # final_interpolated_details 변수는 이미 외부에서 None으로 선언되었으므로 여기서 다시 선언할 필요 없음

    if USE_FRAME_INTERPOLATION:
        interpolation_result = find_refined_impact_with_scipy(
            analysis_dataframe_global,
            final_impact_idx,
            window_radius=INTERPOLATION_WINDOW_RADIUS,
            density_factor=INTERPOLATION_DENSITY_FACTOR,
        )
        if interpolation_result and "refined_original_idx" in interpolation_result:
            final_interpolated_details = interpolation_result
            refined_idx_float = interpolation_result["refined_original_idx"]
            new_impact_idx = int(round(refined_idx_float))
            if new_impact_idx != final_impact_idx:
                print(
                    f"\n✅ [UPDATE] Impact frame has been refined! Changing from {final_impact_idx} to {new_impact_idx}."
                )
                final_impact_idx = new_impact_idx

    # =================================================================================
    # === ✨ 단계 2: 확정된 임팩트 프레임 기준으로 모든 의존성 데이터 재계산 ✨ ===
    # =================================================================================

    final_impact_info = analysis_dataframe_global.loc[final_impact_idx].to_dict()
    print(
        f" Final Impact Frame confirmed at index: {final_impact_idx}. Re-calculating all dependent metrics..."
    )


# --- 백스윙 정점 재탐색 ---
kicking_foot_final = final_impact_info.get("kicking_foot")
if (
    USE_BACKSWING_APPROACH_PATTERN
    and kicking_foot_final
    and kicking_foot_final != "N/A"
):
    search_window_end = final_impact_idx - 1
    search_window_start = max(0, search_window_end - 50)
    search_df = analysis_dataframe_global.loc[search_window_start:search_window_end]
    knee_angle_col = f"{kicking_foot_final}_knee_angle"
    print("💬 kicking_foot_final:", kicking_foot_final)
    print("💬 Search window:", search_window_start, "~", search_window_end)
    print("💬 Knee angle column exists:", knee_angle_col in search_df.columns)
    print("💬 Dropna result length:", len(search_df.dropna(subset=[knee_angle_col])))
    final_impact_info = analysis_dataframe_global.loc[final_impact_idx].to_dict()

    if not search_df.empty and knee_angle_col in search_df.columns:
        valid_candidates = search_df.dropna(subset=[knee_angle_col])
        if not valid_candidates.empty:
            backswing_peak_idx_used_df = valid_candidates[knee_angle_col].idxmin()
            print(
                f"✅ Backswing peak re-calculated for '{kicking_foot_final}' foot at index: {backswing_peak_idx_used_df}"
            )

            final_impact_info["backswing_original_idx"] = backswing_peak_idx_used_df

    fp_contact, bc_contact = final_impact_info.get("foot_pos"), final_impact_info.get(
        "ball_center"
    )
    if (
        fp_contact
        and bc_contact
        and isinstance(fp_contact, tuple)
        and isinstance(bc_contact, tuple)
    ):
        final_impact_info["contact_region_on_ball"] = get_ball_contact_region(
            bc_contact[0] - fp_contact[0], bc_contact[1] - fp_contact[1]
        )
    else:
        final_impact_info["contact_region_on_ball"] = "N/A"

    if backswing_peak_idx_used_df != -1:
        backswing_row = analysis_dataframe_global.loc[backswing_peak_idx_used_df]
        final_impact_info["backswing_peak_distance"] = backswing_row.get(
            f"min_dist_{kicking_foot_final}_foot_to_ball_smoothed"
        )
        final_impact_info["backswing_kicking_knee_angle"] = backswing_row.get(
            f"{kicking_foot_final}_knee_angle"
        )
        final_impact_info["backswing_ball_radius"] = backswing_row.get("ball_radius")
    else:
        final_impact_info["backswing_peak_distance"] = np.nan
        final_impact_info["backswing_kicking_knee_angle"] = np.nan
        final_impact_info["backswing_ball_radius"] = np.nan

    max_foot_speed_for_score_px_fr = 0
    if backswing_peak_idx_used_df != -1:
        speed_search_df = analysis_dataframe_global.loc[
            backswing_peak_idx_used_df:final_impact_idx
        ]
        foot_speeds = speed_search_df["kicking_foot_speed"].dropna()
        if not foot_speeds.empty:
            max_foot_speed_for_score_px_fr = foot_speeds.max()
    final_impact_info["calculated_max_foot_swing_speed_px_fr"] = (
        max_foot_speed_for_score_px_fr
    )

    supporting_foot_side = "left" if kicking_foot_final == "right" else "right"
    supporting_foot_side = "left" if kicking_foot_final == "right" else "right"
    # 재은아 여기 디버깅
    sfa_col = f"{supporting_foot_side}_ankle_pos"
    print(f"[DEBUG] Support ankle column: {sfa_col}")
    print(f"[DEBUG] Available columns: {analysis_dataframe_global.columns.tolist()}")
    print(
        f"[DEBUG] Row at impact idx:\n{analysis_dataframe_global.loc[final_impact_idx]}"
    )

    # 디버깅 여기까쥐 ~~
    sfa_pos = analysis_dataframe_global.loc[final_impact_idx].get(
        f"{supporting_foot_side}_ankle_pos"
    )
    print(f"[DEBUG] sfa_pos (좌표): {sfa_pos} / 타입: {type(sfa_pos)}")
    final_impact_info["supporting_foot_ankle_pos"] = sfa_pos
    if pd.notna(sfa_pos) and pd.notna(bc_contact):
        final_impact_info["dist_ball_to_supporting_foot_ankle"] = np.hypot(
            sfa_pos[0] - bc_contact[0], sfa_pos[1] - bc_contact[1]
        )
    else:
        final_impact_info["dist_ball_to_supporting_foot_ankle"] = np.nan

    if CALCULATE_SUPPORTING_FOOT_STABILITY and backswing_peak_idx_used_df != -1:
        stability_df = analysis_dataframe_global.loc[
            backswing_peak_idx_used_df:final_impact_idx
        ]
        stability_data = stability_df["supporting_foot_displacement"].dropna()
        final_impact_info["supporting_foot_stability"] = (
            stability_data.mean() if not stability_data.empty else np.nan
        )
    else:
        final_impact_info["supporting_foot_stability"] = np.nan

    if not df_ball_kinematics.empty:
        post_impact_speeds = df_ball_kinematics[
            (df_ball_kinematics["original_idx"] > final_impact_idx)
            & (
                df_ball_kinematics["original_idx"]
                <= final_impact_idx + MAX_FRAMES_FOR_MAX_BALL_SPEED_SEARCH
            )
            & (~df_ball_kinematics["is_ball_predicted"])
        ]["ball_speed"].dropna()
        final_impact_info["max_ball_speed_px_fr"] = (
            post_impact_speeds.max() if not post_impact_speeds.empty else np.nan
        )
    else:
        final_impact_info["max_ball_speed_px_fr"] = np.nan

# 임팩트 후보가 아예 없었던 경우에 대한 처리
else:
    print("Critical Error: Could not determine any impact candidates.")
    # 이전에 이미 변수들이 None 또는 -1로 초기화되었으므로 여기서 특별히 할 작업은 없음

# =================================================================================
# === ✨ 단계 3: 최종 정리된 정보로 요약 및 점수 리포트 출력 ✨ ===
# =================================================================================

print_kick_analysis_summary(
    final_impact_idx,
    final_impact_info,
    final_interpolated_details,
    sampling_rate_for_summary,
    original_video_fps,
    pixel_to_cm_scale_global,
)

if final_interpolated_details:
    print("\n\n--- ⚽ Refined Kick Analysis ⚽ ---")
    print("Interpolation result found. Recalculating score with refined data...")
    refined_impact_info = final_impact_info.copy()
    refined_idx = final_interpolated_details["refined_original_idx"]

    point_columns_to_interpolate = [
        "right_shoulder_pos",
        "left_shoulder_pos",
        "right_hip_pos",
        "left_hip_pos",
        "right_knee_pos",
        "left_knee_pos",
        "right_ankle_pos",
        "left_ankle_pos",
        "right_foot_index_pos",
        "left_foot_index_pos",
        "ball_center",
        "foot_pos",
    ]
    interpolated_points = {}

    for col in point_columns_to_interpolate:
        if col in analysis_dataframe_global.columns:
            result = interpolate_point_data_with_quality_and_fallback(
                analysis_dataframe_global, col, refined_idx
            )
            interpolated_points[col] = result.get("interpolated_point")
            # 🔍보간된 좌표가 None인지 확인 (디버깅용)
            if interpolated_points[col] is None:
                print(f"[WARN] 보간 실패: {col} → None 또는 np.nan 반환됨")
    angles_to_refine = {
        "left_hip_angle": ("left_shoulder_pos", "left_hip_pos", "left_knee_pos"),
        "right_hip_angle": ("right_shoulder_pos", "right_hip_pos", "right_knee_pos"),
        "left_knee_angle": ("left_hip_pos", "left_knee_pos", "left_ankle_pos"),
        "right_knee_angle": ("right_hip_pos", "right_knee_pos", "right_ankle_pos"),
        "left_ankle_angle": ("left_knee_pos", "left_ankle_pos", "left_foot_index_pos"),
        "right_ankle_angle": (
            "right_knee_pos",
            "right_ankle_pos",
            "right_foot_index_pos",
        ),
    }

    for angle_name, point_keys in angles_to_refine.items():
        refined_angle = calculate_angle_for_df(
            interpolated_points.get(point_keys[0]),
            interpolated_points.get(point_keys[1]),
            interpolated_points.get(point_keys[2]),
        )
        if not pd.notna(refined_angle):
            refined_angle = final_impact_info.get(angle_name)
        refined_impact_info[angle_name] = refined_angle

    kicking_foot = refined_impact_info.get("kicking_foot")
    supporting_foot_ankle_pos_col = (
        "left_ankle_pos" if kicking_foot == "right" else "right_ankle_pos"
    )
    interp_support_ankle = interpolated_points.get(supporting_foot_ankle_pos_col)
    interp_ball_center = interpolated_points.get("ball_center")
    if interp_support_ankle and interp_ball_center:
        refined_impact_info["dist_ball_to_supporting_foot_ankle"] = np.hypot(
            interp_support_ankle[0] - interp_ball_center[0],
            interp_support_ankle[1] - interp_ball_center[1],
        )

    score_result_refined_global = calculate_kick_score(
        refined_impact_info,
        pixel_to_cm_scale_global,
        fps=original_video_fps
        / (sampling_interval_calc if sampling_interval_calc > 0 else 1),
        max_foot_speed_px_fr=final_impact_info.get(
            "calculated_max_foot_swing_speed_px_fr"
        ),
        analysis_df_for_angles=analysis_dataframe_global,
        impact_df_idx_for_angles=int(refined_idx),
        backswing_df_idx_for_angles=final_impact_info.get("backswing_original_idx", -1),
    )

    if score_result_refined_global:
        print(
            "\n--- ⚽ Refined Kick Performance Score Report (After Interpolation) ⚽ ---"
        )
        print_score_report(score_result_refined_global)
        visualize_interpolated_moment(
            all_processed_frames,
            final_interpolated_details,
            interpolated_points,
            refined_impact_info,
            pixel_to_cm_scale_global,
        )
# --- Matplotlib 상세 시각화 ---

if (
    final_impact_idx != -1
    and final_impact_info is not None
    and final_impact_info
    and 0 <= final_impact_idx < len(all_processed_frames)
):

    img_final_draw = all_processed_frames[final_impact_idx].copy()
    fh_viz, fw_viz, _ = img_final_draw.shape
    scale_viz = get_safe_pixel_to_cm_scale(
        final_impact_info.get("pixel_to_cm_scale", pixel_to_cm_scale_global)
    )

    if (
        0 <= final_impact_idx < len(pose_results_list)
        and pose_results_list[final_impact_idx]
    ):
        mp.solutions.drawing_utils.draw_landmarks(
            img_final_draw,
            pose_results_list[final_impact_idx],
            mp_pose.POSE_CONNECTIONS,
            landmark_drawing_spec=mp.solutions.drawing_utils.DrawingSpec(
                color=(245, 117, 66), thickness=2, circle_radius=4
            ),
            connection_drawing_spec=mp.solutions.drawing_utils.DrawingSpec(
                color=(245, 66, 230), thickness=2, circle_radius=2
            ),
        )

    kicking_foot_viz = final_impact_info.get("kicking_foot")
    if (
        kicking_foot_viz
        and kicking_foot_viz != "N/A"
        and 0 <= final_impact_idx < len(pose_results_list)
        and pose_results_list[final_impact_idx]
    ):
        foot_poly_viz = get_foot_polygon(
            pose_results_list[final_impact_idx], kicking_foot_viz, img_final_draw.shape
        )
        if foot_poly_viz is not None:
            draw_segmentation_mask(img_final_draw, foot_poly_viz, color=FOOT_VIS_COLOR)

    bc_fv, br_fv, bb_fv = (
        final_impact_info.get("ball_center"),
        final_impact_info.get("ball_radius"),
        final_impact_info.get("ball_box"),
    )
    fp_fv = final_impact_info.get("foot_pos")
    sfa_fv = final_impact_info.get("supporting_foot_ankle_pos")
    is_pred_ball_viz = final_impact_info.get("is_ball_predicted", False)
    is_occ_rec_viz = final_impact_info.get("occlusion_recovered", False)
    is_ball_corrected_viz = final_impact_info.get("is_ball_info_corrected", False)

    mask_drawn_on_final_impact_image = False
    ball_mask_points_final_info = final_impact_info.get("mask_points")

    if (
        USE_BALL_SEGMENTATION
        and ball_mask_points_final_info is not None
        and not is_pred_ball_viz
    ):
        draw_segmentation_mask(
            img_final_draw,
            np.array(ball_mask_points_final_info),
            color=SEGMENTATION_VIS_COLOR,
        )
        mask_drawn_on_final_impact_image = True
    elif (
        USE_BALL_SEGMENTATION
        and 0 <= final_impact_idx < len(ball_detections_list)
        and not is_pred_ball_viz
    ):
        ball_data_final_viz = ball_detections_list[final_impact_idx]
        if ball_data_final_viz and ball_data_final_viz.get("mask_points") is not None:
            draw_segmentation_mask(
                img_final_draw,
                ball_data_final_viz["mask_points"],
                color=SEGMENTATION_VIS_COLOR,
            )
            mask_drawn_on_final_impact_image = True

    if not mask_drawn_on_final_impact_image:
        if (
            bb_fv
            and all(isinstance(c, (int, float, np.integer, np.floating)) for c in bb_fv)
            and len(bb_fv) == 4
        ):
            box_color_viz = (
                (255, 100, 100)
                if is_ball_corrected_viz
                else (
                    (255, 165, 0)
                    if is_pred_ball_viz
                    else ((0, 0, 255) if is_occ_rec_viz else (0, 255, 0))
                )
            )
            cv2.rectangle(
                img_final_draw,
                (int(bb_fv[0]), int(bb_fv[1])),
                (int(bb_fv[2]), int(bb_fv[3])),
                box_color_viz,
                2,
            )
        if (
            bc_fv
            and br_fv
            and br_fv > 0
            and isinstance(bc_fv, tuple)
            and len(bc_fv) == 2
        ):
            circle_color_viz = (
                (255, 100, 100)
                if is_ball_corrected_viz
                else (
                    (255, 165, 0)
                    if is_pred_ball_viz
                    else ((0, 0, 255) if is_occ_rec_viz else (0, 255, 255))
                )
            )
            bc_int_fv = tuple(map(int, bc_fv))
            cv2.circle(img_final_draw, bc_int_fv, int(br_fv), circle_color_viz, 1)

    if bc_fv and isinstance(bc_fv, tuple) and len(bc_fv) == 2:
        bc_int_fv = tuple(map(int, bc_fv))
        cv2.circle(img_final_draw, bc_int_fv, 5, (0, 0, 255), -1)
        if (
            fp_fv
            and isinstance(fp_fv, tuple)
            and len(fp_fv) == 2
            and br_fv
            and br_fv > 0
        ):
            fp_int_fv = tuple(map(int, fp_fv))
            cv2.circle(img_final_draw, fp_int_fv, 7, (255, 0, 0), -1)
            cv2.line(img_final_draw, fp_int_fv, bc_int_fv, (255, 255, 0), 1)
            vxs, vys = bc_fv[0] - fp_fv[0], bc_fv[1] - fp_fv[1]
            ds = np.hypot(vxs, vys)
            if ds > 1e-6:
                nxs, nys = vxs / ds, vys / ds
                cpts = (int(bc_fv[0] - nxs * br_fv), int(bc_fv[1] - nys * br_fv))
                cv2.circle(img_final_draw, cpts, 6, (0, 255, 255), -1)
                cv2.putText(
                    img_final_draw,
                    f"Hit: Ball's {final_impact_info.get('contact_region_on_ball','N/A')}",
                    (
                        bc_int_fv[0]
                        + int(br_fv if pd.notna(br_fv) and br_fv > 0 else 10)
                        + 5,
                        bc_int_fv[1],
                    ),
                    cv2.FONT_HERSHEY_SIMPLEX,
                    0.6,
                    (255, 255, 0),
                    2,
                )
    if sfa_fv and isinstance(sfa_fv, tuple) and len(sfa_fv) == 2:
        cv2.circle(img_final_draw, tuple(map(int, sfa_fv)), 7, (255, 0, 255), -1)

    y_off_fv, lh_fv, fs_fv, fc_fv, ft_fv = 30, 25, 0.6, (255, 255, 255), 1
    tbg_fv = (0, 0, 0)
    title_suffix_viz = ""
    if is_ball_corrected_viz:
        title_suffix_viz += " (Ball Corrected)"
    elif is_pred_ball_viz:
        title_suffix_viz += " (Ball Predicted)"
    elif is_occ_rec_viz:
        title_suffix_viz += " (Occlusion Recovered)"

    texts_final_viz = [
        f"Impact Frame (Original): {final_impact_idx}{title_suffix_viz}",
        f"Kicking Foot: {final_impact_info.get('kicking_foot','N/A')} ({final_impact_info.get('kicking_foot_part','toe')})",
    ]
    if isinstance(final_impact_info.get("distance"), (int, float)) and pd.notna(
        final_impact_info.get("distance")
    ):
        texts_final_viz.append(
            f"Dist (Foot-Ball): {final_impact_info['distance']:.1f}px (~{final_impact_info['distance']*scale_viz:.1f}cm)"
        )
    if isinstance(
        final_impact_info.get("dist_ball_to_supporting_foot_ankle"), (int, float)
    ) and pd.notna(final_impact_info.get("dist_ball_to_supporting_foot_ankle")):
        texts_final_viz.append(
            f"Dist (SupportFt-Ball): {final_impact_info['dist_ball_to_supporting_foot_ankle']:.1f}px (~{final_impact_info['dist_ball_to_supporting_foot_ankle']*scale_viz:.1f}cm)"
        )
    if final_interpolated_details:
        interp_d_px_fv, interp_idx_fv = final_interpolated_details.get(
            "min_interpolated_distance"
        ), final_interpolated_details.get("refined_original_idx")
        if isinstance(interp_d_px_fv, (int, float)) and pd.notna(interp_d_px_fv):
            texts_final_viz.append(
                f"Dist (Interpolated): {interp_d_px_fv:.2f}px (~{interp_d_px_fv*scale_viz:.2f}cm)"
            )
        if isinstance(interp_idx_fv, (int, float)) and pd.notna(interp_idx_fv):
            texts_final_viz.append(
                f"Time (Interpolated): {interp_idx_fv:.2f} (Original Idx)"
            )

    analyzed_fps_for_viz = (
        original_video_fps / sampling_rate_for_summary
        if sampling_rate_for_summary > 0
        else original_video_fps
    )

    max_bs_fv_px_fr = final_impact_info.get("max_ball_speed_px_fr")
    if pd.notna(max_bs_fv_px_fr):
        max_bs_cm_s = max_bs_fv_px_fr * scale_viz * analyzed_fps_for_viz
        max_bs_km_h = max_bs_cm_s * CM_S_TO_KM_H
        texts_final_viz.append(
            f"Max Ball Speed After Impact: {max_bs_fv_px_fr:.1f}px/fr (~{max_bs_km_h:.1f}km/h)"
        )

    max_foot_speed_overall_px_fr_viz = final_impact_info.get(
        "calculated_max_foot_swing_speed_px_fr", 0
    )
    if (
        pd.notna(max_foot_speed_overall_px_fr_viz)
        and max_foot_speed_overall_px_fr_viz > 0
    ):
        max_foot_speed_cm_s_viz = (
            max_foot_speed_overall_px_fr_viz * scale_viz * analyzed_fps_for_viz
        )
        max_foot_speed_km_h_viz = max_foot_speed_cm_s_viz * CM_S_TO_KM_H
        texts_final_viz.append(
            f"Max Foot Swing Speed (0~Imp): {max_foot_speed_overall_px_fr_viz:.1f}px/fr (~{max_foot_speed_km_h_viz:.1f}km/h)"
        )

    backswing_knee_viz = final_impact_info.get("backswing_kicking_knee_angle")
    if pd.notna(backswing_knee_viz):
        texts_final_viz.append(f"Backswing Knee Angle: {backswing_knee_viz:.1f} deg")

    support_stability_px_fr_viz = final_impact_info.get("supporting_foot_stability")
    if pd.notna(support_stability_px_fr_viz):
        support_stability_cm_fr_viz = support_stability_px_fr_viz * scale_viz
        support_stability_km_h_viz = (
            support_stability_cm_fr_viz * analyzed_fps_for_viz * CM_S_TO_KM_H
        )
        texts_final_viz.append(
            f"Support Foot Stability (Avg Disp.): {support_stability_px_fr_viz:.2f} px/fr (~{support_stability_cm_fr_viz:.2f}cm/fr | ~{support_stability_km_h_viz:.4f} km/h)"
        )

    for txt_item_fv in texts_final_viz:
        (wt_fv, ht_fv), _ = cv2.getTextSize(
            txt_item_fv, cv2.FONT_HERSHEY_SIMPLEX, fs_fv, ft_fv
        )
        cv2.rectangle(
            img_final_draw,
            (10, y_off_fv - ht_fv - 5),
            (10 + wt_fv + 5, y_off_fv + 5),
            tbg_fv,
            -1,
        )
        cv2.putText(
            img_final_draw,
            txt_item_fv,
            (15, y_off_fv),
            cv2.FONT_HERSHEY_SIMPLEX,
            fs_fv,
            fc_fv,
            ft_fv,
            cv2.LINE_AA,
        )
        y_off_fv += lh_fv

    plt.figure(figsize=(12, 9))
    plt.imshow(cv2.cvtColor(img_final_draw, cv2.COLOR_BGR2RGB))
    plt.title(
        f"Detailed Kick Impact Analysis (Frame {final_impact_idx}{title_suffix_viz})"
    )
    plt.axis("off")
    plt.show()

    print("Key frames around impact:")

# --- Matplotlib 상세 시각화 ---
if (
    final_impact_idx != -1
    and final_impact_info is not None
    and final_impact_info
    and 0 <= final_impact_idx < len(all_processed_frames)
):

    # --- 임팩트 순간 상세 시각화 ---
    img_final_draw = all_processed_frames[final_impact_idx].copy()
    fh_viz, fw_viz, _ = img_final_draw.shape
    scale_viz = get_safe_pixel_to_cm_scale(
        final_impact_info.get("pixel_to_cm_scale", pixel_to_cm_scale_global)
    )

    if (
        0 <= final_impact_idx < len(pose_results_list)
        and pose_results_list[final_impact_idx]
    ):
        mp.solutions.drawing_utils.draw_landmarks(
            img_final_draw,
            pose_results_list[final_impact_idx],
            mp_pose.POSE_CONNECTIONS,
            landmark_drawing_spec=mp.solutions.drawing_utils.DrawingSpec(
                color=(245, 117, 66), thickness=2, circle_radius=4
            ),
            connection_drawing_spec=mp.solutions.drawing_utils.DrawingSpec(
                color=(245, 66, 230), thickness=2, circle_radius=2
            ),
        )

    kicking_foot_viz = final_impact_info.get("kicking_foot")
    if (
        kicking_foot_viz
        and kicking_foot_viz != "N/A"
        and 0 <= final_impact_idx < len(pose_results_list)
        and pose_results_list[final_impact_idx]
    ):
        foot_poly_viz = get_foot_polygon(
            pose_results_list[final_impact_idx], kicking_foot_viz, img_final_draw.shape
        )
        if foot_poly_viz is not None:
            draw_segmentation_mask(img_final_draw, foot_poly_viz, color=FOOT_VIS_COLOR)

    bc_fv, br_fv, bb_fv = (
        final_impact_info.get("ball_center"),
        final_impact_info.get("ball_radius"),
        final_impact_info.get("ball_box"),
    )
    fp_fv = final_impact_info.get("foot_pos")
    sfa_fv = final_impact_info.get("supporting_foot_ankle_pos")
    is_pred_ball_viz = final_impact_info.get("is_ball_predicted", False)
    is_occ_rec_viz = final_impact_info.get("occlusion_recovered", False)
    is_ball_corrected_viz = final_impact_info.get("is_ball_info_corrected", False)

    mask_drawn_on_final_impact_image = False
    ball_mask_points_final_info = final_impact_info.get("mask_points")

    if (
        USE_BALL_SEGMENTATION
        and ball_mask_points_final_info is not None
        and not is_pred_ball_viz
    ):
        draw_segmentation_mask(
            img_final_draw,
            np.array(ball_mask_points_final_info),
            color=SEGMENTATION_VIS_COLOR,
        )
        mask_drawn_on_final_impact_image = True
    elif (
        USE_BALL_SEGMENTATION
        and 0 <= final_impact_idx < len(ball_detections_list)
        and not is_pred_ball_viz
    ):
        ball_data_final_viz = ball_detections_list[final_impact_idx]
        if ball_data_final_viz and ball_data_final_viz.get("mask_points") is not None:
            draw_segmentation_mask(
                img_final_draw,
                ball_data_final_viz["mask_points"],
                color=SEGMENTATION_VIS_COLOR,
            )
            mask_drawn_on_final_impact_image = True

    if not mask_drawn_on_final_impact_image:
        if (
            bb_fv
            and all(isinstance(c, (int, float, np.integer, np.floating)) for c in bb_fv)
            and len(bb_fv) == 4
        ):
            box_color_viz = (
                (255, 100, 100)
                if is_ball_corrected_viz
                else (
                    (255, 165, 0)
                    if is_pred_ball_viz
                    else ((0, 0, 255) if is_occ_rec_viz else (0, 255, 0))
                )
            )
            cv2.rectangle(
                img_final_draw,
                (int(bb_fv[0]), int(bb_fv[1])),
                (int(bb_fv[2]), int(bb_fv[3])),
                box_color_viz,
                2,
            )
        if (
            bc_fv
            and br_fv
            and br_fv > 0
            and isinstance(bc_fv, tuple)
            and len(bc_fv) == 2
        ):
            circle_color_viz = (
                (255, 100, 100)
                if is_ball_corrected_viz
                else (
                    (255, 165, 0)
                    if is_pred_ball_viz
                    else ((0, 0, 255) if is_occ_rec_viz else (0, 255, 255))
                )
            )
            bc_int_fv = tuple(map(int, bc_fv))
            cv2.circle(img_final_draw, bc_int_fv, int(br_fv), circle_color_viz, 1)

    if bc_fv and isinstance(bc_fv, tuple) and len(bc_fv) == 2:
        bc_int_fv = tuple(map(int, bc_fv))
        cv2.circle(img_final_draw, bc_int_fv, 5, (0, 0, 255), -1)
        if (
            fp_fv
            and isinstance(fp_fv, tuple)
            and len(fp_fv) == 2
            and br_fv
            and br_fv > 0
        ):
            fp_int_fv = tuple(map(int, fp_fv))
            cv2.circle(img_final_draw, fp_int_fv, 7, (255, 0, 0), -1)
            cv2.line(img_final_draw, fp_int_fv, bc_int_fv, (255, 255, 0), 1)
            vxs, vys = bc_fv[0] - fp_fv[0], bc_fv[1] - fp_fv[1]
            ds = np.hypot(vxs, vys)
            if ds > 1e-6:
                nxs, nys = vxs / ds, vys / ds
                cpts = (int(bc_fv[0] - nxs * br_fv), int(bc_fv[1] - nys * br_fv))
                cv2.circle(img_final_draw, cpts, 6, (0, 255, 255), -1)
                cv2.putText(
                    img_final_draw,
                    f"Hit: Ball's {final_impact_info.get('contact_region_on_ball','N/A')}",
                    (
                        bc_int_fv[0]
                        + int(br_fv if pd.notna(br_fv) and br_fv > 0 else 10)
                        + 5,
                        bc_int_fv[1],
                    ),
                    cv2.FONT_HERSHEY_SIMPLEX,
                    0.6,
                    (255, 255, 0),
                    2,
                )
    if sfa_fv and isinstance(sfa_fv, tuple) and len(sfa_fv) == 2:
        cv2.circle(img_final_draw, tuple(map(int, sfa_fv)), 7, (255, 0, 255), -1)

    y_off_fv, lh_fv, fs_fv, fc_fv, ft_fv = 30, 25, 0.6, (255, 255, 255), 1
    tbg_fv = (0, 0, 0)
    title_suffix_viz = ""
    if is_ball_corrected_viz:
        title_suffix_viz += " (Ball Corrected)"
    elif is_pred_ball_viz:
        title_suffix_viz += " (Ball Predicted)"
    elif is_occ_rec_viz:
        title_suffix_viz += " (Occlusion Recovered)"

    texts_final_viz = [
        f"Impact Frame (Original): {final_impact_idx}{title_suffix_viz}",
        f"Kicking Foot: {final_impact_info.get('kicking_foot','N/A')} ({final_impact_info.get('kicking_foot_part','toe')})",
    ]
    if isinstance(final_impact_info.get("distance"), (int, float)) and pd.notna(
        final_impact_info.get("distance")
    ):
        texts_final_viz.append(
            f"Dist (Foot-Ball): {final_impact_info['distance']:.1f}px (~{final_impact_info['distance']*scale_viz:.1f}cm)"
        )
    if isinstance(
        final_impact_info.get("dist_ball_to_supporting_foot_ankle"), (int, float)
    ) and pd.notna(final_impact_info.get("dist_ball_to_supporting_foot_ankle")):
        texts_final_viz.append(
            f"Dist (SupportFt-Ball): {final_impact_info['dist_ball_to_supporting_foot_ankle']:.1f}px (~{final_impact_info['dist_ball_to_supporting_foot_ankle']*scale_viz:.1f}cm)"
        )
    if final_interpolated_details:
        interp_d_px_fv, interp_idx_fv = final_interpolated_details.get(
            "min_interpolated_distance"
        ), final_interpolated_details.get("refined_original_idx")
        if isinstance(interp_d_px_fv, (int, float)) and pd.notna(interp_d_px_fv):
            texts_final_viz.append(
                f"Dist (Interpolated): {interp_d_px_fv:.2f}px (~{interp_d_px_fv*scale_viz:.2f}cm)"
            )
        if isinstance(interp_idx_fv, (int, float)) and pd.notna(interp_idx_fv):
            texts_final_viz.append(
                f"Time (Interpolated): {interp_idx_fv:.2f} (Original Idx)"
            )

    analyzed_fps_for_viz = (
        original_video_fps / sampling_rate_for_summary
        if sampling_rate_for_summary > 0
        else original_video_fps
    )

    max_bs_fv_px_fr = final_impact_info.get("max_ball_speed_px_fr")
    if pd.notna(max_bs_fv_px_fr):
        max_bs_cm_s = max_bs_fv_px_fr * scale_viz * analyzed_fps_for_viz
        max_bs_km_h = max_bs_cm_s * CM_S_TO_KM_H
        texts_final_viz.append(
            f"Max Ball Speed After Impact: {max_bs_fv_px_fr:.1f}px/fr (~{max_bs_km_h:.1f}km/h)"
        )

    max_foot_speed_overall_px_fr_viz = final_impact_info.get(
        "calculated_max_foot_swing_speed_px_fr", 0
    )
    if (
        pd.notna(max_foot_speed_overall_px_fr_viz)
        and max_foot_speed_overall_px_fr_viz > 0
    ):
        max_foot_speed_cm_s_viz = (
            max_foot_speed_overall_px_fr_viz * scale_viz * analyzed_fps_for_viz
        )
        max_foot_speed_km_h_viz = max_foot_speed_cm_s_viz * CM_S_TO_KM_H
        texts_final_viz.append(
            f"Max Foot Swing Speed (0~Imp): {max_foot_speed_overall_px_fr_viz:.1f}px/fr (~{max_foot_speed_km_h_viz:.1f}km/h)"
        )

    backswing_knee_viz = final_impact_info.get("backswing_kicking_knee_angle")
    if pd.notna(backswing_knee_viz):
        texts_final_viz.append(f"Backswing Knee Angle: {backswing_knee_viz:.1f} deg")

    support_stability_px_fr_viz = final_impact_info.get("supporting_foot_stability")
    if pd.notna(support_stability_px_fr_viz):
        support_stability_cm_fr_viz = support_stability_px_fr_viz * scale_viz
        support_stability_km_h_viz = (
            support_stability_cm_fr_viz * analyzed_fps_for_viz * CM_S_TO_KM_H
        )
        texts_final_viz.append(
            f"Support Foot Stability (Avg Disp.): {support_stability_px_fr_viz:.2f} px/fr (~{support_stability_cm_fr_viz:.2f}cm/fr | ~{support_stability_km_h_viz:.4f} km/h)"
        )

    for txt_item_fv in texts_final_viz:
        (wt_fv, ht_fv), _ = cv2.getTextSize(
            txt_item_fv, cv2.FONT_HERSHEY_SIMPLEX, fs_fv, ft_fv
        )
        cv2.rectangle(
            img_final_draw,
            (10, y_off_fv - ht_fv - 5),
            (10 + wt_fv + 5, y_off_fv + 5),
            tbg_fv,
            -1,
        )
        cv2.putText(
            img_final_draw,
            txt_item_fv,
            (15, y_off_fv),
            cv2.FONT_HERSHEY_SIMPLEX,
            fs_fv,
            fc_fv,
            ft_fv,
            cv2.LINE_AA,
        )
        y_off_fv += lh_fv

    plt.figure(figsize=(12, 9))
    plt.imshow(cv2.cvtColor(img_final_draw, cv2.COLOR_BGR2RGB))
    plt.title(
        f"Detailed Kick Impact Analysis (Frame {final_impact_idx}{title_suffix_viz})"
    )
    plt.axis("off")
    plt.show()

    print("Key frames around impact:")
    plt.figure(figsize=(18, 3.5))
    num_fr_around, total_plt_fr = 2, 1 + (2 * 2)
    for i_plt in range(total_plt_fr):
        offset_plt = i_plt - num_fr_around
        fr_idx_plt = final_impact_idx + offset_plt
        if (
            0 <= fr_idx_plt < len(all_processed_frames)
            and 0 <= fr_idx_plt < len(pose_results_list)
            and 0 <= fr_idx_plt < len(ball_detections_list)
        ):
            plt.subplot(1, total_plt_fr, i_plt + 1)
            fr_disp_plt = all_processed_frames[fr_idx_plt].copy()
            title_suffix_seq = ""
            current_pose_seq = pose_results_list[fr_idx_plt]
            ball_inf_s = ball_detections_list[fr_idx_plt]

            if ball_inf_s:
                is_pred_ball_seq = ball_inf_s.get("is_predicted", False)
                is_occ_rec_seq = ball_inf_s.get("occlusion_recovered", False)
                if is_pred_ball_seq:
                    title_suffix_seq = " (P)"
                elif is_occ_rec_seq:
                    title_suffix_seq = " (R)"

                kicking_foot_df_row_seq_df = analysis_dataframe_global[
                    analysis_dataframe_global["original_idx"] == fr_idx_plt
                ]
                kicking_foot_seq = (
                    kicking_foot_df_row_seq_df["kicking_foot"].iloc[0]
                    if not kicking_foot_df_row_seq_df.empty
                    and "kicking_foot" in kicking_foot_df_row_seq_df.columns
                    else None
                )
                if kicking_foot_seq and kicking_foot_seq != "N/A" and current_pose_seq:
                    foot_poly_seq = get_foot_polygon(
                        current_pose_seq, kicking_foot_seq, fr_disp_plt.shape
                    )
                    if foot_poly_seq is not None:
                        draw_segmentation_mask(
                            fr_disp_plt, foot_poly_seq, color=FOOT_VIS_COLOR
                        )

                mask_points_seq = ball_inf_s.get("mask_points")
                if (
                    USE_BALL_SEGMENTATION
                    and mask_points_seq is not None
                    and not is_pred_ball_seq
                ):
                    draw_segmentation_mask(
                        fr_disp_plt, mask_points_seq, color=SEGMENTATION_VIS_COLOR
                    )
                elif (
                    ball_inf_s.get("center")
                    and ball_inf_s.get("radius")
                    and ball_inf_s.get("radius") > 0
                ):
                    ball_color_seq = (
                        (255, 165, 0)
                        if is_pred_ball_seq
                        else ((0, 0, 255) if is_occ_rec_seq else (0, 255, 255))
                    )
                    try:
                        cv2.circle(
                            fr_disp_plt,
                            tuple(map(int, ball_inf_s["center"])),
                            int(ball_inf_s["radius"]),
                            ball_color_seq,
                            1,
                        )
                    except (ValueError, TypeError):
                        pass

            if current_pose_seq:
                mp.solutions.drawing_utils.draw_landmarks(
                    fr_disp_plt,
                    current_pose_seq,
                    mp_pose.POSE_CONNECTIONS,
                    landmark_drawing_spec=mp.solutions.drawing_utils.DrawingSpec(
                        color=(0, 255, 0), thickness=1, circle_radius=1
                    ),
                    connection_drawing_spec=mp.solutions.drawing_utils.DrawingSpec(
                        color=(255, 0, 0), thickness=1
                    ),
                )
            plt.imshow(cv2.cvtColor(fr_disp_plt, cv2.COLOR_BGR2RGB))
            title_plt = "Impact" if offset_plt == 0 else f"{offset_plt:^+d}"
            plt.title(f"{title_plt} (F:{fr_idx_plt}){title_suffix_seq}")
            plt.axis("off")
    plt.tight_layout()
    plt.show()

    # ✨✨✨ 백스윙 정점 시각화 (차는 발 기준) - 여기에 추가 ✨✨✨
    if final_impact_info and "backswing_original_idx" in final_impact_info:
        bs_peak_df_idx = final_impact_info.get("backswing_original_idx")
        kicking_foot_at_bs = final_impact_info.get("kicking_foot")

        if (
            bs_peak_df_idx != -1
            and kicking_foot_at_bs
            and kicking_foot_at_bs != "N/A"
            and 0 <= bs_peak_df_idx < len(all_processed_frames)
            and 0 <= bs_peak_df_idx < len(pose_results_list)
            and 0 <= bs_peak_df_idx < len(ball_detections_list)
        ):

            print(
                f"\nVisualizing Kicking Foot Backswing Peak at DataFrame index: {bs_peak_df_idx} (Original Frame Index in Sampled List)"
            )

            img_bs_peak_draw = all_processed_frames[bs_peak_df_idx].copy()
            pose_data_bs_peak = pose_results_list[bs_peak_df_idx]
            ball_data_bs_peak = ball_detections_list[bs_peak_df_idx]

            # 포즈 랜드마크 그리기
            if pose_data_bs_peak:
                mp.solutions.drawing_utils.draw_landmarks(
                    img_bs_peak_draw,
                    pose_data_bs_peak,
                    mp_pose.POSE_CONNECTIONS,
                    landmark_drawing_spec=mp.solutions.drawing_utils.DrawingSpec(
                        color=(0, 255, 0), thickness=2, circle_radius=2
                    ),  # 초록색 점
                    connection_drawing_spec=mp.solutions.drawing_utils.DrawingSpec(
                        color=(255, 0, 0), thickness=2
                    ),  # 파란색 선
                )

            # 차는 발 폴리곤 그리기
            if (
                kicking_foot_at_bs != "N/A" and pose_data_bs_peak
            ):  # kicking_foot_at_bs가 'N/A'가 아닌지 확인
                foot_poly_bs_peak = get_foot_polygon(
                    pose_data_bs_peak, kicking_foot_at_bs, img_bs_peak_draw.shape
                )
                if foot_poly_bs_peak is not None:
                    draw_segmentation_mask(
                        img_bs_peak_draw, foot_poly_bs_peak, color=FOOT_VIS_COLOR
                    )

            # 공 정보 그리기 (마스크 또는 원)
            if ball_data_bs_peak:
                is_predicted_ball_bs = ball_data_bs_peak.get("is_predicted", False)
                mask_points_bs = ball_data_bs_peak.get("mask_points")
                if (
                    USE_BALL_SEGMENTATION
                    and mask_points_bs is not None
                    and not is_predicted_ball_bs
                ):
                    draw_segmentation_mask(
                        img_bs_peak_draw, mask_points_bs, color=SEGMENTATION_VIS_COLOR
                    )
                elif (
                    ball_data_bs_peak.get("center")
                    and ball_data_bs_peak.get("radius")
                    and ball_data_bs_peak.get("radius") > 0
                ):
                    ball_color_bs = (
                        (255, 165, 0) if is_predicted_ball_bs else (0, 0, 255)
                    )  # 예측된 공은 주황색, 아니면 파란색
                    cv2.circle(
                        img_bs_peak_draw,
                        tuple(map(int, ball_data_bs_peak["center"])),
                        int(ball_data_bs_peak["radius"]),
                        ball_color_bs,
                        2,
                    )

            # 백스윙 정점 프레임 정보 표시 (DataFrame에서 가져오기)
            title_bs_lines = [
                f"Backswing Peak ({kicking_foot_at_bs} Foot) - Frame: {bs_peak_df_idx}"
            ]
            bs_peak_row_series = analysis_dataframe_global[
                analysis_dataframe_global["original_idx"] == bs_peak_df_idx
            ]
            if not bs_peak_row_series.empty:
                bs_peak_row = bs_peak_row_series.iloc[0]

                bs_peak_dist_val = bs_peak_row.get(
                    f"min_dist_{kicking_foot_at_bs}_foot_to_ball_smoothed", np.nan
                )
                bs_peak_dist_cm_str = (
                    f"{bs_peak_dist_val * scale_viz:.1f}cm"
                    if pd.notna(bs_peak_dist_val) and pd.notna(scale_viz)
                    else "N/A"
                )
                title_bs_lines.append(
                    f"Foot-Ball Dist (Smooth): {bs_peak_dist_val:.1f}px (~{bs_peak_dist_cm_str})"
                )

                bs_knee_angle_val = bs_peak_row.get(
                    f"{kicking_foot_at_bs}_knee_angle", np.nan
                )
                bs_knee_angle_str = (
                    f"{bs_knee_angle_val:.1f} deg"
                    if pd.notna(bs_knee_angle_val)
                    else "N/A"
                )
                title_bs_lines.append(f"Kicking Knee Angle: {bs_knee_angle_str}")

            # 텍스트 그리기 (Matplotlib 타이틀 대신 cv2.putText 사용 가능, 여기서는 타이틀 활용)
            plt.figure(figsize=(10, 8))  # 크기 조정
            plt.imshow(cv2.cvtColor(img_bs_peak_draw, cv2.COLOR_BGR2RGB))
            plt.title("\n".join(title_bs_lines), fontsize=10)  # 여러 줄 타이틀
            plt.axis("off")
            plt.tight_layout()  # 타이틀 잘리지 않도록
            plt.show()

            # 선택적: 파일로 저장
            # os.makedirs("backswing_peak_captures", exist_ok=True) # 스크립트 상단에 이미 있음
            # backswing_capture_filename = f"bs_peak_origidx{bs_peak_df_idx}_{kicking_foot_at_bs}.jpg"
            # cv2.imwrite(os.path.join("backswing_peak_captures", backswing_capture_filename), img_bs_peak_draw)
            # print(f"Backswing peak capture saved to: {os.path.join('backswing_peak_captures', backswing_capture_filename)}")
        else:
            print(
                "Backswing peak index not valid or data missing for kick-foot backswing visualization."
            )
    else:
        print(
            "Backswing peak information not found in final_impact_info for visualization."
        )
else:
    print(
        "Final impact not determined or related info is missing; detailed visualization skipped."
    )

# --- 결과 비디오 생성 ---
if not all_processed_frames:
    print("Error: No processed frames available to generate result video.")
else:
    print("\n--- Starting result video generation ---")
    # output_video_path는 이미 스크립트 상단에서 video_name_no_ext와 timestamp를 이용해 정의되어 있습니다.
    frame_height, frame_width, _ = all_processed_frames[0].shape
    output_fps_vid = (
        original_video_fps / sampling_interval_calc
        if sampling_interval_calc > 0
        else original_video_fps
    )
    if output_fps_vid <= 0:
        output_fps_vid = 30.0
        print(
            f"Warning: Calculated output FPS is <= 0. Defaulting to {output_fps_vid} FPS."
        )
    fourcc = cv2.VideoWriter_fourcc(*"mp4v")
    out_video = cv2.VideoWriter(
        output_video_path, fourcc, output_fps_vid, (frame_width, frame_height)
    )

    if not out_video.isOpened():
        print(
            f"Error: Could not open VideoWriter. Check path or codec: {output_video_path}"
        )
    else:
        analyzed_fps_for_video_val = (
            original_video_fps / sampling_interval_calc
            if sampling_interval_calc > 0
            else original_video_fps
        )
        max_foot_speed_overall_px_fr_vid = 0
        max_foot_speed_km_h_vid = 0

        if (
            final_impact_idx != -1
            and final_impact_info
            and "kicking_foot" in final_impact_info
        ):
            max_foot_speed_overall_px_fr_vid = final_impact_info.get(
                "calculated_max_foot_swing_speed_px_fr", 0
            )
            if (
                pd.notna(max_foot_speed_overall_px_fr_vid)
                and max_foot_speed_overall_px_fr_vid > 0
            ):
                scale_for_max_speed_vid = get_safe_pixel_to_cm_scale(
                    final_impact_info.get("pixel_to_cm_scale", pixel_to_cm_scale_global)
                )
                max_foot_speed_cm_s_vid = (
                    max_foot_speed_overall_px_fr_vid
                    * scale_for_max_speed_vid
                    * analyzed_fps_for_video_val
                )
                max_foot_speed_km_h_vid = max_foot_speed_cm_s_vid * CM_S_TO_KM_H

        for frame_idx_vid, original_frame_vid in enumerate(all_processed_frames):
            processed_frame_for_video = original_frame_vid.copy()
            current_scale_vid = get_safe_pixel_to_cm_scale(pixel_to_cm_scale_global)
            current_pose_video_item = (
                pose_results_list[frame_idx_vid]
                if frame_idx_vid < len(pose_results_list)
                else None
            )

            if current_pose_video_item:
                mp.solutions.drawing_utils.draw_landmarks(
                    processed_frame_for_video,
                    current_pose_video_item,
                    mp_pose.POSE_CONNECTIONS,
                    landmark_drawing_spec=mp.solutions.drawing_utils.DrawingSpec(
                        color=(0, 255, 0), thickness=1, circle_radius=2
                    ),
                    connection_drawing_spec=mp.solutions.drawing_utils.DrawingSpec(
                        color=(0, 0, 255), thickness=1, circle_radius=1
                    ),
                )

            current_analysis_row_for_foot_df_vid = analysis_dataframe_global[
                analysis_dataframe_global["original_idx"] == frame_idx_vid
            ]
            current_analysis_row_text_item = (
                current_analysis_row_for_foot_df_vid.iloc[0]
                if not current_analysis_row_for_foot_df_vid.empty
                else None
            )

            if current_analysis_row_text_item is not None:
                kicking_foot_video_item = current_analysis_row_text_item.get(
                    "kicking_foot"
                )
                if (
                    kicking_foot_video_item
                    and kicking_foot_video_item != "N/A"
                    and current_pose_video_item
                ):
                    foot_poly_video_item = get_foot_polygon(
                        current_pose_video_item,
                        kicking_foot_video_item,
                        processed_frame_for_video.shape,
                    )
                    if foot_poly_video_item is not None:
                        draw_segmentation_mask(
                            processed_frame_for_video,
                            foot_poly_video_item,
                            color=FOOT_VIS_COLOR,
                        )

            current_ball_info_video_item = (
                ball_detections_list[frame_idx_vid]
                if frame_idx_vid < len(ball_detections_list)
                else None
            )
            if current_ball_info_video_item:
                is_predicted_vid = current_ball_info_video_item.get(
                    "is_predicted", False
                )
                mask_points_vid = current_ball_info_video_item.get("mask_points")
                mask_drawn_on_video_frame_flag = False
                if (
                    USE_BALL_SEGMENTATION
                    and mask_points_vid is not None
                    and not is_predicted_vid
                ):
                    draw_segmentation_mask(
                        processed_frame_for_video,
                        mask_points_vid,
                        color=SEGMENTATION_VIS_COLOR,
                    )
                    mask_drawn_on_video_frame_flag = True

                if not mask_drawn_on_video_frame_flag:
                    ball_box_vid = current_ball_info_video_item.get("box")
                    ball_center_vid = current_ball_info_video_item.get("center")
                    ball_radius_vid = current_ball_info_video_item.get(
                        "radius"
                    )  # 1. 일단 현재 프레임의 반지름을 가져옴
                    is_occlusion_recovered_vid = current_ball_info_video_item.get(
                        "occlusion_recovered", False
                    )
                    is_ball_corrected_on_impact_frame = (
                        frame_idx_vid == final_impact_idx
                    ) and (
                        final_impact_info.get("is_ball_info_corrected", False)
                        if final_impact_info
                        else False
                    )

                    # --- 👇👇👇 핵심 수정 로직 👇👇👇 ---
                    # 2. 만약 현재 프레임이 '임팩트 프레임'이고,
                    #    '백스윙 시점의 공 반지름' 정보가 유효하다면
                    if (
                        frame_idx_vid == final_impact_idx
                        and final_impact_info
                        and "backswing_ball_radius" in final_impact_info
                        and pd.notna(final_impact_info["backswing_ball_radius"])
                    ):
                        # 3. 임팩트 시점의 불안정한 반지름 대신, 더 정확한 백스윙 시점의 값으로 덮어쓴다!
                        ball_radius_vid = final_impact_info["backswing_ball_radius"]
                    # --- 👆👆👆 핵심 수정 로직 끝 👆👆👆 ---

                    box_color_vid_val = (
                        (255, 100, 100)
                        if is_ball_corrected_on_impact_frame
                        else (
                            (255, 165, 0)
                            if is_predicted_vid
                            else (
                                (0, 0, 255)
                                if is_occlusion_recovered_vid
                                else (0, 255, 255)
                            )
                        )
                    )
                    if ball_box_vid:
                        cv2.rectangle(
                            processed_frame_for_video,
                            (int(ball_box_vid[0]), int(ball_box_vid[1])),
                            (int(ball_box_vid[2]), int(ball_box_vid[3])),
                            box_color_vid_val,
                            2,
                        )
                    # 4. 이제 여기서 사용하는 ball_radius_vid는 임팩트 시점에 보정된 값입니다.
                    if ball_center_vid and ball_radius_vid and ball_radius_vid > 0:
                        cv2.circle(
                            processed_frame_for_video,
                            tuple(map(int, ball_center_vid)),
                            int(ball_radius_vid),
                            box_color_vid_val,
                            1,
                        )
                ball_center_vid_pt = current_ball_info_video_item.get("center")
                if ball_center_vid_pt:
                    cv2.circle(
                        processed_frame_for_video,
                        tuple(map(int, ball_center_vid_pt)),
                        3,
                        (0, 0, 255),
                        -1,
                    )

            text_y_offset_vid = 20
            line_height_vid = 20
            font_scale_vid = 0.5
            font_thickness_vid = 1
            text_color_default = (230, 230, 230)
            bg_color_default_rgba = (0, 0, 0, 120)

            def draw_text_on_video_with_alpha_bg(
                img,
                text,
                y_pos,
                x_pos=10,
                font_color=(230, 230, 230),
                font_scale=0.5,
                font_thickness=1,
                bg_color_rgba=(0, 0, 0, 120),
            ):
                (text_w, text_h), baseline = cv2.getTextSize(
                    text, cv2.FONT_HERSHEY_SIMPLEX, font_scale, font_thickness
                )

                rect_x1, rect_y1 = x_pos - 4, y_pos - text_h - 5
                rect_x2, rect_y2 = x_pos + text_w + 4, y_pos + baseline

                rect_x1 = max(0, rect_x1)
                rect_y1 = max(0, rect_y1)
                rect_x2 = min(img.shape[1] - 1, rect_x2)
                rect_y2 = min(img.shape[0] - 1, rect_y2)

                if rect_x1 < rect_x2 and rect_y1 < rect_y2:
                    if bg_color_rgba[3] > 0:
                        sub_img = img[rect_y1:rect_y2, rect_x1:rect_x2]
                        if sub_img.size > 0:
                            bg_rect = np.full(
                                sub_img.shape,
                                (bg_color_rgba[0], bg_color_rgba[1], bg_color_rgba[2]),
                                dtype=np.uint8,
                            )
                            res = cv2.addWeighted(
                                sub_img,
                                1 - (bg_color_rgba[3] / 255.0),
                                bg_rect,
                                (bg_color_rgba[3] / 255.0),
                                0,
                            )
                            img[rect_y1:rect_y2, rect_x1:rect_x2] = res

                cv2.putText(
                    img,
                    text,
                    (x_pos, y_pos - 2),
                    cv2.FONT_HERSHEY_SIMPLEX,
                    font_scale,
                    font_color,
                    font_thickness,
                    cv2.LINE_AA,
                )
                return y_pos + line_height_vid

            text_y_offset_vid = draw_text_on_video_with_alpha_bg(
                processed_frame_for_video,
                f"Frame: {frame_idx_vid} (Orig: {frame_idx_vid * sampling_interval_calc})",
                text_y_offset_vid,
                bg_color_rgba=bg_color_default_rgba,
            )

            if current_analysis_row_text_item is not None:
                k_foot_vid = current_analysis_row_text_item.get("kicking_foot", "N/A")
                k_part_vid = current_analysis_row_text_item.get(
                    "kicking_foot_part", "N/A"
                )
                text_y_offset_vid = draw_text_on_video_with_alpha_bg(
                    processed_frame_for_video,
                    f"Kicking Foot: {k_foot_vid} ({k_part_vid})",
                    text_y_offset_vid,
                    bg_color_rgba=bg_color_default_rgba,
                )

                foot_ball_dist_px_vid = current_analysis_row_text_item.get(
                    "distance_smoothed"
                )
                if pd.notna(foot_ball_dist_px_vid) and foot_ball_dist_px_vid != float(
                    "inf"
                ):
                    dist_cm_vid = foot_ball_dist_px_vid * current_scale_vid
                    text_y_offset_vid = draw_text_on_video_with_alpha_bg(
                        processed_frame_for_video,
                        f"Foot-Ball Dist: {dist_cm_vid:.1f}cm",
                        text_y_offset_vid,
                        bg_color_rgba=bg_color_default_rgba,
                    )

                foot_speed_px_fr_vid = current_analysis_row_text_item.get(
                    "kicking_foot_speed"
                )
                if pd.notna(foot_speed_px_fr_vid):
                    speed_km_h_vid = (
                        foot_speed_px_fr_vid
                        * current_scale_vid
                        * analyzed_fps_for_video_val
                    ) * CM_S_TO_KM_H
                    text_y_offset_vid = draw_text_on_video_with_alpha_bg(
                        processed_frame_for_video,
                        f"K-Foot Spd: {speed_km_h_vid:.1f}km/h",
                        text_y_offset_vid,
                        bg_color_rgba=bg_color_default_rgba,
                    )

                angle_text_y_current = text_y_offset_vid
                angle_text_x_left = 10
                angle_text_x_right = frame_width - 160

                angles_to_display_vid = {
                    "L-Hip": current_analysis_row_text_item.get("left_hip_angle"),
                    "L-Knee": current_analysis_row_text_item.get("left_knee_angle"),
                    "L-Ankle": current_analysis_row_text_item.get("left_ankle_angle"),
                    "R-Hip": current_analysis_row_text_item.get("right_hip_angle"),
                    "R-Knee": current_analysis_row_text_item.get("right_knee_angle"),
                    "R-Ankle": current_analysis_row_text_item.get("right_ankle_angle"),
                }

                next_y_l, next_y_r = angle_text_y_current, angle_text_y_current

                if pd.notna(angles_to_display_vid["L-Hip"]):
                    next_y_l = draw_text_on_video_with_alpha_bg(
                        processed_frame_for_video,
                        f"L-Hip: {int(angles_to_display_vid['L-Hip'])}",
                        next_y_l,
                        x_pos=angle_text_x_left,
                        font_color=(0, 165, 255),
                        bg_color_rgba=bg_color_default_rgba,
                    )
                if pd.notna(angles_to_display_vid["L-Knee"]):
                    next_y_l = draw_text_on_video_with_alpha_bg(
                        processed_frame_for_video,
                        f"L-Knee: {int(angles_to_display_vid['L-Knee'])}",
                        next_y_l,
                        x_pos=angle_text_x_left,
                        font_color=(255, 0, 255),
                        bg_color_rgba=bg_color_default_rgba,
                    )
                if pd.notna(angles_to_display_vid["L-Ankle"]):
                    next_y_l = draw_text_on_video_with_alpha_bg(
                        processed_frame_for_video,
                        f"L-Ankle: {int(angles_to_display_vid['L-Ankle'])}",
                        next_y_l,
                        x_pos=angle_text_x_left,
                        font_color=(0, 128, 255),
                        bg_color_rgba=bg_color_default_rgba,
                    )

                if pd.notna(angles_to_display_vid["R-Hip"]):
                    next_y_r = draw_text_on_video_with_alpha_bg(
                        processed_frame_for_video,
                        f"R-Hip: {int(angles_to_display_vid['R-Hip'])}",
                        next_y_r,
                        x_pos=angle_text_x_right,
                        font_color=(255, 200, 100),
                        bg_color_rgba=bg_color_default_rgba,
                    )
                if pd.notna(angles_to_display_vid["R-Knee"]):
                    next_y_r = draw_text_on_video_with_alpha_bg(
                        processed_frame_for_video,
                        f"R-Knee: {int(angles_to_display_vid['R-Knee'])}",
                        next_y_r,
                        x_pos=angle_text_x_right,
                        font_color=(100, 255, 200),
                        bg_color_rgba=bg_color_default_rgba,
                    )
                if pd.notna(angles_to_display_vid["R-Ankle"]):
                    next_y_r = draw_text_on_video_with_alpha_bg(
                        processed_frame_for_video,
                        f"R-Ankle: {int(angles_to_display_vid['R-Ankle'])}",
                        next_y_r,
                        x_pos=angle_text_x_right,
                        font_color=(200, 100, 255),
                        bg_color_rgba=bg_color_default_rgba,
                    )

                text_y_offset_vid = max(next_y_l, next_y_r)

            ball_kin_row_vid = df_ball_kinematics[
                df_ball_kinematics["original_idx"] == frame_idx_vid
            ]
            if not ball_kin_row_vid.empty:
                ball_speed_px_fr_vid = ball_kin_row_vid["ball_speed"].iloc[0]
                # ...
                if pd.notna(ball_speed_px_fr_vid) and ball_speed_px_fr_vid > 0.1:
                    # ✅ 수정 1: '공 속도' 변수인 ball_speed_px_fr_vid를 사용
                    # ✅ 수정 2: 계산 결과를 올바른 변수인 ball_speed_km_h_vid에 저장
                    ball_speed_km_h_vid = convert_speed_to_kmh_normalized(
                        ball_speed_px_fr_vid,
                        current_scale_vid,
                        analyzed_fps_for_video_val,
                    )
                    text_y_offset_vid = draw_text_on_video_with_alpha_bg(
                        processed_frame_for_video,
                        f"Ball Spd: {ball_speed_km_h_vid:.1f}km/h",  # 😊 이제 정상적으로 사용 가능
                        text_y_offset_vid,
                        bg_color_rgba=bg_color_default_rgba,
                    )

            if (
                frame_idx_vid == final_impact_idx
                and final_impact_info is not None
                and final_impact_info
            ):
                imp_text_y_offset_vid = frame_height - 150
                imp_scale_vid = 0.5
                imp_color_vid = (0, 255, 255)
                imp_bg_color_rgba = (0, 0, 0, 180)

                imp_text_y_offset_vid = draw_text_on_video_with_alpha_bg(
                    processed_frame_for_video,
                    "--- IMPACT FRAME ---",
                    imp_text_y_offset_vid,
                    font_color=imp_color_vid,
                    font_scale=imp_scale_vid,
                    bg_color_rgba=imp_bg_color_rgba,
                )
                contact_region_vid = final_impact_info.get(
                    "contact_region_on_ball", "N/A"
                )
                imp_text_y_offset_vid = draw_text_on_video_with_alpha_bg(
                    processed_frame_for_video,
                    f"Ball Contact: {contact_region_vid}",
                    imp_text_y_offset_vid,
                    font_color=imp_color_vid,
                    font_scale=imp_scale_vid,
                    bg_color_rgba=imp_bg_color_rgba,
                )

                max_ball_speed_final_px_fr = final_impact_info.get(
                    "max_ball_speed_px_fr"
                )
                if pd.notna(max_ball_speed_final_px_fr):
                    max_ball_speed_final_km_h = (
                        max_ball_speed_final_px_fr
                        * current_scale_vid
                        * analyzed_fps_for_video_val
                    ) * CM_S_TO_KM_H
                    imp_text_y_offset_vid = draw_text_on_video_with_alpha_bg(
                        processed_frame_for_video,
                        f"Max Ball Spd: {max_ball_speed_final_km_h:.1f}km/h",
                        imp_text_y_offset_vid,
                        font_color=imp_color_vid,
                        font_scale=imp_scale_vid,
                        bg_color_rgba=imp_bg_color_rgba,
                    )

                if (
                    pd.notna(max_foot_speed_overall_px_fr_vid)
                    and max_foot_speed_overall_px_fr_vid > 0
                ):
                    imp_text_y_offset_vid = draw_text_on_video_with_alpha_bg(
                        processed_frame_for_video,
                        f"Max Foot Spd: {max_foot_speed_km_h_vid:.1f}km/h",
                        imp_text_y_offset_vid,
                        font_color=imp_color_vid,
                        font_scale=imp_scale_vid,
                        bg_color_rgba=imp_bg_color_rgba,
                    )

                dist_support_ball_px_vid = final_impact_info.get(
                    "dist_ball_to_supporting_foot_ankle"
                )
                if pd.notna(dist_support_ball_px_vid):
                    dist_support_cm_vid = dist_support_ball_px_vid * current_scale_vid
                    imp_text_y_offset_vid = draw_text_on_video_with_alpha_bg(
                        processed_frame_for_video,
                        f"SupportFt-Ball: {dist_support_cm_vid:.1f}cm",
                        imp_text_y_offset_vid,
                        font_color=imp_color_vid,
                        font_scale=imp_scale_vid,
                        bg_color_rgba=imp_bg_color_rgba,
                    )

                cv2.rectangle(
                    processed_frame_for_video,
                    (0, 0),
                    (frame_width - 1, frame_height - 1),
                    (0, 255, 255),
                    3,
                )

            out_video.write(processed_frame_for_video)
            if frame_idx_vid % 50 == 0 and frame_idx_vid > 0:
                print(
                    f"      ... Writing video frame {frame_idx_vid+1}/{len(all_processed_frames)} ..."
                )

        # === 마지막 프레임에 총점 정보 추가 시작 ===
        if score_result_global and all_processed_frames:
            last_frame_for_score = all_processed_frames[-1].copy()
            fh_score, fw_score, _ = last_frame_for_score.shape
            font_score = cv2.FONT_HERSHEY_DUPLEX
            font_scale_title = 1.2
            font_scale_details = 0.8
            font_thickness = 1
            text_color_title = (80, 255, 80)
            text_color_details = (255, 255, 255)
            bg_color_score_rgba = (0, 0, 0, 180)

            title_text_score = "KICK ANALYSIS SCORE"
            (w_title, h_title), base_title = cv2.getTextSize(
                title_text_score, font_score, font_scale_title, font_thickness
            )

            total_s = score_result_global.get("total_score", 0)
            max_s = score_result_global.get("max_score", 100)
            percentage_s = score_result_global.get("percentage", 0)
            grade_s = "N/A"
            if percentage_s >= 90:
                grade_s = "A+ (Excellent)"
            elif percentage_s >= 80:
                grade_s = "A (Very Good)"
            elif percentage_s >= 70:
                grade_s = "B+ (Good)"
            elif percentage_s >= 60:
                grade_s = "B (Above Average)"
            elif percentage_s >= 50:
                grade_s = "C (Average)"
            elif percentage_s >= 40:
                grade_s = "D (Below Average)"
            else:
                grade_s = "F (Needs Improvement)"

            score_text_line1 = (
                f"Total Score: {total_s} / {max_s}  ({percentage_s:.1f}%)"
            )
            score_text_line2 = f"Overall Grade: {grade_s}"
            (w_line1, h_line1), base_line1 = cv2.getTextSize(
                score_text_line1, font_score, font_scale_details, font_thickness
            )
            (w_line2, h_line2), base_line2 = cv2.getTextSize(
                score_text_line2, font_score, font_scale_details, font_thickness
            )

            num_detail_lines = 0
            max_detail_width = 0
            if "categories" in score_result_global:
                for cat_name, cat_data in score_result_global["categories"].items():
                    num_detail_lines += 1
                    for detail_name, detail_val in cat_data["details"].items():
                        num_detail_lines += 2
                        temp_text = f"  - {detail_name.replace('_', ' ').title()}: {detail_val['score']}/{detail_val['max_score']} ({str(detail_val['value'])[:50]})"  # 값 길이 제한
                        (w_temp, _), _ = cv2.getTextSize(
                            temp_text, font_score, 0.6, font_thickness
                        )
                        if w_temp > max_detail_width:
                            max_detail_width = w_temp

            text_block_height = (
                (h_title + base_title)
                + (h_line1 + base_line1)
                + (h_line2 + base_line2)
                + (num_detail_lines * 20)
                + 100
            )
            text_block_width = (
                max(w_title, w_line1, w_line2, max_detail_width + 50) + 60
            )

            rect_x1 = max(10, (fw_score - text_block_width) // 2)
            rect_y1 = max(10, (fh_score - text_block_height) // 2)
            rect_x2 = min(fw_score - 10, rect_x1 + text_block_width)
            rect_y2 = min(fh_score - 10, rect_y1 + text_block_height)

            if rect_x1 < rect_x2 and rect_y1 < rect_y2:
                sub_img_score = last_frame_for_score[rect_y1:rect_y2, rect_x1:rect_x2]
                if sub_img_score.size > 0:
                    bg_rect_score = np.full(
                        sub_img_score.shape,
                        (
                            bg_color_score_rgba[0],
                            bg_color_score_rgba[1],
                            bg_color_score_rgba[2],
                        ),
                        dtype=np.uint8,
                    )
                    res_score = cv2.addWeighted(
                        sub_img_score,
                        1 - (bg_color_score_rgba[3] / 255.0),
                        bg_rect_score,
                        (bg_color_score_rgba[3] / 255.0),
                        0,
                    )
                    last_frame_for_score[rect_y1:rect_y2, rect_x1:rect_x2] = res_score

            current_y = rect_y1 + h_title + 20
            cv2.putText(
                last_frame_for_score,
                title_text_score,
                (rect_x1 + (text_block_width - w_title) // 2, current_y),
                font_score,
                font_scale_title,
                text_color_title,
                font_thickness + 1,
                cv2.LINE_AA,
            )
            current_y += h_line1 + base_line1 + 20
            cv2.putText(
                last_frame_for_score,
                score_text_line1,
                (rect_x1 + (text_block_width - w_line1) // 2, current_y),
                font_score,
                font_scale_details,
                text_color_details,
                font_thickness,
                cv2.LINE_AA,
            )
            current_y += h_line2 + base_line2 + 10
            cv2.putText(
                last_frame_for_score,
                score_text_line2,
                (rect_x1 + (text_block_width - w_line2) // 2, current_y),
                font_score,
                font_scale_details,
                text_color_details,
                font_thickness,
                cv2.LINE_AA,
            )
            current_y += 30

            if "categories" in score_result_global:
                for cat_name, cat_data in score_result_global["categories"].items():
                    category_title_text = f"{cat_name.replace('_', ' ').title()}: {cat_data['subtotal']}/{sum(d.get('max_score',0) for d in cat_data['details'].values())}"
                    (w_cat_title, h_cat_title), _ = cv2.getTextSize(
                        category_title_text, font_score, 0.7, font_thickness
                    )
                    if current_y + h_cat_title < rect_y2:
                        cv2.putText(
                            last_frame_for_score,
                            category_title_text,
                            (rect_x1 + 20, current_y),
                            font_score,
                            0.7,
                            text_color_title,
                            font_thickness,
                            cv2.LINE_AA,
                        )
                        current_y += h_cat_title + 10
                        for detail_name, detail_val in cat_data["details"].items():
                            if current_y + (h_cat_title * 2) < rect_y2:
                                detail_text = f"  - {detail_name.replace('_', ' ').title()}: {detail_val['score']}/{detail_val['max_score']}"
                                value_text = (
                                    f"    Value: {str(detail_val['value'])[:50]}"
                                )
                                cv2.putText(
                                    last_frame_for_score,
                                    detail_text,
                                    (rect_x1 + 30, current_y),
                                    font_score,
                                    0.6,
                                    text_color_details,
                                    font_thickness,
                                    cv2.LINE_AA,
                                )
                                current_y += h_cat_title
                                cv2.putText(
                                    last_frame_for_score,
                                    value_text,
                                    (rect_x1 + 30, current_y),
                                    font_score,
                                    0.5,
                                    text_color_details,
                                    font_thickness - 1 if font_thickness > 1 else 1,
                                    cv2.LINE_AA,
                                )
                                current_y += h_cat_title
                            else:
                                break
                    else:
                        break

            score_card_duration_frames = int(output_fps_vid * 3)
            for _ in range(score_card_duration_frames):
                out_video.write(last_frame_for_score)
        # === 마지막 프레임에 총점 정보 추가 끝 ===

        out_video.release()
        print(f"Result video saved to: {output_video_path}")
        try:
            from IPython.display import HTML, display
            from base64 import b64encode

            mp4 = open(output_video_path, "rb").read()
            data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
            file_name = os.path.basename(output_video_path)
            html_code = f"""
            <h3>Analysis Result Video</h3>
            <p>You can view the results via the video player below and save it by clicking the link.</p>
            <video width=720 controls style="border: 1px solid #ccc; max-width: 100%;">
                    <source src="{data_url}" type="video/mp4"> Your browser does not support the video tag.
            </video><br><br>
            <a href="{data_url}" download="{file_name}" style="display: inline-block; padding: 10px 20px; font-size: 16px; font-weight: bold; color: white; background-color: #4CAF50; text-align: center; text-decoration: none; border-radius: 5px; cursor: pointer;">
                <strong>▶ Click here to download the video ({file_name})</strong>
            </a><p><small><i>(If clicking the link doesn't start the download, try right-clicking the link and selecting 'Save Link As...')</i></small></p>"""
            display(HTML(html_code))
            print(f"\nResult video displayed above. File path: '{output_video_path}'")
            if IN_COLAB:
                print("\nGoogle Colab environment. Attempting to download the file...")
                try:
                    from google.colab import files

                    files.download(output_video_path)
                except Exception as e_download:
                    print(
                        f"Colab auto-download failed: {e_download}. Please use the 'Click here to download' link above."
                    )
        except ImportError:
            print(
                f"\nIPython.display not found. Video saved at '{output_video_path}'. Please open manually."
            )
        except Exception as e_display:
            print(
                f"\nError displaying video: {e_display}. Video saved at '{output_video_path}'. Please open manually."
            )
# --- 결과 비디오 생성 ---
if not all_processed_frames:
    print("Error: No processed frames available to generate result video.")
else:
    print(
        "\n--- Starting result video generation (v-Final with Occlusion-Aware Logic) ---"
    )

    # 비디오 설정
    frame_height, frame_width, _ = all_processed_frames[0].shape
    analyzed_fps_for_video = (
        original_video_fps / sampling_interval_calc
        if sampling_interval_calc > 0 and SAMPLING_RATE_TARGET_FRAMES > 0
        else original_video_fps
    )
    if analyzed_fps_for_video <= 0:
        analyzed_fps_for_video = 30.0

    fourcc = cv2.VideoWriter_fourcc(*"mp4v")
    out_video = cv2.VideoWriter(
        output_video_path, fourcc, analyzed_fps_for_video, (frame_width, frame_height)
    )

    if not out_video.isOpened():
        print(f"Error: Could not open VideoWriter for path: {output_video_path}")
    else:
        # --- ✨ 안정적인 공 시각화를 위한 핵심 변수 ---
        last_known_good_radius = (
            final_impact_info.get("backswing_ball_radius")
            if final_impact_info
            else None
        )
        if not pd.notna(last_known_good_radius):
            if pixel_to_cm_scale_global > 1e-6:
                last_known_good_radius = (
                    REAL_BALL_DIAMETER_CM / pixel_to_cm_scale_global
                ) / 2.0
            else:
                last_known_good_radius = 15
        print(
            f"Starting video render with initial stable radius: {last_known_good_radius:.2f}px"
        )

        # 모든 프레임을 순회하며 비디오 생성
        for frame_idx_vid, original_frame_vid in enumerate(all_processed_frames):
            frame_to_draw = original_frame_vid.copy()
            current_scale_vid = get_safe_pixel_to_cm_scale(pixel_to_cm_scale_global)
            current_pose = pose_results_list[frame_idx_vid]
            ball_info = ball_detections_list[frame_idx_vid]
            current_analysis_row = analysis_dataframe_global[
                analysis_dataframe_global["original_idx"] == frame_idx_vid
            ]

            # 1. 포즈 그리기
            if current_pose:
                mp.solutions.drawing_utils.draw_landmarks(
                    frame_to_draw,
                    current_pose,
                    mp_pose.POSE_CONNECTIONS,
                    landmark_drawing_spec=mp.solutions.drawing_utils.DrawingSpec(
                        color=(245, 117, 66), thickness=2, circle_radius=3
                    ),
                    connection_drawing_spec=mp.solutions.drawing_utils.DrawingSpec(
                        color=(245, 66, 230), thickness=2
                    ),
                )

            # 2. 공 그리기 (개선된 상황별 로직 적용)
            if ball_info:
                ball_center = ball_info.get("center")
                current_detected_radius = ball_info.get("radius")

                # --- ✨ 최종: IoU 기반의 상황별 공 반지름 결정 로직 ✨ ---
                radius_to_draw = (
                    last_known_good_radius  # 기본적으로는 마지막 정상 값을 사용
                )

                # 현재 프레임의 발과 공의 정보를 가져옴
                is_occluded_by_foot = False
                if not current_analysis_row.empty:
                    kicking_foot = current_analysis_row.iloc[0].get("kicking_foot")
                    if kicking_foot and kicking_foot != "N/A":
                        foot_poly = get_foot_polygon(
                            current_pose, kicking_foot, frame_to_draw.shape
                        )
                        if foot_poly is not None:
                            x, y, w, h = cv2.boundingRect(foot_poly)
                            foot_box = [x, y, x + w, y + h]
                            ball_box = ball_info.get("box")
                            if ball_box:
                                iou = calculate_iou(foot_box, ball_box)
                                if iou > 0.01:  # 1% 이상 겹치면 가려진 것으로 판단
                                    is_occluded_by_foot = True

                # 반지름 결정
                if frame_idx_vid == final_impact_idx:
                    # [상황 1: 임팩트 순간] -> 가장 신뢰도 높은 백스윙 값 사용
                    radius_to_draw = final_impact_info.get(
                        "backswing_ball_radius", last_known_good_radius
                    )
                elif pd.notna(current_detected_radius):
                    # [상황 2: 발에 가려졌을 때] -> 마지막 정상 값 사용
                    if is_occluded_by_foot:
                        radius_to_draw = last_known_good_radius
                    # [상황 3: 가려지지 않고, 정상적으로 탐지되었을 때] -> 현재 값 사용하고, 정상 값 업데이트
                    else:
                        radius_to_draw = current_detected_radius
                        last_known_good_radius = current_detected_radius

                # 최종 결정된 반지름으로 원을 그립니다.
                if ball_center and pd.notna(radius_to_draw):
                    cv2.circle(
                        frame_to_draw,
                        tuple(map(int, ball_center)),
                        int(radius_to_draw),
                        (0, 255, 255),
                        3,
                    )

            # 3. 텍스트 정보 그리기 (헬퍼 함수)
            text_y_offset_vid = 20
            line_height_vid = 22

            def draw_text_on_video_with_alpha_bg(
                img,
                text,
                y_pos,
                x_pos=10,
                font_color=(230, 230, 230),
                font_scale=0.5,
                font_thickness=1,
                bg_color_rgba=(0, 0, 0, 150),
            ):
                (text_w, text_h), baseline = cv2.getTextSize(
                    text, cv2.FONT_HERSHEY_SIMPLEX, font_scale, font_thickness
                )
                rect_x1, rect_y1 = x_pos - 5, y_pos - text_h - 5
                rect_x2, rect_y2 = x_pos + text_w + 5, y_pos + baseline - 2

                sub_img = img[rect_y1:rect_y2, rect_x1:rect_x2]
                bg_rect = np.full(
                    sub_img.shape,
                    (bg_color_rgba[0], bg_color_rgba[1], bg_color_rgba[2]),
                    dtype=np.uint8,
                )
                res = cv2.addWeighted(
                    sub_img,
                    1 - (bg_color_rgba[3] / 255.0),
                    bg_rect,
                    (bg_color_rgba[3] / 255.0),
                    0,
                )
                img[rect_y1:rect_y2, rect_x1:rect_x2] = res

                cv2.putText(
                    img,
                    text,
                    (x_pos, y_pos - 4),
                    cv2.FONT_HERSHEY_SIMPLEX,
                    font_scale,
                    font_color,
                    font_thickness,
                    cv2.LINE_AA,
                )
                return y_pos + line_height_vid

            # 4. 실시간 정보 표시 (모든 정보 복원)
            current_analysis_row = analysis_dataframe_global.loc[
                analysis_dataframe_global["original_idx"] == frame_idx_vid
            ]
            if not current_analysis_row.empty:
                current_analysis_row_item = current_analysis_row.iloc[0]

                # --- 발 속도 ---
                foot_speed = current_analysis_row_item.get("kicking_foot_speed")
                if pd.notna(foot_speed):
                    speed_kmh = convert_speed_to_kmh_normalized(
                        foot_speed, current_scale_vid, analyzed_fps_for_video
                    )
                    text_y_offset_vid = draw_text_on_video_with_alpha_bg(
                        frame_to_draw,
                        f"Foot Speed: {speed_kmh:.1f} km/h",
                        text_y_offset_vid,
                    )

                # --- 모든 각도 ---
                for side in ["left", "right"]:
                    for joint in ["hip", "knee", "ankle"]:
                        angle_val = current_analysis_row_item.get(
                            f"{side}_{joint}_angle"
                        )
                        if pd.notna(angle_val):
                            text_y_offset_vid = draw_text_on_video_with_alpha_bg(
                                frame_to_draw,
                                f"{side.capitalize()} {joint.capitalize()}: {angle_val:.1f} deg",
                                text_y_offset_vid,
                            )

            # 5. 임팩트 / 백스윙 프레임 특별 표시 (상세 정보 포함)
            if frame_idx_vid == final_impact_idx and final_impact_info:
                cv2.rectangle(
                    frame_to_draw,
                    (0, 0),
                    (frame_width - 1, frame_height - 1),
                    (0, 255, 255),
                    5,
                )
                cv2.putText(
                    frame_to_draw,
                    "IMPACT",
                    (10, frame_height - 100),
                    cv2.FONT_HERSHEY_DUPLEX,
                    2,
                    (0, 0, 255),
                    3,
                    cv2.LINE_AA,
                )

                # --- 임팩트 시점 상세 정보 표시 ---
                info_y = frame_height - 80
                support_dist = final_impact_info.get(
                    "dist_ball_to_supporting_foot_ankle"
                )
                if pd.notna(support_dist):
                    info_y = draw_text_on_video_with_alpha_bg(
                        frame_to_draw,
                        f"Support Foot Dist: {support_dist*current_scale_vid:.1f}cm",
                        info_y,
                        x_pos=10,
                        font_scale=0.6,
                        font_color=(0, 255, 255),
                    )

                max_ball_speed = final_impact_info.get("max_ball_speed_px_fr")
                if pd.notna(max_ball_speed):
                    speed_kmh = convert_speed_to_kmh_normalized(
                        max_ball_speed, current_scale_vid, analyzed_fps_for_video
                    )
                    info_y = draw_text_on_video_with_alpha_bg(
                        frame_to_draw,
                        f"Max Ball Speed: {speed_kmh:.1f} km/h",
                        info_y,
                        x_pos=10,
                        font_scale=0.6,
                        font_color=(0, 255, 255),
                    )

            elif final_impact_info and frame_idx_vid == final_impact_info.get(
                "backswing_original_idx", -1
            ):
                cv2.rectangle(
                    frame_to_draw,
                    (0, 0),
                    (frame_width - 1, frame_height - 1),
                    (0, 255, 0),
                    5,
                )
                cv2.putText(
                    frame_to_draw,
                    "BACKSWING",
                    (10, frame_height - 100),
                    cv2.FONT_HERSHEY_DUPLEX,
                    2,
                    (0, 255, 0),
                    3,
                    cv2.LINE_AA,
                )

            out_video.write(frame_to_draw)

        # 6. 마지막 프레임에 상세 점수 카드 추가
        score_to_display, title_for_score_card = None, "KICK ANALYSIS SCORE"
        if score_result_refined_global:
            score_to_display, title_for_score_card = (
                score_result_refined_global,
                "REFINED SCORE (Interpolated)",
            )
        elif score_result_global:
            score_to_display = score_result_global

        if score_to_display:
            last_frame_for_score = all_processed_frames[-1].copy()
            fh, fw, _ = last_frame_for_score.shape

            font = cv2.FONT_HERSHEY_SIMPLEX

            all_lines = []
            all_lines.append(
                {
                    "text": title_for_score_card,
                    "scale": 1.2,
                    "color": (80, 255, 80),
                    "offset": 0,
                }
            )

            total_s = score_to_display.get("total_score", 0)
            percentage_s = score_to_display.get("percentage", 0)
            score_line = f"Total Score: {total_s:.1f} / 100 ({percentage_s:.1f}%)"
            all_lines.append(
                {
                    "text": score_line,
                    "scale": 1.0,
                    "color": (255, 255, 255),
                    "offset": 15,
                }
            )

            if "categories" in score_to_display:
                for cat_name, cat_data in score_to_display["categories"].items():
                    cat_total = sum(
                        d.get("max_score", 0) for d in cat_data["details"].values()
                    )
                    cat_title = f"\n--- {cat_name.replace('_', ' ').title()} ({cat_data['subtotal']:.1f}/{cat_total}) ---"
                    all_lines.append(
                        {
                            "text": cat_title,
                            "scale": 0.8,
                            "color": (0, 255, 255),
                            "offset": 10,
                        }
                    )
                    for detail_name, detail_val in cat_data["details"].items():
                        detail_line = f"  - {detail_name.replace('_', ' ').title()}: {detail_val['score']}/{detail_val['max_score']}"
                        all_lines.append(
                            {
                                "text": detail_line,
                                "scale": 0.7,
                                "color": (255, 255, 255),
                                "offset": 25,
                            }
                        )
                        value_line = f"    Value: {str(detail_val['value'])[:50]}"
                        all_lines.append(
                            {
                                "text": value_line,
                                "scale": 0.6,
                                "color": (200, 200, 200),
                                "offset": 40,
                            }
                        )

            # Calculate text block size
            total_h = (
                sum(
                    cv2.getTextSize(line["text"], font, line["scale"], 1)[0][1] + 20
                    for line in all_lines
                )
                + 40
            )

            rect_x1, rect_y1 = 30, (fh - total_h) // 2
            rect_x2, rect_y2 = fw - 30, rect_y1 + total_h

            overlay = last_frame_for_score.copy()
            cv2.rectangle(
                overlay, (rect_x1, rect_y1), (rect_x2, rect_y2), (0, 0, 0), -1
            )
            cv2.addWeighted(
                overlay, 0.75, last_frame_for_score, 0.25, 0, last_frame_for_score
            )

            current_y = rect_y1 + 50
            for line in all_lines:
                if line["text"].startswith("\n"):
                    current_y += 10
                cv2.putText(
                    last_frame_for_score,
                    line["text"].strip(),
                    (rect_x1 + line["offset"] + 20, current_y),
                    font,
                    line["scale"],
                    line["color"],
                    1,
                    cv2.LINE_AA,
                )
                current_y += int(35 * line["scale"]) + 10

            score_card_duration = int(analyzed_fps_for_video * 5)
            for _ in range(score_card_duration):
                out_video.write(last_frame_for_score)

        out_video.release()
        print(f"✅ Result video generation complete: {output_video_path}")

        # ... (프로그램 종료 및 자원 해제 로직) ...
        try:
            from IPython.display import HTML, display
            from base64 import b64encode

            mp4 = open(output_video_path, "rb").read()
            data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
            file_name = os.path.basename(output_video_path)
            html_code = f"""
            <h3>Analysis Result Video</h3>
            <p>You can view the results via the video player below and save it by clicking the link.</p>
            <video width=720 controls style="border: 1px solid #ccc; max-width: 100%;">
                    <source src="{data_url}" type="video/mp4"> Your browser does not support the video tag.
            </video><br><br>
            <a href="{data_url}" download="{file_name}" style="display: inline-block; padding: 10px 20px; font-size: 16px; font-weight: bold; color: white; background-color: #4CAF50; text-align: center; text-decoration: none; border-radius: 5px; cursor: pointer;">
                <strong>▶ Click here to download the video ({file_name})</strong>
            </a><p><small><i>(If clicking the link doesn't start the download, try right-clicking the link and selecting 'Save Link As...')</i></small></p>"""
            display(HTML(html_code))
            print(f"\nResult video displayed above. File path: '{output_video_path}'")
        except ImportError:
            print(
                f"\nIPython.display not found. Video saved at '{output_video_path}'. Please open manually."
            )
        except Exception as e_display:
            print(
                f"\nError displaying video: {e_display}. Video saved at '{output_video_path}'. Please open manually."
            )

# 최종 contact_region_on_ball 확인 (주로 fallback 시 필요할 수 있음)
if final_impact_idx != -1 and final_impact_info is not None and final_impact_info:
    if (
        "contact_region_on_ball" not in final_impact_info
        or pd.isna(final_impact_info.get("contact_region_on_ball"))
        or final_impact_info.get("contact_region_on_ball") == "N/A"
    ):
        fp_contact_check, bc_contact_check = final_impact_info.get(
            "foot_pos"
        ), final_impact_info.get("ball_center")
        if (
            fp_contact_check
            and bc_contact_check
            and isinstance(fp_contact_check, tuple)
            and isinstance(bc_contact_check, tuple)
            and len(fp_contact_check) == 2
            and len(bc_contact_check) == 2
        ):
            final_impact_info["contact_region_on_ball"] = get_ball_contact_region(
                bc_contact_check[0] - fp_contact_check[0],
                bc_contact_check[1] - fp_contact_check[1],
            )
        else:
            final_impact_info["contact_region_on_ball"] = "N/A"
elif final_impact_idx == -1:
    print(
        "No impact detected, cannot generate detailed final summary or video annotations related to impact."
    )
    if final_impact_info is None:
        final_impact_info = {}
    final_impact_info["contact_region_on_ball"] = "N/A"

print("\n--- Kick Analysis Program End ---")

# 프로그램 종료 후 자원 해제 (선택적)
if "pose_model" in locals() and pose_model is not None:
    pose_model.close()
if "yolo_model" in locals() and yolo_model is not None:
    del yolo_model
if "cap" in locals() and cap is not None and cap.isOpened():
    cap.release()
# out_full은 이전에 사용된 디버깅용 비디오 객체로, 현재 메인 로직에서는 사용되지 않으므로 제거해도 됩니다.
# if 'out_full' in locals() and out_full is not None and out_full.isOpened():
#     out_full.release()
if "out_video" in locals() and out_video is not None and out_video.isOpened():
    out_video.release()
if (
    "temp_cap_for_scale" in locals()
    and temp_cap_for_scale is not None
    and temp_cap_for_scale.isOpened()
):
    temp_cap_for_scale.release()
cv2.destroyAllWindows()